In \[1\]:

    library(tidyverse)
    library(optimx)

    Warning message:
    "package 'tidyverse' was built under R version 3.5.3"-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
    √ ggplot2 3.3.0     √ purrr   0.3.3
    √ tibble  2.1.3     √ dplyr   0.8.5
    √ tidyr   1.0.2     √ stringr 1.4.0
    √ readr   1.3.1     √ forcats 0.5.0
    Warning message:
    "package 'ggplot2' was built under R version 3.5.3"Warning message:
    "package 'tibble' was built under R version 3.5.3"Warning message:
    "package 'tidyr' was built under R version 3.5.3"Warning message:
    "package 'readr' was built under R version 3.5.3"Warning message:
    "package 'purrr' was built under R version 3.5.3"Warning message:
    "package 'dplyr' was built under R version 3.5.3"Warning message:
    "package 'stringr' was built under R version 3.5.3"Warning message:
    "package 'forcats' was built under R version 3.5.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
    x dplyr::filter()  masks stats::filter()
    x purrr::flatten() masks jsonlite::flatten()
    x dplyr::lag()     masks stats::lag()
    Warning message:
    "package 'optimx' was built under R version 3.5.3"

Part1. Challenger 우주왕복선 O-ring 고장확률예측<a href="#Part1.-Challenger-우주왕복선-O-ring-고장확률예측" class="anchor-link">¶</a>
=====================================================================================================================================

-   Logistic -\> logit
-   Normal -\> probit
-   Gompertz -\> gompit

In \[2\]:

    oring <- read.csv("oring.csv", na.strings="*")
    names(oring) <- c("id", "n", "y", "x")
    oring<-data.frame(x=oring$x, y=oring$y, n=oring$n)
    oring<-na.omit(oring)

(a) AIC로부터 최적 예측모형 구하기<a href="#(a)-AIC로부터-최적-예측모형-구하기" class="anchor-link">¶</a>
---------------------------------------------------------------------------------------------------------

In \[3\]:

    # Inf 처리 위한 초기값 설정 
    p<-rep(0,length(oring$y)) 
    for(i in 1:length(oring$y)){ 
      if(oring$y[i]==0){ 
        p[i]<-0.1 
      }else if(oring$y[i]==1){ 
        p[i]<-0.5 
      }else{ p[i]<-0.9 } }

In \[4\]:

    # G(z) 
    mylogit<- glm(cbind(y,6-y) ~ x, data=oring, family=binomial(link = "logit"))
    myprobit<- glm(cbind(y,6-y) ~ x, data=oring, family=binomial(link = "probit"))
    mygompit<- glm(cbind(y,6-y) ~ x, data=oring, family=binomial(link = "cloglog"))

In \[5\]:

    # alpha, beta 
    par1 <- mylogit$coeff
    par2 <- myprobit$coeff
    par3 <- mygompit$coeff
    par1; par2; par3

(Intercept)  
5.08497723236888

x  
-0.11560116666022

(Intercept)  
2.23452259704643

x  
-0.0554984149180786

(Intercept)  
4.71422003697337

x  
-0.110763095632717

In \[6\]:

    # G(a+bx)
    trans <- function(par,x) par[1]+par[2]*x[,1]
    p.logit <- function(par,x) 1/(1+exp(-trans(par,x)))
    p.normal<-function(par,x) pnorm(trans(par,x)) 
    p.gomp<-function(par,x) 1-exp(-exp(trans(par,x)))

    q.gomp<- function(p) log(-log(1-p))

In \[7\]:

    # log-likelihood 
    logit.loglik <- function(par,x) -2*sum(x[,2]*log(p.logit(par,x)) + (x[,3]-x[,2])*log(1-p.logit(par,x)))
    norm.loglik <- function(par,x) -2*sum(x[,2]*log(p.normal(par,x)) + (x[,3]-x[,2])*log(1-p.normal(par,x)))
    gomp.loglik <- function(par,x) -2*sum(x[,2]*log(p.gomp(par,x)) + (x[,3]-x[,2])*log(1-p.gomp(par,x)))

In \[8\]:

    # MLE
    logit.mle<-optimx(par=par1, fn=logit.loglik, x=oring)
    norm.mle<-optimx(par=par2, fn=norm.loglik, x=oring)
    gomp.mle<-optimx(par=par3, fn=gomp.loglik, x=oring)

    Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method Nelder-Mead"Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method BFGS"Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method Nelder-Mead"Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method BFGS"Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method Nelder-Mead"Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method BFGS"

In \[9\]:

    # AIC 
    AIC.list <- rbind(logit.mle$value[1], norm.mle$value[1], gomp.mle$value[1])

    # AIC table 
    AIC.table <- cbind(AIC.list, rep(4,3), AIC.list+rep(4,3))
    colnames(AIC.table) <- c("-2loglik","2p","AIC")
    rownames(AIC.table) <- c("logit","probit","gompit")

    AIC.table
    names(which.min(AIC.table[,3]))

|        | -2loglik | 2p  | AIC      |
|--------|----------|-----|----------|
| logit  | 60.39634 | 4   | 64.39634 |
| probit | 60.62131 | 4   | 64.62131 |
| gompit | 60.32405 | 4   | 64.32405 |

'gompit'

AIC가 64.324인 Gompertz 모형이 최적 예측모형이다.

(b) 적절한 예측모형 선택하기<a href="#(b)-적절한-예측모형-선택하기" class="anchor-link">¶</a>
---------------------------------------------------------------------------------------------

### H0: beta=0 vs H1: beta != 0<a href="#H0:-beta=0-vs-H1:-beta-!=-0" class="anchor-link">¶</a>

In \[11\]:

    par4 <- c(mean(q.gomp(p)),0); par4
    gomp.opt2<-hjn(par=par4, fn=gomp.loglik, x=oring, bdmsk=c(1,0)) 
    print(gomp.opt2)

1.  -1.57262508475702
2.  0

    $par
    [1] -2.696498  0.000000

    $value
    [1] 66.54037

    $counts
    [1] 57 NA

    $convergence
    [1] 0

In \[12\]:

    # AIC table 
    AIC.table2 <- matrix(c(gomp.opt2$value, gomp.mle$value[1], 
                        c(2,4), rep(0,2)), ncol=3)
    AIC.table2[,3]<-AIC.table2[,1]+AIC.table2[,2]
    colnames(AIC.table2) <- c("-2loglik","2p","AIC")
    rownames(AIC.table2) <- c("H0","H1")

    AIC.table2
    names(which.min(AIC.table2[,3]))

|     | -2loglik | 2p  | AIC      |
|-----|----------|-----|----------|
| H0  | 66.54037 | 2   | 68.54037 |
| H1  | 60.32405 | 4   | 64.32405 |

'H1'

H1에서 AIC값이 더 작기 때문에 beta != 0 인 모형이 적절한 예측모형이다.

(c) 모수들의 추정오차, 신뢰구간 구하기<a href="#(c)-모수들의-추정오차,-신뢰구간-구하기" class="anchor-link">¶</a>
-----------------------------------------------------------------------------------------------------------------

In \[13\]:

    gomp.opt<-optim(par=par3, fn=gomp.loglik, x=oring, method="Nelder-Mead", hessian = TRUE) 
    gomp.hess<-sqrt(c(solve(gomp.opt$hessian)[1,1], solve(gomp.opt$hessian)[2,2])); gomp.hess

1.  1.98119753059996
2.  0.030809812720911

In \[15\]:

    CI.alpha<-c(gomp.opt$par[1] - 1.96*gomp.hess[1], gomp.opt$par[1] + 1.96*gomp.hess[1]) 
    CI.beta<-c(gomp.opt$par[2] - 1.96*gomp.hess[2], gomp.opt$par[2] + 1.96*gomp.hess[2])

    cat("CI.alpha: [", CI.alpha[1], ",", CI.alpha[2], "] \n")
    cat("CI.beta: [", CI.beta[1], ",", CI.beta[2], "]")

    CI.alpha: [ 0.8310729 , 8.597367 ] 
    CI.beta: [ -0.1711503 , -0.05037586 ]

(d) x\*=31F<a href="#(d)-x*=31F" class="anchor-link">¶</a>
----------------------------------------------------------

In \[17\]:

    par31<-gomp.opt$par
    p31<-p.gomp(par31, as.matrix(31))
    cat("p(31) = ", p31, "\n") # o-ring의 고장 확률 
    cat("E(31) = ", 6*p31) # o-ring의 고장 개수

    p(31) =  0.9726399 
    E(31) =  5.835839

-   E(y\|31)이 5.83이므로 6개의 O-ring중 거의 모든 O-ring이 고장난다고
    볼 수 있다.
-   따라서 기존의 자료와 비교했을 때 Temperature가 낮을수록 더 고장이 잘
    난다고 생각할 수 있다.

Part2. Leukemia 환자 생존수명 예측<a href="#Part2.-Leukemia-환자-생존수명-예측" class="anchor-link">¶</a>
=========================================================================================================

In \[18\]:

    leukemia<-read.csv("leukemia.csv", header=TRUE) 
    colnames(leukemia)<-c("y", "x")

(a) (lny, lnx)<a href="#(a)-(lny,-lnx)" class="anchor-link">¶</a>
-----------------------------------------------------------------

### LSE에 대한 잔차의 Normal Q-Q Plot, Gompertz Q-Q Plot 그리기<a href="#LSE에-대한-잔차의-Normal-Q-Q-Plot,-Gompertz-Q-Q-Plot-그리기" class="anchor-link">¶</a>

In \[21\]:

    # (lny, lnx) 산점도 
    leukemia<-as.tibble(leukemia) %>% mutate(lny=log(y),lnx=log(x))
    ggplot(leukemia,aes(lnx,lny))+geom_point()

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAANlBMVEUAAAAzMzNNTU1oaGh8%0AfHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD////agy6EAAAACXBIWXMA%0AABJ0AAASdAHeZh94AAAaBElEQVR4nO3d4VYU2RlG4WOpcUbjxL7/m42IEtoA3cipql279/fD%0AlazIkxdwT9Mto+PUdd2bb+w9oOsMV0hdN+EKqesmXCF13YQrpK6bcIXUdROukLpuwhVS1024%0A14b0n+/344cZF7SZE7SKU0hHgXCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHc%0AIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHc%0AIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHc%0AIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHc%0AIDFUSGIIN0gMwUIaY6y4/NYg3CAxxAppjOtLwn0oeRBukBhChTTGK0rCfSh5EG6QGCokMYQb%0AJIYKSQzhBokhVEg9R5oL4QaJoZVDWu7u+pB61W4qhBskhtYO6XWPSCsvvzUIN0gMFZIYwg0S%0AQ+uGdN5RIW0M4QaJoZVDeniK9O7uLv30rrvJu+4R6TUvNqz7j4Bbg3CDxNAGL38X0tbQr9c+%0AMYNuACokH/Twu3GUQbcAbfBiQyFtC/3v+0Mgg24CWj+kR6/czRwe9NwV0h7QBt/ZUEjbQoW0%0AB7TBc6RC2hjqOdIOUCEJoV612x4qJDGEGySGCkkM4QaJoUJ6O4T9Sgo3SAwV0oxnJNDn9rhB%0AYqiQprxGxny1GTdIDBVSIQXt4xTS2RVS0J85hXR+PUcKKqQZUK/aBRWSGcINEkOFJIZwg8RQ%0AIYkh3CAxVEhiCDdIDBWSGMINEkOFJIZwg8RQIYkh3CAxVEhi6DXOi399Ae4940GFJIZe4bz8%0AF+rg3jMeVEhi6Hrnwl/xhnvPeFAhiaFC2g4qJDFUSNtBhSSGeo60HVRIYqhX7baDCkkM4QaJ%0AoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJ%0AoUISQ3/gPP0NDrj3jAcVkhh6vfPMt9zh3jMeVEhi6NXOc98EjnvPeFAhiaFC2g4qJDFUSNtB%0AhSSGeo60HVRIYqhX7baDCkkM4QaJoUISQ7hBYqiQxBBukBgqJDGEGySGCokETf7LaEHvmR4q%0AJBA0+69H57xnfqiQONDDb4cW0vGgQuJAhXRgqJA4UCEdGCokENRzpONChUSCetXusFAhiSHc%0AIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHc%0AIDFUSGIIN0gMFZIYwg0SQ5uG1HXdE9cjEhvCDRJDfWknhnCDxFAhiSHcIDFUSGIIN0gMFZIY%0Awg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIY%0Awg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIY%0Awg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIY%0Awg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIY%0Awg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4X0amiMMQd69grpeNBvzrO/SB6/yW2H%0ANMZzHyTcu4YbJIbOned/kTx+k5sOaYxnP0i4dw03SAydOS/8Inn8JoVUSEEvOYV0+Qop6KJT%0ASFdcz5GCLjo9R7rietUu6KLTq3YmCDdIDPX7SGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFU%0ASGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFU%0ASGIIN0gMFZIYuuxc893+V0FXnhcqJDF00bnq3z+7Brr2vFAhiaFLznX/RvQV0NXnhdYPaSmk%0AvaBC2g5aPaSlkHaDCmk7aO2Qlh6R9oN6jrQdtHJIS1/a7Qj1qt120FYhvbu7iz+9627xLoa0%0AnHpE2hHCDRJDqz4iLQ8/FNIeEG6QGFo3pPsrpJ0g3CAx1O8jiSHcIDFUSGIIN0gMFZIYwg0S%0AQ+uH9OhmDg/azglaxSmko0C4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAY%0AKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAY%0AKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAY%0AKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAY%0AKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCOhQ0xpjivPKC1nAK%0AaTdojFeVdKD37PBQIR0IGuN1JR3nPTs+VEgHggqJCxXSgaBC4kKFdCSo50hYqJAOBfWqHRUq%0AJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgq%0AJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgq%0AJDGEGySGCkkM4QaJoUISQ7hBYmjTkLque+J6RGJDuEFiqC/txBBukBgqJDGEGySGCkkM4QaJ%0AoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJ%0AoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJ%0AoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJoUISQ7hBYqiQxBBukBgqJDGEGySGCkkM4QaJ%0AoUISQ7hBYqiQxBBu0AxojDEHur9CCtrMIUFj/CgJtOiPnUI6CoQb9HZo3B9o0Z87hXQUCDeo%0AkM7epJAOAuEGFdLZmxTSQSDcoJ4jnb1JIR0Ewg3qVbuzNymkg0C4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbtB06P5fl50A%0AvfkKSQzhBs2Gfv4BDm+H3n6FJIZwgyZDv/5IoTdDE66QxBBuUCGdvUkhHQTCDSqkszcppINA%0AuEE9Rzp7k0I6CIQb1Kt2Z29SSAeBcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDK4e0fL9C2gvC%0ADRJD64a0PPxQSDtAuEFiqJDEEG6QGNrgOVIh7QXhBomhrUJ6d3dX/fSuu7W7IqRebNgRwg0S%0AQ31pJ4Zwg8RQIYkh3CAx1Kt2Ygg3SAwVkhjCDRJDfWeDGMINEkMbPEcqpL0g3CAxVEhiCDdI%0ADBWSGMINEkOFJIZwg8RQIYkh3CAxVEhiCDdIDBWSGMINEkOFJIZwg8RQIYkh3CAxVEhiCDdI%0ADBWSGMINEkOFJIZwg8RQIYkh3CAxVEhiCDdIDBWSGMINEkOFJIZwg8RQIYkh3CAxVEhiCDdI%0ADBWSGMINEkOFJIZwg8RQIYkh3CAxVEhiCDdIDBXSkaELf6n3gd+zw0GFdGBojJdLOu57djyo%0AkI4LjXGhpMO+ZweECum4UCGBoEI6LlRIIKiQDgz1HIkDFdKRoV61w0CFJIZwg8RQIYkh3CAx%0AVEhiCDdIDBWSGMINEkOFJIZwg8RQIYkh3CAxVEhiCDdIDBWSGMINEkOFJIZwg8RQIYkh3CAx%0AVEhiCDdIDBWSGMINEkOFJIZwg8RQIYkh3CAxVEhiCDdIDBWSGMINEkOFJIZwg8RQIYkh3CAx%0AVEhiCDdIDBWSGMINEkMzQ3r/178LiQThBomhmSGNMZZ/fSkkDIQbJIZmhvTt88e7P2jtw+d/%0ACgkB4QaJodnPkb58Wr639P7px6WZw4O2c4JWcV4M6fTPpx9//ueHQtofwg0SQ5ND+vrxx8PR%0Avz+Mj4W0O4QbJIamhvTlw8NXdeOpR6uZw4O2c4JWcZ4N6f0YH7/++p+WQtodwg0SQ1Nf/v70%0A9fTizRwetJ0TtIrzbEjfXs6okLaGcIPE0NTnSD9e+r67QkJAuEFiaGZIn8YoJBKEGySGZoa0%0AjL9f+LqukDaHcIPE0NQXGy59N/jM4UHbOUGrOM+G9HFcfLmh67r/u99C+mf58Ny3q97fzH8C%0ABG3nBK3iPBvS6MUGFoQbJIYKSQzhBomhqb+PdOlmDg/azglaxSmko0C4QWJoWkjj8RUSAsIN%0AEkOFJIZwg8RQX9qJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjC%0ADRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjC%0ADRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjC%0ADRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjC%0ADRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjC%0ADRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjC%0ADRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjC%0ADRJDhSSGcIPEUCGJIdwgMVRIYgg3SAytHNLy/QppLwg3SAytG9Ly8EMh7QDhBomhQhJDuEFi%0AaIPnSIW0F4QbJIa2Cund3V3107vu1u6qkHqxYTcIN0gMrf+IVEi7QbhBYmj1kB51VEgbQ7hB%0AYmjtkB53VEgbQ7hBYmjt35A9+28zhwdt5wSt4rwipGU5+9aGmcODtnOCVnFe84h0fjOHB23n%0ABK3iFNJRINwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSG%0AcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSG%0AcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSG%0AcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSG%0AcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSG%0AcIPEUCGJIdwgMVRIYgg3SAwVkhjCDRJDhSSGcIPEUCGJIdwgEDTGmAP9vEISQ7hBHGiMnyUV%0AUtBmjg8a41dJhRS0meODCulNy28Nwg3CQIX0puW3BuEGcaCeI71l+a1BuEEgqFft3rD81iDc%0AIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHc%0AIDFUSGIIN0gMFZIYwg0SQ5uG1HXdE9cjEhvCDRJDfWknhnCDxFAhiSHcIDFUSGIIN0gMFZIY%0Awg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMFZIY%0A2m3Qrz998c3Qs4eDCkkM7TXo4c8Dfiv0/OGgQhJDOw36359Q/0bohcNBhSSGCmk7qJDEUCFt%0ABxWSGOo50nZQIYmhXrXbDiokMYQbJIYKSQzhBomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzh%0ABomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzhBomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzh%0ABomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzhBomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzh%0ABomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzhBomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzh%0ABomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzhBomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzh%0ABomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzhBomhQhJDuEFiqJDEEG6QGCokMYQbJIYKSQzh%0ABomh1UNaCmk3CDdIDK0d0lJI+0G4QWJo5ZCWHpF2hHCDxFBf2okh3CAxtFVI7+7uip/edbd3%0APSKxIdwgMdSXdmIIN0gMFZIYwg0SQ4UkhnCDxFAhiSHcIDFUSGIIN0gMrR7S45s5PGg7J2gV%0Ap5COAuEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBsk%0AhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWJo05C6rnviekRiQ7hB%0AYqgv7cQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4%0AQWJo5ZCW71dIe0G4QWJo3ZCWhx8KaQcIN0gM/eaMMS6/SSEdBMINEkPnzhhXlFRIR4Fwg8TQ%0AmTPGNSX9UUjv7u7ST+86xf0M6fqff+kn9Ii0K4QbJIa2ekQqpB0g3CAx1HMkMYQbJIZ61U4M%0A4QaJoX4fSQzhBomhvrNBDOEGiaGVQzq/mcODtnOCVnEK6SgQbpAYKiQxhBskhgpJDOEGiaFC%0AEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFC%0AEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFC%0AEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFCEkO4QWKokMQQbpAYKiQxhBskhgpJDOEGiaFN%0AQ5p6vL9IE7cIN6hFT18hnR1uEW5Qi56+Qjo73CLcoBY9fYV0drhFuEEtevp2DanrLFdIXTfh%0ACqnrJlwhdd2EK6Sum3CF1HUTbseQlu+33//70wcbxPsQ8RadGJ+1/UJaHn7gHOwXCe9DxFt0%0AgnzWCunRLbg9J9iHiLeI8lkrpLOj7TkRJ7EWLYxBhXR2tD0n4iTWokJ6+AF0tD28RbQXG5YT%0A42NUSGdH20NchJqE+VVUSGdH28MbdGJtWu5v7xmF9Nu15+VDftIQgwrp7GB7YHOgnzTEoL6z%0A4exYgyhftjw63KC7Iyzqe+26bsIVUtdNuELquglXSF034Qqp6yZcIXXdhCukrptwhdR1E66Q%0Aum7CFdKRbvTpol6fmSNdIWGvz8yRrpCw12fmSHcX0hj/fBzLp9Pp4/h6On0dH/Ye1d1dIR3p%0A7kNaxvf7dPo23p9OH+5q6va/QjrS3Yf04dvp77GcTn+NL5+/B9URrpCOdD+/tPv5n5j/ctCN%0AXiEd6e5D+vWfTp/H+Lzzou7nFdKRrpCwV0hHut9CWt6/70s7yBXSke48pL/Gly/jr703dT+u%0AkI50ZyH9ePn7/fi296ju7grpSHcW0s/fkP2496ju7gqp6yZcIXXdhCukrptwhdR1E66Qum7C%0AFVLXTbhC6roJV0hdN+EKqesmXCF13YQrpK6bcIXUdRPuv7p1ySyp03vmAAAAAElFTkSuQmCC%0A)

In \[23\]:

    # LSE
    lm_leuk<-lm(lny~lnx,data=leukemia)
    alphahat<-lm_leuk$coefficients[1]
    betahat<-lm_leuk$coefficients[2]
    c(alphahat,betahat)

(Intercept)  
5.42539580261603

lnx  
-0.817800667958474

In \[24\]:

    # residual Q-Q Plot 
    leukemia<-leukemia %>% mutate(e=lny-alphahat-betahat*lnx,r=rank(e),
                                  probit=qnorm(r/(nrow(leukemia)+1)),
                                  gompit=log(-log(1-r/(nrow(leukemia)+1))))

In \[25\]:

    ggplot(leukemia,aes(probit,e))+geom_point()+geom_smooth(method="lm",se=F)+
      ggtitle("Normal Q-Q plot \n R-squared:", summary(lm(e~probit, data=leukemia))$r.squared)

    `geom_smooth()` using formula 'y ~ x'

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzMzZv9NTU1o%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///9uyY2wAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2diXrquBIGdcaQkI0E3v9hBy8YG7xoaclquf7v%0A3pMEEEVb1AjLRjZXQkhwzNYvgJASgkiECASRCBEIIhEiEEQiRCCIRIhAEIkQgSASIQJBJEIE%0AgkiECMRTJGO++t88n2D419ebMebt6+VRc7fPsS/vXi+GkNB4i1T1v3k+weP334NpU51Hj5m7%0AffJJQl4MIaHxFsm833/zfIL+19/KvP3cfn4fjRkaM3f75JMEvRhCQuMt0lv35g4X6WBO3W8n%0Acxg8ZO72ySeZuYGQNPEW6dK9ubv37vm9MtX7ubvvYN7qOz4qc/i+Xj9vPz6bR51uH9aOP4Nm%0At3zfHnvPwXxfV26/Nbw93/H78SQPdPM50K8gQoLiLdL103x2v13rd32b72s7Wt0GE2M+mpt+%0A3psf9YOr4YPuT/U+kucxWzBz+/1pT/cnGaARiWwVf5Guh+rS/XY9397el+vlZMxvfcuxvaP6%0Arj+VVe2P2/j1YT6u9b/H61Ckg7n0T3vp5zBmb7897dfNnqr+ZFk/yRParxxCAhMg0rkZJpr3%0A7qnbnXmvf94GofYh9YhyaX+05rVqNC0eb/nRm3/wx8ztjTC3VKf21ie0XzmEBCZApNt7txsV%0Aboq07+7feuDp3s1moM1jV+r74xgo0rH9+Xa42/mKJiR1QkS6VIfn8aX+ZazO8Mdnt5M0bDH6%0ACHe92XB/yPPtT78dzIj29AchaRMiUjPf4CDS523v6fT1+yTScFKh/rR4F+n59juhE+mISCSj%0ABIlUzzcsfLR7/nEwj5nv4fR3+2Ht51Lv8Axn6iZvv0+K89GO5JQwkW4jRTfZ0I4Ywz3+1x/9%0ATPl19JY/NtMF59tw9TU68Dp9e7frdXlMNryiCUmdMJGu7+0ez+3j2qmdgz4vjkifzcT1k0i/%0AVXuQ9qN6OUVo4vZ2Vv32LJf2SZ7Qv371EBKWQJEu1dQB2eFDxvtIbX7GY0d/cqqpfoaQydub%0Aw71D0hB94IAs2SaBIl2/pk4Rus78+Kwf89Oc+zN+w3/Xchw/vg+D04Jmbr81vI0+Q9IAfT4M%0AjugSki6Z/Qf887R2O0MOyTHq3paIRHKMurclIpEco+5tiUgkx/C2JEQgiESIQBCJEIEEiXQ/%0AXno4XdYf7PP0MZ6VkAgREcmYKoZJiETUJFCk9ufP0cwcSA0KIhE1ERHpeo5yZg4iETWREWn8%0Anj+/m/qcuub3enXHj9fv+w3X5bq0Z9LdhrVula2mVfXx/MUlQvJNhBHpZ3A69qn57eNZpNG6%0AXM3iXfXZr/dVtrpW74hE1ERmH6ka7iMdmgX2my/jnZvF9r+qJ5HG63I1i3edmwHqfKzlalt1%0AX1wiREOEZu2Oo1v7X9/bRSS/nkQar8vVfMI7dV98rT/lndpWlwqRiJbIHEf6HN76dttB+uoW%0An7vr8bImyvO6XP13+B6e3Z4o5MURkjDhH+1uA8d391frwW/Vy3UfnJ5Fel2XywxEurc6IhLR%0AEoF9pJ9+adX70tvfzXLfH7MiTazLNdwfGrYiREUkJhs+Jk5sOL/XE3ndA54/2k2sy1WN1jd5%0AtCJERURm7Q5m4pKT9Z1Va8xnK1Jt2/fMulzv7VOc62mL7tJLn4hE1EREpPNoHa12+vtUz799%0ANpeO+Go+8R3N26Wb055Yl+vcHEI6N/tbw1aEqIjMAdn34cqO3QHZqp64aw+tvtUPbG8+za3L%0A9T04IHt8HMZlR4moiNApQpUZzID/NKcItRPgP2+mOrUP/Dn05/1Mrcv1e+ov7Hf94hQhoitp%0A3qbYQAoPIhEiEEQiRCCIRIhAeIcTIhBEIkQgiESIQBCJEIEgEiECQSRCBIJIhAgEkQgRCCIR%0AIhBEIkQgiESIQBCJEIF4ilQv1P320/95Ppqqu0bS5+GxzF23ntbjwV+DO/vVtmZaHJ9bDB93%0Aah/3+/5YQvy+hNHjNkKSxU+kS/u+vS/UcH58t7xZiKv7umy3esnjwV+DO/u1TSZafPdLgw9a%0ADB936pzpv7B+7kV63EZIuviJ9FmvrfDerOBdp1mZuLntXK/k/dWu4HDuhojHg+tluL679fbv%0A9061aJbr+qlve7QYPu69e9x7/cyn+rbvfvHxx22EpIufSM2CWefmaizNkzTPUq+kdWpWEGpz%0A6FbBn3xwf+9Ui+G/918Gj6uqn/a+qn/cZ39vNW5LSJL4veGqiXf6tV1z6/f+mA/z9fx2r9ON%0AHf29Uy3e2hHpbdhi8LjT08Ksh3oc+n4zh/PoNkLSxU+kpyHjWH+0O3VLP97e0PVa4PUQNDG+%0AvLV6jO59adGu4tV9WOtaPB43QteefdYPavI7vI2QdBERqVmy7tiKdOx29avqMiXSqb083/De%0A1xatF92A1LV4PG6Evv5W7VWWmoUl34e3EZIuIiJdv29v9m6B71P9hj7Wn7WuUyLVE3Gn8b2v%0ALU795EXf4vG46+jZRs50t+IRSZ4gkUZXYP6t3+T9/tDLVVoeDx7fa9niaU+rXyy5er3E2RmP%0ASPL4iXQYT8Q1+amHjLcJLR4Pbq87Nr53qsVjDOtbPB53Hfz8ur+ETrPD8DZC0sVPpFO9N/Le%0A79Efbp/KLm/tIdfTYF/l/kHt/uBTfTDp6d6pFm/1ZPZnO51+bzH1uJ/+kpvdkazP4W2EpIuf%0ASN2JBPfR4vN5/ftu9ux+yPX+4EvVnwHR37vQ4jxqMfG4Yz+GdVcJHN1GSLoEnGt3fFwtbHAa%0A3Kkyx/vxHPN85lx9Htz77/jeqRbnN2Pezk8tXh/3+DB4/b21OF3GtxGSLLzhCBEIIhEiEEQi%0ARCCIRIhAEIkQgSASIQJBJEIEgkiECASRCBEIIhEiEEQiRCBeIv15xrshOHBZ4hAJHDgBHCKB%0AAyeAQyRw4ARwiAQOnAAOkcCBE8AhEjhwAjhEAgdOAIdI4MAJ4BAJHDgBHCKBAyeAQyRw4ARw%0AiAQOnAAOkcCBE8AhEjhwAjhEAgdOAIdI4MAJ4BAJHDgBHCKBAyeAQyRw4ARwiAQOnAAOkcCB%0AE8AhEjhwAjhEAgdOAIdI4MAJ4BAJHDgBHCKBAyeAQyRw4ARwiAQOnAAOkcCBW8EZY9bbIRI4%0AcIs4YyxMQiRw4BZxxtiYhEjgwC3iEAkcOAFcEpEIKT6NRw4P92HIeR814MAF4Ji1AwcuEQ6R%0AwIETwCESOHACOEQCB04Ah0jgwAngEAkcOAEcIoEDJ4BDJHDgBHCIBA6cAA6RwIETwCESOHAC%0AOEQCB04Ah0jgwAngEAkcOAEcIoEDJ4BDJHDgBHCIBA6cAA6RwIETwCESOHACOEQCB04Ah0jg%0AwAngEAkcOAEcIoEDJ4BDJHDgBHCIBA6cAA6RwIETwCESOHACOEQCB04Ah0jgwAngEAkcOAEc%0AIoEDJ4BDJHDgBHCIBA6cAA6RwIETwCESOHACOEQCB04Ah0jgwAngEAkcOAEcIoEDJ4BDJHDg%0ABHCIBA6cAA6RwIETwCESOHACOEQCB04Ah0jgwAngEAkcOAEcIoEDJ4BDJHDgVnD/2bRDJHDg%0AFnH/IRI4cMG4//6zMQmRwIFbwP1n5xEigQO3gLP1CJHAgZvHWXuESODAzeKsNUIkcODmcPbD%0A0R8igQM3g3PyCJHAgZvEuXmESODATeHcNEIkcOCmcK4eIRI4cK84Z48QCRy4F5y7R4gEDtwz%0AzsMjd5GqWxAJXLk4x+m6eztHkar+H0QCVyLOzyNEAgduGD+NPPeREAlcmTjP4egvRKR/dezb%0AEJJ9Go9Cn8ReJCYbwBWJ8x+P/EYkRAJXIq7RyBfnIdLAI0QCVwquG47SiTT0CJHAFYK7f6xL%0AJtLII0QCVwau3z1KJVJVjU5t8H3dRXYGOK24wSxDwn0kRiRwZeGGs3WIBA6cX0az3ogEDpxX%0AxgePEAkcOI88H4RFJHDg3PNyMgMigQPnnNeTghAJHDjXTJxbh0jgwDlm6hxVRAIHzi2T53oj%0AEjhwTpn+zgQigQPnkpnvHiESOHD2mf0OHyKBA2ed+e/CIhI4cLZZ+Eo5IoEDZ5fFpRkQCRw4%0AqywvcYJI4MDZZGWpIEQCB84iaytuIRI4cKtZX7kOkcCBW4vFCpCIBA7cSmxWUkUkcOAWY7cg%0AMSKBA7cUy4W9EQkcuIXYLpCPSODAzcf6OhOIBA7cXByu14JI4MDNxOW6R4gEDtx0nK4fhkjg%0AwE3G7TJ8iAQO3FQcL2eJSODATcT1srCIBA7ca5wvr4xI4MC9xP0y5YgEDtxTnKbrAnGIBK5U%0AnI9HiAQO3Dg+GiESOHCjeA1H/jhEAlckztcjRAIH7hFvjxAJHLg+3hohEjhw9/gPR164rh0i%0AgSsLF+QRIoED1yTMI0QCB+4vWCNEAgfuT8AjRAIHTsAjRAIHLlwjRAK3e5zAcOSCe26HSOCK%0AwMl4hEjg9o0T8giRwO0aJ6QRIoHbNU7MI0QCt2OcnEeIBG6/OEGPEAncbnGSHiESuJ3ipKbr%0ALHGz7RAJnGacsEeIBG6XOGGNEAncHnHSw9EKbrEdIoHTiovgESKB2x0uhkdbiUTIVqk12vo1%0ATIURCZwiXJThaB633g6RwCnExfIIkcDtCRfNI0QCtx9cPI0QCdx+cDE9QiRwe8FF9QiRwO0E%0AF1UjRAK3D1zc4egF59AOkcDpwUX3CJHA7QAX3yNEAlc+Lr5GiASufFwKjxAJXOm4JB4hErjC%0AcWk8QiRwZeMSeYRI4IrGpfIIkcCVjEvmESKBKxiXziNEAlcsLsFh2CHOsx0igcsbl9QjRAJX%0AKC6tR4gErkxcWo0QCVyRuMTD0R8igSsRl94jRAJXHq7xSEl1iAQuU1w3HCmpDpHA5Ym7f6xT%0AUh0igcsS1+8eKakOkcDliHvMMiipDpHA5Ycbzta54owxIWhEAlcMbjTr7YgzJswkRAJXCm58%0A9MgNZ0ygSYgErhDc00FYRIrxcsEVj3s+mQGRYrxccKXjXk4KYh8pxssFVzju9eQ6Zu1ivFxw%0AZeMmTlJVUh0igcsGN3myt5LqEAlcLrjpL00oqQ6RwGWCe9Lovq+jpDpEApcF7nk46mfflFSH%0ASOBywE161JikpDpEApcB7mX3CJHivlxwReJeZxkQKe7LBVcgbnK2jn2kqC8XXHm4maWCmLWL%0A+XLBacZNnr2ztuSWkuoQCVwq3NT5pOsr1ympDpHAJcJNfcPBYgVIJdUhErhEuAmRbFZSVVId%0AIoFLhHsVyWpBYiXVIRK4VLgnjywX9lZSHSKBS4bz8UhLdYgEbhOc9YUmlFSHSOC2wNlfr0VJ%0AdYgEbgOcw3WPlFSHSODS41yuH6akOkQClxzndB0+JdUhErjUOLfrWSqpDpHApcW5XhdWSXWI%0ABC4pzvn6ykqqQyRwKXHulylXUh0igUuHcx6OwnA+SShShUjgvOLjkZbq3EWqEAmcV7w80lKd%0As0gVIxI4r3hppKY69xEJkcB5xG848sZ5J71I/+rYtiE7T+PR1i8iTRiRwEXDeY9HKqpr2iES%0AuNi4AI0UVNe1QyRwkXFBHmVf3b0dIoGLiwvzKPfq+naIBC4qLkyj3Kt7tEMkcBFxgcORKy48%0AnGsHLkNcuEc5Vzdqh0jgouEEPMq4unE7RAIXCyegUcbVPbVDJHCRcCIeZVvdcztEAhcHJ+NR%0ArtW9tEMkcFFwQh5lWt1rO0QCFwMn5VGe1U20QyRw8jiJ6ToHnGAQCVw+OEGPMqxuuh0igZPG%0ACWqUYXUz7RAJnCxOcjiywAkHkcDlgRP2KLPq5tshEjhJnLRHeVW30A6RwAnipDXKq7qldogE%0ATgwnPhwt42IEkcBtjovhUT7VrbRDJHBCuCgeZVPdWjtEAieCi6NRLtWtt0MkcBK4WB7lUZ1F%0AO0QCJ4CL5lEW1dm0QyRw4bhoGmVRnVU7RAIXios3HE3iogaRwG2Fi+rR5tXZtkMkcGG4uB5t%0AXZ11O0QCF4SLq9HW1dm3QyRwIbjYHmnZmIgELgQX3SMtGxORwAXg4nukZWMiEjh/XAKPtGxM%0ARALni4s8XfeMSxNEApcYl8YjLRsTkcD54dJopGZjIhI4H1yi4ehPzcZEJHAeuHQeadmYiATO%0AHZfQIy0bE5HAOeNeNDLGRMQlDSKBS5TX4ciYiCYp2ZiIBM4tMx5FM0nJxkQkcE6Z2D1CpD9E%0AAueUyVkGRPpDJHAuaTx6xbGPhEjgHNKORxM4Zu0QCZx1uo91hVYXiEMkcHbpd4+KrC4Yh0jg%0ArPKYZiixunAcIoGzyWC6rsDqBHCIBM4iw1nv8qqTwCESuPWMjh4VV50IDpHArcSY8VHYsqqT%0AwiESuOU8e1RWdWI4RAK3mNaj4fHWkqqTwyESuMU003UGkVbbIRK4hXQeIdJqO0QCN59Oo9GZ%0AdOVUJ4lDJHCzaY/CPp+RWkp1sjhEAjeXmSVOCqlOGIdI4GYyt1RQGdVJ4xAJ3HRmV9wqojpx%0AHCKBm8rCynUFVBcBh0jgJrK0AqT+6mLgEAncaxZXUlVfXRRcoEikwLRLnBDPMCKBa7K2sLfu%0A6mLhEAncOKsL5KuuLhoOkcCNsn6dCc3VxcMhErhBbK7Xore6mDhEAveI1XWP1FYXFYdIe8cN%0Azkm1u36YquqS4RBp57jBut2Wl+HTVF06HCLtGze4koTt5SwVVZcQh0j7xj1Esr4srKLqEuIQ%0Aad+4XiT7yysrqi4hDpF2jnP2SFV16XCItHdc7ZHddJ0IzjlKcIgEzs0jddWlwSFS6bj1q+k5%0AaZRZddngEKlw3Or1Xd2GozWcfJTgEKls3OoVx509yqm6jHCIVDZuTSR3j3KqLiMcIpWNWxHJ%0AXaOsqssIh0iF45Y88hiO1nDyUYJDpNJx0h7lVV02OETaLc7TIyXVpcYh0k5xvhrpqC49DpH2%0AifP3SEN1G+AQaZe4AI8UVLcFDpH2iAvQSEF1m+AQaX+4kOHIAxcYJThE2h0u0KPMq9sKh0h7%0Aw4V6lHd1m+EQaWe4UI3yrm47HCKpx61/4WiQcI/K3piItFvc6heOhhHwqOiNiUi7xa1+4WgY%0ACY9K3pgBOERSjnMRScSjkjdmAA6RlOPsRQqerutS8MYMwCGSdlxqj4remIi0Y5y9RzK8ojdm%0ADJE+34y5Hs+IVABObDiyw0lGCW5WpMuh/shwNeYHkdTjJD3Kr7oscLMivZvTzaLrlzkiknac%0AqEfZVZcHblakm0T9/xFJNU5Uo+yqywSHSKXjZIejVZx4lOBmReo+2p3MOyJpxol7lFV1+eBm%0ARbpU7ZG+6heRFOPkPcqpuoxwsyJdrx8HYw6ny5JHiJQ3LoJGGVWXFW5BJJukfrngXHBRPMqm%0AurxwiFQuLo5HuVSXGQ6RisXF0SiX6nLDIVKhuEjD0RwuXpTgEKlMXDyPcqguQxwiFYmL6FEG%0A1eWIQ6QScRE1yqC6LHGIVCAuqkebV5cnDpHKw8X1aOvqMsUhUnG4yB7ta2Pat0OkwnCxPdrV%0AxnRoh0hF4WJO103gEkQJDpGKwiXwaD8b060dIhWES6DRfjamYztEKgeXxKO9bEzXdohUDC6N%0ARzvZmM7tEKkUXCKP9rEx3dshUiG4VB7tYmN6tEOkInAppusGuIRRgnMWqboFkXLDJfSo/I3p%0A185RpKr/B5HywSXUqPyN6dkOkdTjUg5Hf6VvTETaLS6xR2VvzA1E+lfHqg2JmcajrV8EeYQR%0ASSUu8XD0V/TGDMAhkmpcOxwlwzUBN9kOkRTj2t2jUqtThUMkxbhumqHQ6nThEEkv7r57VGZ1%0AynCuInFmQza4fpqhyOq04ZxFGif1ywXX5zFdV2J16nCIpBQ3mPYusDp9OETSiRsePiqvOoU4%0ARNKIG58VVFp1KnGIpBD3dHZdYdXpxCGSPtzzWUFlVacUh0jacK8ne5dUnVocIinDTXxpoqDq%0A9OIQSRdu6stH5VSnGIdIqnCTX5oopjrNOERShJv5Lmwh1enGIZIe3Nx3ysuoTjkOkdTgZtdm%0AKKI67ThEUoJbWOKkgOr04xBJB25pqSD91RWAQyQVuMUlt9RXVwIOkTTglpcK0l5dEThEyh+3%0AtgKk7uoKwSFS9rjVlVRVV1cKDpFyx62vSKy5umJwiJQ5zmIlVcXVlYNDpLxxNisS662uIBwi%0AZY2zWtlbbXUl4RApZ5zdCvlaqysKh0gZ4yyvNKG0urJwiJQtzvoCYiqrKw2HSLni7C/Ep7G6%0A4nCIlCnO4QJiCqsrD4dIWeKcrgurrroScYiUI87t+sraqisSh0gZ4hyvU66sujJxiJQfzvX6%0AyrqqKxSHSLnhHIejUJx7wE22Q6S8cO4eaaquXBwi5YXz8EhRdQXjECknnI9GeqorGodIGeH8%0APNJSXdk4RMoH5+mRkuoKxyFSNjhPjZRUVzoOkTLB+Q5HnriAgJtsh0hZ4AI8UlDdDnCIlAUu%0AxKP8q9sDDpFywC1rZIyRxYUF3GQ7RNoet+rRskmZV7cPHCJtj7PwaNGkvKvbCQ6RNset7B4h%0AkgocIm2NW5tmQCQVOETaFmcxXcc+kgYcIm2Ks5r2ZtZOAQ6RtsSFHD3ywMkE3GQ7RNoMF3QU%0A1h0nFXCT7RBpK5yUR3lWtzccIm2FE/Moy+p2h0OkjXBiGmVZ3f5wiLQJTm44ssKJBtxkO0Ta%0AACfqUXbV7RKHSBvgZD3Krbp94hApOU5Yo8yq2ysOkVLjxD3Kqrrd4hApMU7eo5yq2y8OkdLi%0A5DXKqbod4wJFIk5phqOtXwSJG0ak6LgIH+uWcJECbrIdIiXDRfIok+p2jkOkZLhIGmVS3d5x%0AiJQKF82jLKrbPQ6REuHieZRDdeAQKQ1uyaO1r5J74GIG3GQ7REqBW/EozKTNqwOHSAlwxixO%0A160vt+WGix5wk+0QKTJuxSNEKgOHSJFxrUcLniBSEThEioxrhqNFT9hHKgGHSGGZdGCA6zxa%0AFIVZuwJwiBSU6dHkgbPxKDilbEzVOEQKycz+TY9rZhlie1TKxtSNQ6SQLIsU6yTVGVyqgJts%0Ah0gBWRQplUelbEzdOEQKysI+UjKPitmYqnGIFJa5Wbt0GhW0MTXjECkKLqVHxW9MFThEioFL%0A6lHpG1MHDpEiJKlGpW9MJThEEk/a4eiv7I2pBodI0knuUckbUw8OkYST3qOCN6YiHCLJptGo%0A2OrAzbdDJMm0w1Gp1YFbaIdIguk+1hVaHbildogkl/vuUZnVgVtsh0hi6acZiqwO3HI7RBLK%0AYLquwOrArbVDJJkMp73Lqw7cajtEEsno6FFx1YFbb4dIAnk6CltYdeBs2iFSeJ7PZiirOnBW%0A7RApOC9nBRVVHTi7dogUmteT60qqDpxlO0QKy9RJquVUB866HSIFZfJk72KqA2ffDpFCMv2l%0AiVKqA+fQDpH8M/fdozKqA+fUDpG8M/sdviKqA+fWDpF8M/9d2BKqA+fYDpE8s/CV8gKqA+fa%0ADpG8srg0g/rqwLm3QySfLC9xor06cB7tEMkjK0sFKa8OnE87RHLP2opbuqsD59UOkZyzunKd%0A6urA+bVDJNesrwCpuTpwnu0QyTEWK6kqrg6cbztEcovNisR6qwOHSElwdit7a60OHCKlwVmu%0AkK+0OnABOERyiO2FJnRWBy4Eh0jWsb9gi8bqwIXhEMk2Dhc+UlgduEAcIlnG5QJi+qoDF4pD%0AJLs4XYdPXXXggnGIZBPH61kqqw6cAA6RLOJ6XVhd1YGTwCHSepyvr6yqOnAiOERai8dlyhVV%0AB04I5yFStSuRPDxSVB04KZy7SNWuRPLxSE914MRwziJVuxqRfDTSUx04OZz7iLQjkbyGI3+c%0Ab8BlgPMX6V8d2zYq03i09Ysg2sKI9BTf8UhHdeBkcYg0F2+NVFQHThhnL1JVdfN1+xApwCMF%0A1YGTxjEiTSfEo/yrAyeOQ6TJBHmUfXXg5HGINJUwj3KvDlwEnIdIw6R+uUlw/tN1XrjggMsA%0Ah0gvCfYo6+rAxcEh0nOCNcq6OnCRcIg0Tvhw5IQTCbgMcIg0iohH2VYHLh4OkYaR8SjX6sBF%0AxCHSIDIa5VoduJg4ROojNBzZ4uQCLgMcIt0j51GO1YGLjEOkLoIeZVgduNg4RGoiqVF+1YGL%0Aj0OkOrIe5VYduAQ4RPoT9yiz6sClwCGS3Ky3JU484DLAIZL0cLSCixBwGeB2L1IEjzKqDlwq%0A3N5FmvXIGBMBFyXgMsDtXKTZ4ciYAJNyqQ5cOty+RVr2yNukTKoDlxC3a5Hmd48QCZxjux2L%0AtDDNgEjgHNvtV6TF6Tr2kcC5tdurSGvT3szagXNqt1ORYhw+WsBFDbgMcDsVKaZG21cHLj1u%0AnyLF9Wjr6sBtgNulSJE90tL34ARxexQptkda+h6cIG6HIkX3SEvfgxPE7U6kqNN1r7gUAZcB%0Abm8ipfBIS9+DE8TtTKQUGqnpe3CCuF2JlGQ4+lPT9+AEcXsSKZVHWvoenCBuRyIl80hL34MT%0AxO1HpGQaqel7cIK4vYiUbjj6U9P34ARxOxEpqUda+h6cIG4fIqX1SEvfgxPE7UKktBqp6Xtw%0Agrg9iJTaIy19D04QtwORknukpe/BCeLKF6n2SElngNOLK16kZjxS0hng9OIKF6mbrlPSGeD0%0A4soW6T7traQzwOnFFS1SP82gpDPA6cUVLNLgKKySzgCnF1euSMOzGZR0Bji9uGJFGp0VpKQz%0AwOnFlSrS+Cisks4ApxdXpkjPJ6kq6QxwenFFivRysreSzgCnF1eiSK9fmlDSGeD04soTaeq7%0AR0o6A5xeXHEiTX6HT0lngNOLK02k6e/CKukMcHpxhYk0890jJZ0BTi+uKJFml2ZQ0hng9OJK%0AEml+iRMlnQFOL64gkRaWClLSGeD04gJFyijNV8oJ2TrKR6TFJU6U/FcNnF5cKSItLxWkpDPA%0A6cUVItLKkltKOgOcXlwZIq0tXaekM8DpxZUg0vrK3ko6A5xeXAEiWayQr6QzwOnF6RfJZkVi%0AJZ0BTi9Ou0h2F2xR0hng9OKUi2R54SMlnQFOL063SLYXEFPSGeD04lSLZH3BFiWdAU4vTrFI%0ADtezVNIZ4IoWL5kAAAY1SURBVPTi9Irkcl1YJZ0BTi9OrUhO11dW0hng9OKUiuR4mXIlnQFO%0AL06nSI4eaekMcHpxKkVy9UhLZ4DTi9MokvtlypV0Bji9OH0irQ1HxhhJnE/A7Q+nTiQLj15N%0AUtIZ4PTitIlk5dGLSUo6A5xenDKRVnePEAncJjhdIq1PMyASuE1wqkSyma5jHwncFjhNItlN%0AezNrB24DnCKR3A8fBeECAm5/ODUiOZ/NEIYLCrj94aKLNPVJy+PlBnmkpTPA6cXFFmly39/9%0A5QZppKYzwOnFRRZpejba9eWGDUfOuOCA2x9Og0jBHmnpDHB6cQpECvdIS2eA04uLLJLAPlK4%0ARmo6A5xeXGyRQmftBIYjF5xMwO0PF12ksJcr45GWzgCnF5e3SEIeaekMcHpxOYskpZGazgCn%0AF5exSHIeaekMcHpx+Yok6JGWzgCnF5etSIIaqekMcHpxmYokORxZ4IQDbn+4PEUS9khLZ4DT%0Ai8tSJGmPtHQGOL24HEWS1khNZ4DTi8tQJHmPtHQGOL24/ESK4JGWzgCnF5edSDE80tIZ4PTi%0AchMpikdaOgOcXlxeIolP1y3jYgXc/nBZiRTLIy2dAU4vLieRYmmkpjPA6cXlI1K04WgaFzPg%0A9ofLRqSYHmnpDHB6cbmIFNUjLZ0BTi8uE5GiaqSmM8DpxWUhUtzh6AUXPeD2h8tBpOgeaekM%0AcHpxGYgU3yMtnQFOL25zkRJopKYzwOnFbS1SEo+0dAY4vbiNRUrjkZbOAKcXt61IaTRS0xng%0A9OK2FCnRcPSnpjPA6cU5i1TdIiNSOo+0dAY4vThXkar+n1CREnqkpTPA6cVtJlJCjdR0Bji9%0AOFeRHjaFiZTUIy2dAU4vzl+kf3Xs24xTe+TblpCcYy9S+GRD2vFIzX/VwOnFbSJSao+0dAY4%0AvTh7kfp574FHXtSU03VdlHQGOL049xFp6JEPtfFIydYBB862natII488qO1wpGTrgANn285R%0ApKoandrgirt/rFOydcCBs23nOiKN40jrd4+UbB1w4GzbpRTpMc2gZOuAA2fbLqFIg9k6JVsH%0AHDjbdslEGs16K9k64MDZtksl0vjokZKtAw6cbbtEIj0dhVWydcCBs22XRKSXkxmUbB1w4Gzb%0ApRDp9aQgJVsHHDjbdglEmji5TsnWAQfOtl18kabOUVWydcCBs20XXaTJc72VbB1w4GzbpRBJ%0A7uV6Bhy4yLj4Iv1NffdIydYBB862XXyRJF8uOHB54hAJHDgBHCKBAyeAQyRw4ARwiAQOnAAO%0AkcCBE8AhEjhwAjhEAgdOAIdI4MAJ4BAJHDgBHCKBAyeAQyRw4ARwiAQOnAAOkcCBE8AhEjhw%0AAjhEAgdOAIdI4MAJ4BAJHDgBHCKBAyeAQyRw4ARwiAQOnAAOkcCBE8AhEjhwAjhEAgdOAIdI%0A4MAJ4BAJHDgBHCKBAyeAQyRw4ARwiAQOnAAOkcCBE8AFiqQk/7Z+AVFDdVkFkbSG6rIKImkN%0A1WUVRNIaqssqJYtESLIgEiECQSRCBIJIhAgEkQgRCCIRIpCyRaq2fgHRUt2y9WuIGXXFFS1S%0Aue+1qv+nzOjruZJFqsp9qxUuksKeK1mkgt9qhYuksDZEUhlEyi2IpDKIlFsQSWUQKbcUKVI/%0AN6yuO2yDSLmlSJH6qOsO2yBSbkEklUGk3IJIOsOZDZmlbJEISRREIkQgiESIQBCJEIEgEiEC%0AQSRCBIJIhAgEkQgRCCIRIhBE0hxjhr99qjsdoKAgkuaMRTJ05nZh22vOWB1E2jBs+/xzE+TN%0AHH+b387V8Xr9fTfm/ffpntv/MGm7sOnzz82amyPVpf7taN6vl8r0f7/1vyHSpmHT55+bPpfr%0A0Zzq327/XE/mNip1fz/u4aPdpmHb55/bB7rbxzlzqH+rP8Yd6n8ff3e/IdKmYdvnn1aQhypT%0AfyPSxmHb5x9EUhC2ff7pPsAd76rw0S7DsO3zj7k5dDmaj7sqo8mGwT2ItGHY9vmnnvQ2prre%0AVZmZ/r62jyGbBJHyz+0D3LE/AFtnfED27X7PJyJtF0TKP3xkUxD6KP8gkoLQR/kHkRSEPso/%0AiKQg9BEhAkEkQgSCSIQIBJEIEQgiESIQRCJEIIhEiEAQiRCB/A/tJFD1UuRpugAAAABJRU5E%0ArkJggg==)

In \[26\]:

    ggplot(leukemia,aes(gompit,e))+geom_point()+geom_smooth(method="lm",se=F)+
      ggtitle("Gompertz Q-Q plot \n R-squared:", summary(lm(e~gompit, data=leukemia))$r.squared)

    `geom_smooth()` using formula 'y ~ x'

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzMzZv9NTU1o%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///9uyY2wAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2di5aiRgBEyaozs/Nc5/8/NgI+AEGhaZoq+tY5%0ASWYNc610e4MiNMUvIWR2irULELKFIBIhEYJIhEQIIhESIYhESIQgEiERgkiERAgiERIhiERI%0AhCASIRESLtLn674odq+fEUocXwf/1cdLURQvH6MfP6fo/nc9eApC5idUpJ9Dcc4hQomhFj/7%0A85Psvkc9PkwcfApCYiTw9fWzK/afx9P/6D92xcv8EgMtTs/y8nX65+fJ2u8Rjz8gIhJZNIGv%0Ar0PxdvlxV8x+dzf0Kt9fn+Wt2I94/AERkciiCXt9fTVev5/1q/r7dXf6xFTtH04v2r/lHuv3%0A9/30j/f6kdOPh7NxX6cdSf1zURz3xUv1Jq38wzk38m1nt2/oOvB480lqyK1Ti0tI9IS9ut6K%0Axsf8Y/m3z7ME5cu4KP5WP3+9Vv94vz1SKfdRXH8uSovehkR6bclzO1ow8HjzSSpIoxMikWUT%0A9uraF8f2A9+nV/Hx93hy4qd81e4+S9l29T/21SMn8z535Sea7+Jw+nzzfahf34cS1HyNH25H%0AL5rPcix2vc/eeLz5JCWx0ynoP5SQcQl7fd29LN/On1pey3/W+6Vj/Y/femfzU/3r3Vu5aaXB%0AsXx/dtprdXANj9rP0vjDwOPNJykf7XQK+g8lZFzmiXR9L7Y/v4h/6v3PsbFRJdLZjpd9uent%0ALVxjkzpNjyaL1HiSnk5B/6GEjEvY62t3fnPVNeKy/7n+eHnkfGxi3/wodC/SseVR+w3kCXGz%0Atv1456d90arR+QMhSyTs9dX8uD/4ou0R6dB5Sbe3/NndDqp3n+W7eL2K1H38Aus8CSKRdAl7%0AfX02v74ZehvVFOl8vLp817VrfIfa2rLr0elZ6v3T17H8wPP57PHmk/DWjqTN/C9kj/UH+3rH%0A0Pxg3xTpfIChPA7wWm/6XdrQ3PLOo8uzfBfF4aP1xWv/480nGepEyDKZcYrQx+l/+Mevt6I8%0AAP1TFG/1oebvfpHKI+Gf1Uer7+qbnu9dffj7vMlPuafqelQ+S3movPx6t3uKUM/jzScpwZ1O%0AP2H/pYSMSfBJqy/XYwbV67/9hWyNbor0cv23l03fbpuUB/JaByGuz3I9xLf7aj173+PNJ+l+%0AIVs/BSFLJfzV9f1WngD+8nE+gtY+Rajzj9Nfpx3D63nv8fN2PnPossn3/rRX6xPpJEMpx+Hv%0A575zbmzP480n6Z4iVD8FIUslzf+mI+wN3u/e+HUfZ5dD1ouNSCJPQkhvEImQCEEkQiKEFx8h%0AEYJIhEQIIhESIbNEunzxs387Pt84BL8ElZAFEkWkotgtYRIiEZvMFKn+59fh/jy5CEEkYpMo%0AIv1+L3L+DSIRm8QRqf2a/34t1zKuzyYt13D8e3+539u+qE/g/q1X5PptLtJV/dau/i1sIhZZ%0AYI/01Tjpul5q629XpF3zVPGX6kzwxiJd5996RSRikzifkVrXEu2rRe+qS+6+i/Lnj11HpL+n%0AnVT590P152pFruYiXfVvfe5wiNgk0lG7Q+vR64+vRXW1xEdHpH1jmaHzilzNRbre6t867hCJ%0AuCTO90jvzUdfTh+QPs5LzF30uFsS5fvz76H51q25SNdllaAXRCIumf/W7nhZRf/iwc/uKtdt%0A/bu2SO+76yV814dvIl1+64BIxCURPiN9XddLvVze+lkt+v13UKT303vBt4+ftkhdLMcZiFFi%0AHGz423Niw/dreSDvvEH3rd2+sVLx+eGeRbqOiERsEuWo3b7oubFk+S93tTHvtyW5Pm9OfTZF%0Aai7S9VJL9Y5IxCZRRPpurZZVH/5+K4+/vVc3iPio3vEdipfj+Zj2vjwsdz68fTn40Fikq/lb%0AhFgkzheyr831G89fyO7KA3f1V6sv5Yb1w2/nz0h1vm6MxiJdv4fb17h8UCIWiXSK0K5oHAH/%0Aqk4Rqg+Af70Uu7d6w6/99byf93KZrK/P+s4u59+6LdJ12htxihDxyobWbCBkvSASIRGCSIRE%0ACCIREiG8wgmJEEQiJEIQiZAIQSRCIgSRCIkQRCIkQhCJkAhBJEIiBJEIiRBEIiRCEImQCEEk%0AQiIkUKRyoe6Xr+sfvw/Frr5H0s/rZVHv2ybtx6qfbmtv3R4rc16B6/YbH/vL1X63n37LK29/%0AB8iErJEwkY61BpeFGr6v15b/XC8hv21ye+zzuuT35bZKzcd+r+ud3H6jXhG89Of20291BXt7%0AuxaFkOQJE+m9XFvhtVrBu0y1MvH5sbfyZb5vbnJ7rFqG6+u6vsNnuUpK87Hv83on7d/4rFbo%0Av/1U/vu+7RpkQhInTKRqwazv6m4sFaSilCtp1ct1l3++bXJ7rChuW5+y2/+2H9uf183ftbfr%0A3j1mt/vqbtchE5I4YS+93ld64/W+v9ukeuyl3m+c/Xur3ok1HvtbfLQWXD3vXz6vt7o4//TW%0AWZj1jkxI6oSJ1DHnUL61e7v+8bP8Y2eT6rF6da6zFsezKNfHyt1XQ5DP8weil6set5/ut2uS%0ACUmeKCJVS9YdLn/82R3uNqkfO6lQrnJXP/S3Wkey8dhud2wIcv6N6uZ+L92f7rdrkglJnigi%0A/X6eXuKXpbrPr+zWJufH3q4HJX4v7/1uj73WR/POxKtHv+UBu7fuT93tWmRCkmeWSK07MP/U%0Adxv7PhvQ3KT3saLzWOO+LrffaD5d86frYsk9ZELSJ0ykffuoXZWvan/wcXmwscn1seZe6v38%0AEej6WFOk62/Udyqr1+M//t6J1EsmJH3CXnpv5b0jXq/LFO9P78qO1fHur+tNMG+b3B57KT8W%0AvRfnDzXfd49dVGhS/pb/9rX5U992bQohqRMmUn0qw3Uf8X5d//5w3avcNrl7rDLocn/L5mMX%0AQW6/cdxdzpm4/dS3XZtCSOrMONfucLtb2PvlPLjG27O+8+q+X4ripX61X9+GNR67vUW7/kZ5%0ANl29Hv/tp77tWhRCUodPFYRECCIREiGIREiEIBIhEYJIhEQIIhESIYhESIQgEiERgkiERAgi%0AERIhiERIhASJ9O+axo+RsxzZsjTkROhpZESC7EyWKY1IkJ3JMqURCbIzWaY0IkF2JsuURiTI%0AzmSZ0ogE2ZksUxqRIDuTZUojEmRnskxpRILsTJYpjUiQnckypREJsjNZpjQiQXYmy5RGJMjO%0AZJnSiATZmSxTGpEgO5NlSiMSZGeyTGlEguxMlimNSJCdyTKlEQmyM1mmNCJBdibLlEYkyM5k%0AmdKIBNmZLFMakSA7k2VKIxJkZ7JMaUSC7EyWKY1IkJ3JMqURCbIzWaY0IkF2JsuURiTIzuQk%0ApYuieL41IkE2JqcoXRQjTEIkyM7kBKWLYoxJiATZmbwVkQjZes4ijd8+5Enu/Y0f6/+dQV6W%0AzGek8fGeBciLkjlqNz7mswA5iylEJMjOZJnSiATZmSxTGpEgO5NlSiMSZGeyTGlEguxMlimN%0ASJCdyTKlEQmyM1mmNCJBdibLlEYkyM5kmdKIBNmZLFMakSA7k2VKIxJkZ7JMaUSC7EyWKY1I%0AkJ3JMqURCbIzWaY0IkF2JsuURiTIzmSZ0ogE2ZksUxqRIDuTZUojEmRnskxpRILsTJYpjUiQ%0AnckypREJsjNZpjQiQXYmy5RGJMjOZJnSiATZmSxTGpEgO5NlSiMSZGeyTGlEguxMlimNSJCd%0AyTKlEQmyM1mmNCJBdibLlEYkyM5kmdKIBNmZLFMakSA7k2VKIxJkZ7JMaUSC7EyWKY1IkJ3J%0AMqURCbIzWaY0IkF2JsuURiTIzmSZ0ogE2ZksUxqRIDuTZUojEmRnskxpRILsTJYpjUiQncky%0ApREJsjNZpjQiQXYmy5RGJMjOZJnSiATZmSxTGpEgO5NlSiMSZGeyTGlEguxMlimNSJCdyTKl%0AEQmyM1mm9GSRdqcgEmQRskzpqSLtrn9DJMjrk2VKIxJkZ7JM6aki3WxCJMjrk2VKh4v0X5nx%0Av0NIHhkvEgcbIKuQZUojEmRnskzpAJEaHiES5HXJMqWni9T0CJEgr0uWKT1ZpJZHiAR5XbJM%0A6aki7XatUxtCn3ZSxcXIlqUhJ0Jzrp0CGnIaskxpRILsTJYpjUiQnckypREJsjNZpjQiQXYm%0Ay5RGJMjOZJnSiATZmSxTGpEgO5NlSiMSZGeyTGlEguxMlimNSJCdyTKlEQmyM1mmNCJBdibL%0AlEYkyM5kmdKIBNmZLFMakSA7k2VKIxJkZ7JMaUSC7EyWKY1IkJ3JMqURCbIzWaY0IkF2JsuU%0ARiTIzmSZ0ogE2ZksUxqRIDuTZUojEmRnskxpRILsTJYpjUiQnckypREJsjNZpjQiQXYmy5RG%0AJMjOZJnSiATZmSxTGpEgO5OTlP4zZmtEgmxMTlH6DyKth4achrx86T9lRmyNSJCNyYuXHukR%0AIkG2Ji9c+s9YjxAJsjV52dKjNUIkyN7kJUuP3x39QyTI3uQF0VM0QiTI3uTl0NM8QiTI1uSl%0A0BM1QiTI3uSF0JM9QiTI1uRl0JVHnGungIachrwIut4dIZICGnIa8gLoy9s6RFJAQ05Djo++%0AfjxCJAU05DTk6OjbYQZEUkBDTkOOjG4erUMkBTTkNOS46NZRb0RSQENOQ46Kbn97hEgKaMhp%0AyDHRnS9hEUkBDTkNOR767mQGRFJAQ05Djoa+PykIkRTQkNOQI6H7Lj1CJAU05DTkOOjec1QR%0ASQENOQ05BnrgStikIhHinvpU73hhjwTZjzwbPbwwA2/tFNCQ05Dnoh9cwYdICmjIacgz0Y+u%0AhEUkBTTkNORZ6McXlCOSAhpyGvIc9JOFGRBJAQ05DXkG+tkCJ4ikgIachhyOfrpOECIpoCGn%0AIYeiR6y3hUgKaMhpyIHoMevWIZICGnIachh61PqPiKSAhpyGHIIeew++aT0QCbIxOQA9+h58%0A03ogEmRj8nT0+HvwTeuBSJCNyZPRE+7BN60HIkE2Jk9ET7oH37QeiATZmDwNPeluLYikgIac%0AhjwFPemesIikgYachjwBPfXmYYikgIachjwaPXF3NIF83hqRIBuTx6Kn38sSkSTQkNOQx6Gn%0A747Gkm9bIxJkY/IodIhGiKSBhpyGPAYd5hEiSaAhpyE/RwdqhEgaaMhpyE/RwR4hkgQachry%0AM3S4R4gkgYachvwEHa4RImmgIachP0QP7I6KophL7tkakSAbkx+hhz0aZRIiKaAhpyE/QD/y%0AaIxJiKSAhpyGPIgePMqASFZoyGnIQ+jho3WIZIWGnIY8gH501JvPSE5oyGnI/ejHR705ameE%0AhpyG3Iee8SXsE/KjrREJsjG5Bx3JI0SSQENOQ+6giyLo0qMR5KdbIxJkY3IbffZoAfLzrREJ%0AsjG5hY64O/qHSBpoyGnILXSl0bhDchPJI7ZGJMjG5Ab6z9kjRBqquBjZsjTkfvRFo1geIZIE%0AGvIi5HtPLuj601FEjxBJAg15CXLP2T01OuZBhjZ59NaIBNmF3He+aYVewiNEkkBDXoA8JNIi%0AHiGSBBryAuQBkRbRCJE00JCXIPd9Rlpmd/QPkTTQkBch3x+VW8wjRJJAQ05DXs4jRJJAQ05B%0AXlAjRNJAQ05AXtQjRJJAQ16eXHmkUhqRIJuS692RSmlEgmxJvrytUymNSJAdydePRyqlEQmy%0AIfl2mEGlNCJBtiM3j9aplA4QaYdIkNckt456q5SeLtIOkSCvSW5/e6RSerJIO/ZIkNckd76E%0AVSk9fY+ESJDXI9+dzKBSOlyk/8qM/R1CYqQ+mUE77JEgq5N7Tq5TKY1IkF3IveeoqpRGJMgm%0A5P5zvVVKIxJkD/LANRMqpREJsgN58NIjldKIBNmAPHwJn0rpAJGaCX3aSRUXI1uWzpH84FJY%0AldKIBFmd/PCKcpXSiARZnPx4ZQaV0ogEWZv8ZIUTldKIBFma/GyhIJXSiARZmPx8wS2V0ogE%0AWZc8YuE6ldKIBFmWPGYBSJXSiARZlDxuHVWV0ogEWZM8cj1ildKIBFmSPHZdb5XSiARZkTx6%0AeXyV0ogEWY884TYTKqURCbIcecrtWlRKIxJkMfKfSbc9EimNSJDFyBPvHqZRGpEga5Gn7Y4m%0AoacGkRTQkIPI029mKVC63hqRIKuQJ++OxqMDgkgKaMjTyUH3Vl679HVrRIKsQQ67R7nKcCAS%0AZAlymEY6w4FIkBXIoR7JDAciQRYgB3skMxyIBHl9crBGOsOBSJDXJofvjp6i5wSRFNCQR5Nn%0AeSQzHIgEeV3yPI9khgORIK9JnqmRznAgEuQVybM9khkORIK8Hnm+RzLDgUiQVyP3alQURQR0%0AhCCSAhryU3L/7qgoppmkMhyIBHkd8gOPppikMhyIBHkN8tClR4ik8h8kgob8kDx4lAGRVP6D%0ARNCQH5AfXQnLZ6TI/xkpyJal/cmPD3pz1G6pINKmyCELM4xExw0iKaAhD5DjaqQzHIgEOSU5%0Atkcyw4FIkNORo2ukMxyIBDkZeQGPZIYDkSCnIt88mnhk7jl6iSCSAhryHfm2O5r6XdFT9CJB%0AJAU05A75T8ejWCapDAciQU5Bbn48QiREghxGbh1mQCREghyS7tE6PiMhEuTpuT/qzVE7RII8%0ANUt8e3SLynAgEuRlyaVGGUwhIkFeklzvjjKYQkSCvCD5/LYugylEJMiLka+XHmUwhTNFImQw%0AlUZrl0gf9kiQY6Z5JWwGU4hIkBchtw56ZzCFiAR5AXJnYYYMphCRIMcnd7+DzWAKEQlydPLd%0AuQwZTCEiQY5MvmjUOKEugylEJMhxyQ2PriZlMIWIBDkqueXRxaQMphCRIMckN9c3QSREghyU%0AoYUZMphCRIIcjXx/QXks8nBUhgORIMcid456c9QOkSCPzc2WRxfCZjCFiAR5Bvn2/u3hBeUZ%0ATCEiQQ4n344oPF6YIYMpRCTI4eSrSE/WN8lgChEJcjj5LNLTdYIymEJEgjyDPM6jHKYQkSDP%0AIY/yKIcpRCTIs8ijln/MYAoRCfIc8rhlVDOYQkSCPIM8cjniDKYQkSCHk8eu6p3BFCIS5FDy%0A+NXxM5hCRIIcSJ5wl4kMphCRIAeR/0zwKIcpRCTIIeRpNz3KYAoRCfITcs/t9SbtjobJMaIy%0A0IgE+TG554avk+/Bl8EUIhLkh+T7W5BP3R0NkeNEZaARCfJD8p1IIbeEzWAKEQnyQ3JXpKBb%0AK2cwhYgE+TG55VHgHcozmEJEgvyEPN+jHKYQkSCPJod6lMMUIhLkseRQjbKYQkSCPI4cvDt6%0ASp4VlYFGJMijyHM8ymEKEQnyGPIsj3KYQkSC/Jw8T6MsphCRID8lz/UohylEJMjPyLM9ymEK%0AEQnyE/JsjbKYQkSC/JA8f3c0RI4TlYFGJMiPyFE8ymEKEQnyMDng0qOR5GhRGWhEgjxIjqRR%0AFlOISJAHyLF2R/fkmFEZaESC3E+Op1EWU4hIkPvIEXdH/7KYQkSC3EOOqlEWU4hIkO/JkT3K%0AYQoRCXKXHFujLKYQkSB3yPE9ymEKEQlym7yARzlMISJBbpEX0CiLKUQkyA3yErujf1lMISJB%0AvpEX8iiHKXwg0vtLUfwevjuP7k5BpG2Sl/IohykcFOm4L5eq/S2Kr7ZH178h0sbIi2mUxRQO%0AivRavJ0s+v0oDoiUBXlBj3KYwkGRThJd/0Kk7ZOX9CiHKQwX6b8yv2QbKTVau8NG0v/W7q14%0AbT3MHmmL5Hp35NV5cXSsgw27+g5Tux9E2jr5/LbOqvPy6GiHv//ui2L/dmw/iEibI18vPTLq%0AnAK97BeyiLQ18u0og0/nJGhEUkC7kJtXwrp0ToRe+BQhzmzYErl10Nukcyo059opoC3InYUZ%0ALDqnQyOSAtqB3P0O1qFzQjQiKaANyHfnMhh0TolGJAW0PLnnlCD5zmnRiKSAVif3nVqn3jkx%0AGpEU0OLk3lNUxTunRiOSAlqb3H+mt3bn5GhEUkArk4eumFDuvAIakRTQwuTBK4+EO6+BRiQF%0AtC55+Ao+3c6roBFJAa1KfnQhrGrnldCIpIAWJT+8oFy081poRFJAa5IfL8yg2Xk1NCIpoCXJ%0AT9Y3key8HhqRFNCC5KfrBAl2XhONSApoPfLz9bb0Oq+KRiQFtBx5xLp1cp3XRSOSAlqMPGr5%0AR7HOa6MRSQGtRR63jKpW59XRiKSAliKPXI5YqvP6aERSQCuRx67qrdRZAI1ICmgd8vjV8XU6%0AS6ARSQEtQ55wlwmZzhpoRFJAi5D/TPBIpbMKGpEU0BrkaTc90ugsg0YkBbQCedLuaBJ5ajKY%0AQkTaLHnyPfgEOiuhEUkBvTp56u5oPDkgGUwhIm2THHJL2LU7i6ERSQG9Mjno1sqOoyFTGpE2%0ASA68Q7njaMiURqTtkQM9shwNmdKItDlyqEeWoyFTGpG2Rg7VyHM0ZEoj0rbIwbujp+Q5yWAK%0AEWlT5DkeWY6GTGlE2hJ5lkeWoyFTGpE2RJ6lkedoyJRGpO2Qwz0qiuIheW4ymEJE2gx5lkel%0ASY6jIVMakbZCnunRySTH0ZApjUjbIM85zIBIEciItAnyrMN1iBSBjEgbIM876s1npBhkRPIn%0Az/WIo3YRyIjkTg64FHYkOV4ymEJEMidH08hzNGRKI5I1Od7uqEuOmgymEJGcyTE18hwNmdKI%0AZEyO65HlaMiURiRbcmSNPEdDpjQiuZKje2Q5GjKlEcmUHN8jy9GQKY1InuT4GnmOhkxpRHIk%0AL7A7+uc5GjKlEcmQvIxHlqMhU3qmSGSFVB6tXYIMhT2SB3mh3dE/z9GQKY1IZuTlPHIcDZ3S%0AiORFXtAjw9FYFI1ICuiFyAtqZDgay6IRSQG9CLk+yrAEuYojWaY0IvmQ67d1IeT6Cthn8RqN%0AxdGIpICOT75cenRHfm7JeU2GZ3EajQRoRFJARydfjzJ0yc8tuawS9CxGo5ECjUgK6MjkxpWw%0AHfIISxApARmRHMjNg96IlAiNSAroCOTra7+9MMN0kfiMlICMSKrk66u/8x3s9M9IHLVLQEYk%0AUfJ1T9M9lyHgqN3ICI/GGmhEUkDHEun+lCDhziuQZUojkih5yCPlziuQZUojkip5wCPpzunJ%0AMqURSZZce7QEeSCOZJnSiCRLHrhiQrpzcrJMaURSJQ9deaTcOT1ZpjQiiZIHr+AT7rwCWaY0%0AIkmSH1wIK9t5FbJMaURSJD+6oFy18zpkmdKIJEh+uDCDaOeVyDKlEUmP/HhhBs3Oa5FlSiOS%0AGvnZOkGKndcjy5RGJDHy0/W2BDuvSJYpjUhS5BH3hJXrvCpZpjQiKZHHLP+o1nldskxpRNIh%0Aj7tFuVbntckypRFJhjxyNWKpzquTZUojkgh53O4ohDw6jmSZ0oikQR6/OL5OZwWyTGlEkiBP%0AuMmETGcJskxpRBIgT7pXi0hnEbJMaURanzztnkcanVXIMqURaXXyxHuHSXSWIcuURqS1yVPv%0AHabQWYcsUxqR1iVPv5Xl+p2VyDKlEWlVcsAtYVfvLEWWKY1Ia5JDbq28dmctskxpRFqPHHaH%0AcseB3uwUNrZGpLXIYR5ZDvRWp7C5NSKtRA70yHKgNzqFra0RaR1yoEaeA73NKWxvjUhrkEN3%0AR8/JM+JIlimNSGuQZ3hkOdAbnMK7rREpPXmOR5YDvb0pvN8akVKTZ2nkOdBbm8K+rREpMXmm%0AR5YDvbEp7N0akdKS53pkOdDbmsL+rREpKXmuRp4DvakpHNgakRKSZ++OBskx4kiWKY1ICckx%0APLIc6O1M4fDWiJSKPHrBrcnkSHEky5RGpFTkOBp5DvRGpvDh1oiUhBxpd9RDjhdHskxpREpC%0AjqaR50BvYQqfbT1dpB0iTSTH2x11yVHjSJYpPV2kHSJNJMfUyHOg7adwxNZTRdqxR5pIjuuR%0A5UC7T+GYrSfvkRBpEjmyRp4D7T2F47YOFum/MmN/J9tUHq1dgqQMe6QFyNH3R54D7TyFY7dG%0ApAXJ8TXyHGjjKRy99WiRdrvz8TpEGkleYHf0z3OgbadwwtbskZYiL+OR5UC7TuGUrRFpIfJC%0AHlkOtOkUTtoakRbJUhp5DrTlFHKunQB6OY8sB9pxChFJAL2gR5YDbTiFiCSAXlAjz4H2m8LJ%0AZESKnvpkhiXIVSAnQiPSuuj6bZ3jcDiSZUojUtxcLj1yHA5HskxpRIqa61EGx+FwJMuURqSI%0AaVwJ6zgcjmSZ0ogUL82D3o7D4UiWKY1IsdJemMFxOBzJMqURKVI638E6DocjWaY0IsVJ91wG%0Ax+FwJMuURqQYuT8lyHE4HMkypREpQnpOrXMcDkeyTGlEmp++U1T7yEVRRHg2x4GWn8L5ZESa%0And5TVHvIRRHFJMeBVp/CCGREmpmBKybuyUURxyTHgdaewihkRJqXoSuPECkNWaY0Is3K4BV8%0AiJSGLFMakWbkwYWwfEZKQ5YpjUjheXRBOUft0pBlSiNScB4uzOA4HI5kmdKIFJrHCzM4Docj%0AWaY0IoXl2TpBjsPhSJYpjUhBebreluNwOJJlSiNSQEbcE9ZxOBzJMqURaXrGLP/oOByOZJnS%0AiDQ1425R7jgcjmSZ0og0MSNXI3YcDkeyTGlEmpRxu6MQ8uhAToRGpOXQ4xfHdxwOR7JMaUSa%0AkAk3mXAcDkeyTGlEGp1J92pxHA5HskxpRBqbafc8chwOR7JMaUQamYn3DnMcDkeyTGlEGpep%0A9w5zHA5HskxpRBqT6beydBwOR7JMaUQakYBbwjoOhyNZpjQiPU/IrZUdh8ORLFMakZ4l7A7l%0AjsPhSJYpjUhPEuaR5XA4kmVKI9LjBHpkORyOZJnSiPQwgRp5DocjWaY0Ij1I6O7oOXlGICdC%0AI1Is9AyPLIfDkSxTGpGGMvrSo8nkeYGcCI1IUdCzNPIcDkeyTGlE6s283dEj8uxAToRGpPno%0AuRp5DocjWaY0It1n9u5okBwjkBOhEWkmOoJGnsPhSJYpjUjdRPHIcjgcyTKlEamdOBp5Docj%0AWaY0IrUSyyPL4XAky5RGpGaieWQ5HI5kmdKI1Eg0jTyHw5EsU3qmSFtKtTtauwSxT+57pHhv%0A67rkuIGcCJ57X5QAAAlVSURBVM1buyB0XI8sh8ORLFMakapE1shzOBzJMqURqUx0jyyHw5Es%0AUxqR/i3hkeVwOJJlSiNS1KPeHfISgZwIjUjT0Avsjv55DocjWaZ09iIt45HlcDiSZUrnLtJC%0AHlkOhyNZpnTeIi2lkedwOJJlSmct0nIeWQ6HI1mmdM4iLeiR43BYkmVKZyzSgho5DocnWaZ0%0AtiLV53ovgi4DOQ1ZpnSuItVv68xKQ06JRqQROX888ioNOSkakZ7mepThDl0URZyncBoOZ7JM%0A6RxFuh2t66KLIpZJRsNhTZYpnaFIjaPeHXRRRDPJZzi8yTKlsxOp9eURIrmTZUrnJlL7S1hE%0AcifLlM5MpM7JDHxGcifLlM5KpLtzgjhq506WKZ2TSPfn1hmUhrwWWkyk+f+PjzVUPeeoqswC%0AZEG0lkgRPnVEGqq+c1RVZgGyIFpKpBjHwaIMVf8lEyqzAFkQjUg9Gbj0SGUWIAuiEek+Q5fw%0AqcwCZEG0lEgSn5GGr4RVmQXIgmgtkQSO2j24olxlFiALosVEmp+Z5EcrM8iWhrw+GpFaebgy%0Ag2ppyAJoRGrkyUJBmqUhS6AR6ZZnC25JloasgUakS/48XbhOsDRkFTQinTNi/Ue90pBl0IhU%0A5fnuKBg9JpDTkGVKb1WkccsRi5WGrIRGpJG7ozD0yEBOQ5YpvUmRRq+Or1QashgakcbfZUKo%0ANGQ1dO4iTblZi0xpyHrozEWadNMjldKQBdF5izTt5mEipSErorMWaeLNwzRKQ5ZEZyzS5HtZ%0AKpSGLIrOV6Tp94QVKA1ZFZ2tSAH3Vl6/NGRZdKYiBd2ifO3SkIXReYoU5NHapSEro7MUKcwj%0AmVmALIjOUaQwjXRmAbIgOj+RAndHY9DBgZyGLFN6AyKFeyQzC5AF0ZmJNPbSowD0nEBOQ5Yp%0A7S7SHI10ZgGyIDonkWbtjh6jZwZyGrJM6cki7U5REWmmRjqzAFkQvaxIu+vfVhdp7u7oAXp+%0AIKchy5T2FWm+RjqzAFkQneAzkoJIMTySmQXIguhUIv1XZvzvxE2l0VpPTsiDjBdp/YMNUXZH%0A/ehIgZyGLFPaUqRYHsnMAmRB9FIiXY97NzxaR6RYGunMAmRB9NJ7pKZHa4gUbXd0j44YyGnI%0AMqUni9TyaAWRYnokMwuQBdELf4+0a53aEPq0kyo2/xDVI5lZgCyI3vK5dnE10pkFyILoDYsU%0A2yOZWYAsiN6uSNE9kpkFyILozYoUXSOdWYAsiN6oSPF3R/90ZgGyIHqbIi3ikcwsQBZEb1Gk%0ACJceDaEXCuQ0ZJnSFiItpJHOLEAWRG9OpKV2R/90ZgGyIHprIi2nkc4sQBZEb0ukBXdH/3Rm%0AAbIgelMiLaqRzixAFkRvSaT6ivIlyHVUZgGyIHo7Ip13RxnMAmRB9GZEuryty2AWIAuityLS%0A9eNRBrMAWRC9EZFuRxkymAXIguhNiNQ8WpfBLEAWRG9BpNZR7wxmAbIgegMitb89ymAWIAui%0A7UXqfgmbwSxAFkS7i3R3MkMGswBZEG0u0v1JQRnMAmRBtLdIPefWZTALkAXRziL1nqOawSxA%0AFkQbi9R/rncGswBZEO0r0sA1ExnMAmRBtKtIg5ceZTALkAXRpiINX8KXwSxAFkR7ivTgUtgM%0AZgGyINpSpEdXlGcwC5AF0YYiPV6ZIYNZgCyI9hPpyQonGcwCZEG0m0hPF9zKYBYgC6LNRHq+%0A4FYGswBZEG0l0pj1HzOYBciCaCeRRq3/mMEsQBZE+4g0cjniDGYBsiDaRqSxyxFnMAuQBdEu%0AIo1e1juDWYAsiPYQacLq+BnMAmRBtIVIU+4ykcEsQBZEO4g06W4tGcwCZEG0gUjTbnqUwSxA%0AFkTLizT15mEZzAJkQbS6SJNvwpfBLEAWRIuLNP1mlhnMAmRBtLRIIfeEzWAWIAuilUUKurdy%0ABrMAWRAtLFLYPcozmAXIgmhdkYI0ymIWIAuiVUUK2x2NIYdHZRYgC6JFRQr2KIdZgCyIlhRp%0A5KVH/RXDfm1dNOQ0ZJnSaUSaoVEWswBZEK0n0pzd0WPy3KjMAmRBdFKRxqTSaCk4IWJZao80%0Ab3f0iDw/Kv87gyyIVntrN1OjLGYBsiBaTaR/Mz3KYRYgC6LlRJqbDGYBsiAakRTQkNOQZUoj%0AEmRnskxpRILsTJYpjUiQnckypREJsjNZpjQiQXYmy5RGJMjOZJnSiATZmSxTGpEgO5NlSiMS%0AZGeyTGlEguxMlimNSJCdyTKlEQmyM1mmNCJBdibLlEYkyM5kmdKIBNmZLFMakSA7k2VKIxJk%0AZ7JMaUSC7EyWKY1IkJ3JMqURCbIzWaY0IkF2JsuURiTIzmSZ0ogE2ZksUxqRIDuTZUojEmRn%0AskxpRILsTJYpjUiQnckypREJsjNZpvRMkW75b96vrxPH0nROlNDSiOQROicKIo2PY2k6Jwoi%0AjY9jaTonykoiEULKIBIhEYJIhEQIIhESIYhESIQgEiERMlOk3SlxiiSMZelft8qeoxw8zPNE%0A2s156rViWfrX7VXpOcrhw4xIHtm5NbYc5RnDHOEzkt1olfErbdbYU6S13trNeupV41farDEi%0ATXxet/fudQxLm1VGpGRPvWYMO5tVRqSRz3fbFfmM1rW0T2XLzmUQafKz+o2WX+Nfu9KmLw1E%0AmhC7wlXMWnu+NFb7jOT49fVu59ja7kVpOcirHmwghCASIRGCSIRECCIREiGIREiEIBIhEYJI%0AhEQIIhESIYhESIQg0nZSnCbz3fFsgi0EkbaTUqSCCV0njPu2gkgrhXEXz8+h2H9Wevy8FsXr%0Az28ly0vx8vuzL16O5z8dzo8XBSatE4ZdO8ddUdR21D/tKnVeTj997E9/ey3/9Hp9HJFWC8Ou%0Anb/F4fd4KO14O/30eyjeKnV+P4rTTx/1p6LD8fI4b+1WC+OunX1xetP2U+px/mlfyvJT/u14%0AUef7+jgirRbGXTu1GDdFrj9d/9Z+HJFWCuOuHUQyCeOunf63dr8tkarHD4i0ahh37ZwPMXQO%0ANvy2RKoOR/xFpFXDuGun//D3b1uk8vHrnzhHaJUgknjKL2Q/7r6Qbb+1O9wef0ekdYJIDnlk%0AB2/mJMIsaKcovsqPR6+PtkjXhgyGWdDOW/0R6Wd4C0SSCLMgnvf9+ZPRUBBJIswCIRGCSIRE%0ACCIREiGIREiEIBIhEYJIhEQIIhESIYhESIT8D+arVlvPpM2mAAAAAElFTkSuQmCC)

Normal 분포의 R-squared 값이 더 크기 때문에, 오차의 분포는 e \~ N(0,1)
임을 알 수 있다.

(b) log-normal 모형<a href="#(b)-log-normal-모형" class="anchor-link">¶</a>
---------------------------------------------------------------------------

In \[27\]:

    lnleuk <- data.frame(lnx = leukemia$lnx, lny = leukemia$lny)

In \[28\]:

    trans2<- function(par,x)  (x[,2]-par[1]-par[2]*x[,1])/par[3] 
    norm.loglik2<- function(par,x) -2*sum( -trans2(par,x)^2/2 -0.5*log(2*pi*(par[3]^2)))

In \[29\]:

    par5<-lm(lny~lnx, data=leukemia)$coeff 
    par5<-c(par5, lm(e~probit, data=leukemia)$coeff[2]) 
    par5 #initial values

(Intercept)  
5.42539580261603

lnx  
-0.817800667958474

probit  
1.33705190964411

In \[30\]:

    lognorm.opt<-optimx(par=par5, fn=norm.loglik2, x=lnleuk); lognorm.opt

    Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method Nelder-Mead"Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method BFGS"

|             | X.Intercept. | lnx        | probit   | value    | fevals | gevals | niter | convcode | kkt1 | kkt2 | xtime |
|-------------|--------------|------------|----------|----------|--------|--------|-------|----------|------|------|-------|
| Nelder-Mead | 5.425380     | -0.8177778 | 1.157902 | 53.23033 | 86     | NA     | NA    | 0        | NA   | NA   | 0.03  |
| BFGS        | 5.425399     | -0.8177905 | 1.158062 | 53.23033 | 31     | 6      | NA    | 0        | NA   | NA   | 0.00  |

In \[31\]:

    lognorm.mle<-lognorm.opt[1,1:3]
    colnames(lognorm.mle)<-c("alpha", "beta", "sigma"); rownames(lognorm.mle)<-"MLE"
    lognorm.mle

|     | alpha   | beta       | sigma    |
|-----|---------|------------|----------|
| MLE | 5.42538 | -0.8177778 | 1.157902 |

In \[32\]:

    cat("AIC of log-normal : ", lognorm.opt$value[1] + 6)

    AIC of log-normal :  59.23033

(c) log-Gompertz 모형<a href="#(c)-log-Gompertz-모형" class="anchor-link">¶</a>
-------------------------------------------------------------------------------

In \[33\]:

    trans3<- function(par,x) (x[,2]-par[1]-par[2]*x[,1]+0.57721)/par[3]
    gomp.loglik2 <- function(par,x) 
      -2*sum(log(1/par[3]*exp(-exp(trans3(par,x)) + trans3(par,x))))

In \[34\]:

    par6<-lm(lny~lnx, data=leukemia)$coeff
    par6<-c(par6, lm(e~gompit, data=leukemia)$coeff[2])
    par6 #initial values

(Intercept)  
5.42539580261603

lnx  
-0.817800667958474

gompit  
1.07450249576402

In \[35\]:

    loggomp.opt<-optimx(par=par6, fn=gomp.loglik2, x=lnleuk, hessian = TRUE); loggomp.opt
    # hessian 값이 출력되지 않습니다...

    Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method Nelder-Mead"Warning message in log(1/par[3] * exp(-exp(trans3(par, x)) + trans3(par, x))):
    "NaN이 생성되었습니다"Warning message in log(1/par[3] * exp(-exp(trans3(par, x)) + trans3(par, x))):
    "NaN이 생성되었습니다"Warning message in optimx.run(par, optcfg$ufn, optcfg$ugr, optcfg$uhess, lower, :
    "Gradient not computable after method BFGS"

|             | X.Intercept. | lnx        | gompit    | value    | fevals | gevals | niter | convcode | kkt1 | kkt2 | xtime |
|-------------|--------------|------------|-----------|----------|--------|--------|-------|----------|------|------|-------|
| Nelder-Mead | 5.723224     | -0.4768700 | 0.9783357 | 53.44568 | 96     | NA     | NA    | 0        | NA   | NA   | 0.00  |
| BFGS        | 5.723482     | -0.4769899 | 0.9784933 | 53.44568 | 41     | 13     | NA    | 0        | NA   | NA   | 0.02  |

In \[36\]:

    loggomp.mle<-loggomp.opt[1,1:3]
    colnames(loggomp.mle)<-c("alpha", "beta", "sigma"); rownames(loggomp.mle)<-"MLE"
    loggomp.mle

|     | alpha    | beta     | sigma     |
|-----|----------|----------|-----------|
| MLE | 5.723224 | -0.47687 | 0.9783357 |

In \[37\]:

    cat("AIC of log-Gompertz : ", loggomp.opt$value[1] + 6)

    AIC of log-Gompertz :  59.44568

(d) log-Gompertz 모수들의 추정오차, 신뢰구간 구하기<a href="#(d)-log-Gompertz-모수들의-추정오차,-신뢰구간-구하기" class="anchor-link">¶</a>
-------------------------------------------------------------------------------------------------------------------------------------------

In \[39\]:

    loggomp.opt$hessian

    NULL

In \[38\]:

    loggomp.hess<-rep(0,3)
    for(i in 1:3) loggomp.hess[i]<-sqrt(solve(loggomp.opt$hessian)[i,i])
    loggomp.hess<-t(as.matrix(loggomp.hess))
    colnames(loggomp.hess)<-c("alpha", "beta", "sigma"); rownames(loggomp.hess)<-"SE"
    loggomp.hess

    Error in array(x, c(length(x), 1L), if (!is.null(names(x))) list(names(x), : 'data'는 반드시 벡터형이어야 하지만 'NULL'입니다
    Traceback:

    1. solve(loggomp.opt$hessian)
    2. solve.default(loggomp.opt$hessian)
    3. as.matrix(a)
    4. as.matrix.default(a)
    5. array(x, c(length(x), 1L), if (!is.null(names(x))) list(names(x), 
     .     NULL) else NULL)

In \[ \]:

    cat("CI.alpha: [", loggomp.mle[1] - 1.96*loggomp.hess[1], ",", loggomp.mle[1] + 1.96*loggomp.hess[1], "]") 
    cat("CI.beta: [", loggomp.mle[2] - 1.96*loggomp.hess[2], ",", loggomp.mle[2] + 1.96*loggomp.hess[2], "]") 
    cat("CI.sigma: [", loggomp.mle[3] - 1.96*loggomp.hess[3], ",", loggomp.mle[3] + 1.96*loggomp.hess[3], "]")

(e) log-normal vs log-Gompertz<a href="#(e)-log-normal-vs-log-Gompertz" class="anchor-link">¶</a>
-------------------------------------------------------------------------------------------------

In \[40\]:

    AIC.table3<-matrix(c(lognorm.opt$value[1], loggomp.opt$value[1], 
                        rep(6,2), rep(0,2)), ncol=3) 
    AIC.table3[,3]<-AIC.table3[,1]+AIC.table3[,2] 
    colnames(AIC.table3)<-c("-2loglik", "2p", "AIC") 
    rownames(AIC.table3)<-c("log-normal", "log-Gompertz")

    AIC.table3
    names(which.min(AIC.table3[,3]))

|              | -2loglik | 2p  | AIC      |
|--------------|----------|-----|----------|
| log-normal   | 53.23033 | 6   | 59.23033 |
| log-Gompertz | 53.44568 | 6   | 59.44568 |

'log-normal'

-   log-normal모형에서 AIC가 더 작기 때문에 log-normal모형이 더
    적절하다고 할 수 있다.
-   하지만 일반적으로, 수명분포는 Gompertz 분포를 따른다고 알려져 있다.
-   (e)의 결과가 이상하다고 생각할 수 있겠으나 문제 (f)를 풀어보면  
    극단적인 값에서는 Gompertz 분포를 더 따른다는 것을 확인할 수 있다.

(f) x\*=20인 백혈병 환자의 생존확률 구하기<a href="#(f)-x*=20인-백혈병-환자의-생존확률-구하기" class="anchor-link">¶</a>
------------------------------------------------------------------------------------------------------------------------

In \[41\]:

    # 1년, 2년, 5년 = 52주, 104주, 260주 
    life <- matrix(c(rep(log(20),3), log(c(52,104,260))), ncol=2)

In \[43\]:

    life.lognorm<- 1-pnorm(trans2(as.numeric(lognorm.mle), life)) 
    life.loggomp<- exp(-exp(trans3(as.numeric(loggomp.mle), life)))

In \[44\]:

    lifeexp<-matrix(c(52,104,260, life.lognorm, life.loggomp), ncol=3) 
    colnames(lifeexp)<-c("t", "log-normal", "log-Gompertz")
    lifeexp

| t   | log-normal | log-Gompertz |
|-----|------------|--------------|
| 52  | 0.19971188 | 0.280844111  |
| 104 | 0.07475358 | 0.075834872  |
| 260 | 0.01278728 | 0.001387626  |

-   log-Gompertz분포로 계산했을 때는 극단적인 값(꼬리부분)에서
    log-normal분포일 때보다 더욱 잘 들어맞는다.  
    하지만 극단적이지 않은 중심부분에서는 log-Gompertz분포와는 거리가
    있어 보인다.
-   반대로, log-normal분포의 경우 극단적이지 않은 중심부분에서
    log-Gompertz분포일 때보다 더욱 잘 들어맞았고,  
    극단적인 꼬리부분에서는 log-normal분포와는 거리가 있다.
-   따라서 자료의 분포를 추정할 때, 관심사(중심부, 꼬리부분)에 따라
    분포를 다르게 설정해야 한다.

Part3. Q-Q Plot을 이용한 Weibull 분포 모수추정<a href="#Part3.-Q-Q-Plot을-이용한-Weibull-분포-모수추정" class="anchor-link">¶</a>
=================================================================================================================================

In \[46\]:

    steel <- read.csv("table1.csv", header=T)
    colnames(steel) <- c("x", "exp", "obs", "norm")

    cotton <- read.csv("table3.csv", header=T)
    colnames(cotton) <- c("x", "exp", "obs", "pears")

(a) Histogram<a href="#(a)-Histogram" class="anchor-link">¶</a>
---------------------------------------------------------------

In \[47\]:

    steel<-as.tibble(steel) %>% mutate(new=obs-lag(obs))
    steel[1,"new"]<-steel[1,"obs"]
    ggplot(steel,aes(x,new))+geom_histogram(stat="identity")+ggtitle("Histogram of Yield Strength")

    Warning message:
    "Ignoring unknown parameters: binwidth, bins, pad"

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzNNTU1ZWVlo%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///8Yrk7HAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2diVYjxxIF221gGJaZRv//sdYKkkBSZmXWrSor%0A8pw3C1DReZsblgT4eVoxDBOeqfUCDPN/GERimIRBJIZJGERimIRBJIZJGERimIRBJIZJGERi%0AmIRBJIZJGERimIQpF2maTv40nZE+fhWTr87Hr2l62P3x1/R0eOvT9OvbBidvOHnn2695DXn+%0AKNlz9/HfrsXc+VQTqVbVnqZpetz/eZ5ed394neYfrnhJpA1iO28Fe+4+HpGY00kT6fK7c2ea%0A/nz++W2ad48q89aJaysc/fllmjcf/fdli0IkJmMGFOnoL0+7J3dHz/EufejRn+fp7+4PL+vn%0Ag4jEpEzyU7s/61cw86/3zV+nw1vm9Vv2jyJvj9P8sv3Qafp42Nb/+WH9VO19j/k9Tw/rR4uX%0A9W8vR5f6Ykyf3O2sH4rWR993D0z7t78/roFvXxturvn7uPkni+95n+ucnH7erXO8+OfHb5bc%0A7c0w2SK9f774OBT+7ejlyOr39o/PuyauX6k8bx4ejl6t7N7//mv725dJR4wzkXYvjh52L5V2%0Ab3/dfcjz4Q3P27/+Ojr0uH3n5+7T8Tonpx8/V/ta/PPjd0tiErObXJF2lX7dfFlt/3i0tuNj%0A9bGu4N/tX9bvfpt3TXzcPIr8nn5vf33cEjavXZ6neffbwwF/wjh/UvU0PT/vn9ht3/Nn+zDx%0A53Fn3ek19/N38/D2enildXiqtl3n9PT8vvp42qx2uvjqsOvunQyzion0NavvX7rb/fF5/0//%0AX5vfn3ePMq+7Pm7/Yf4wfay+Dm/+4f+xf/T6Qp0wzkVaP7nbf8Xh8AC0+/LDRq7NG/bXXH/Y%0A15m/uy/bPR1dZ7/O6en9OueLr1Yn72SYVbZIT5t/1v/dv3vz68P+hf3fzQPMw6Gm07EPf95+%0AP+4JX1Idf8AJ49vL/JfPJ4H7x8SvnTb/219zvdnxoY/tNfePg0dXOz+9OoF8HIt0tiRz55P7%0A1O7v9iXP9gsFZ1/dOpbn+M8v86mKP3T0hHHlu0X7d5+qcHjv4/eg74+b55VnYvwo0vfFEYk5%0AneSv2q3eti/Czwt6UaT148nj8+vfTJFO33N8zfM/fXx9G/cHMRCJcUy2SKvtl8A/C3rrqd3D%0A9H5OcD61OxNpPvp+7dF1jl7NPH598/abGOenVzy1Y2xTQaTjgj5Pux9lO/5iw8vZE6ft17dX%0Al0U6YdwS6dfug/9sXv9s3vC0U+Pl69jr59cDXw8f9Ak5P71/z/fFEYk5nVyRdl/+ft590Wvz%0AOLJ+0va8+9L1n8NXkV+PX4E8bCq6/+L0JZFOGLdE+rP9JtCf+fDl75dp/rzmfh7Xb1o/wvx9%0A3n9n6uiL6uen9+85Xfzo4xGJ2U+uSPtvyM5/d18AW/34DdljkV4Ob3m/LNIp44ZIhw/+fPDa%0AfVf199Gxv4+Hi26+h/VwvM630/vfjhY/+XhEYvaT/NTuffsjQpt/Zv952LxS+v4jQtPT+3Fz%0AXzbvfX87fNtn9WNHjxm3RFo/0hx+wGj3htfzHxFar/G0/dcotsDdnp/vPj990PNz8ZOPRyRm%0APw2a8PHzj5j2P8MuztQfpUjbVxqbb9+83PzQvmbYxRnZKEU6vNIY7gfUhl2ckY30qd3b9l9v%0AfVVeMmeGXZxRDa+WGSZhEIlhEgaRGCZhEIlhEgaRGCZhEIlhEgaRGCZhEIlhEgaRGCZhSkVa%0AzubbG4KTzQN4D0D5gogE8P8IRCR3AoAA6/MQCeBdAhHJnQAgwPo8RAJ4l0BEcicACLA+D5EA%0A3iUQkdwJAAKsz0MkgHcJRCR3AoAA6/MQCeBdAhHJnQAgwPo8RAJ4l0BEcicACLA+D5EA3iUQ%0AkdwJAAKsz0MkgHcJRCR3AoAA6/MQCeBdAhHJnQAgwPo8RAJ4l0BEcicACLA+D5EA3iUQkdwJ%0AAAKsz0MkgHcJRCR3AoAA6/MQCeBdAhHJnQAgwPo8RAJ4l8CuRZrXc/w7IgHsFdizSPP+l/nz%0AL4gEsFMgIrkTAARYn4dIAO8SOJZI/2zm1rH/xfxbPK03Z9Rj/mLDPT4ilYuk2hBgVR5P7XKA%0AiDQYEJHcCSRARBoMiEjuBBIgIg0GRCR3AgkQkQYD9izSPf9kAyINBuxapJ+ndoQuPmeINBgQ%0AkdwJJEBEGgyISO4EEiAiDQZEJHcCCRCRBgMikjuBBIhIgwERyZ1AAkSkwYCI5E4gASLSYEBE%0AcieQABFpMCAiuRNIgIg0GBCR3AkkQEQaDIhI7gQSICINBkQkdwIJEJEGAyKSO4EEiEiDARHJ%0AnUACRKTBgIjkTiABItJgQERyJ5AAEWkwICK5E0iAiDQYEJHcCSRARBoMiEjuBBIgIg0GRCR3%0AAgkQkQYDIpI7gQSISIMBEcmdQAJEpMGAiOROIAEi0mBARHInkAARaTAgIrkTSICINBgQkdwJ%0AJEBEGgyISO4EEiAiDQZEJHcCCRCRBgMikjuBBIhIgwERyZ1AAkSkwYCI5E4gASLSYEBEcieQ%0AABFpMCAiuRNIgIg0GBCR3AkkQEQaDIhI7gQSICINBkQkdwIJEJEGAyKSO4EEiEiDARHJnUAC%0ARKTBgIjkTiABItJgQERyJ5AAEWkwICK5E0iAiDQYEJHcCSRARBoMiEjuBBIgIg0GRCR3AgkQ%0AkQYDIpI7gQSISIMBEcmdQAJEpMGAiOROIAEi0mBARHInkAARaTAgIrkTSICINBgQkdwJJEBE%0AGgyISO4EEiAiDQZEJHcCCRCRBgMikjuBBIhIgwERyZ1AAkSkwYCI5E4gASLSYEBEcieQABFp%0AMCAiuRNIgIg0GBCR3AkkQEQaDIhI7gQSICINBkQkdwIJEJEGAyKSO4EEiEiDARHJnUACRKTB%0AgIjkTiABItJgQERyJ5AAEWkwICK5E0iAiDQYEJHcCSRARBoMiEjuBBIgIg0GRCR3AgkQkQYD%0AIpI7gQSISIMBEcmdQAJEpMGAiOROIAEi0mBARHInkAARaTAgIrkTSICINBgQkdwJJEBEGgyI%0ASO4EEiAiDQZEJHcCCRCRBgMikjuBBIhIgwEHFOkuplyk1psz6uER6crwiDQYcMBHpNoRuvic%0AIdJgQERyJ5AAEWkwICK5E0iAiDQYEJHcCSRARBoMiEjuBBIgIg0GRCR3AgkQkQYDIpI7gQSI%0ASIMBEcmdQAJEpMGAiOROIAEi0mBARHInkAARaTAgIrkTSICINBgQkdwJJEBEGgyISO4EEiAi%0ADQZEJHcCCRCRBgMikjuBBIhIgwERyZ1AAkSkwYCI5E4gASLSYEBEcieQABFpMCAiuRNIgIg0%0AGBCR3AkkQEQaDIhI7gQSICINBkQkdwIJEJEGAyKSO4EEiEiDARHJnUACRKTBgIjkTiABItJg%0AQERyJ5AAEWkwICK5E0iAiDQYEJHcCSRARBoMiEjuBBIgIg0GRCR3AgkQkQYDIpI7gQSISIMB%0AEcmdQAJEpMGAiOROIAEi0mBARHInkAARaTAgIrkTSICINBgQkdwJJEBEGgyISO4EEiAiDQZE%0AJHcCCRCRBgMikjuBBIhIgwERyZ1AAkSkwYCI5E4gASLSYEBEcieQAJuLVL7Av4WRvRv2BUQk%0AdwIJEJFubtgXEJHcCSRARLq5YV9ARHInkAAR6eaGfQERyZ1AAkSkmxv2BUQkdwIJEJFubtgX%0AEJHcCSRARLq5YV9ARHInkAAR6eaGfQERyZ1AAkSkmxv2BUQkdwIJEJFubtgXEJHcCSRARLq5%0AYV9ARHInkAAR6eaGfQERyZ1AAkSkmxv2BUQkdwIJEJFubtgXEJHcCSRARLq5YV9ARHInkAAR%0A6eaGfQERyZ1AAkSkmxv2BUQkdwIJMCxSFFB+HpEkQEQyTdQDRBIDEcmdQAKMeoBIYiAiuRNI%0AgFEPEEkMRCR3Agkw6gEiiYGI5E4gAUY9QCQxEJHcCSTAqAeIJAYikjuBBBj1AJHEQERyJ5AA%0Aox4gkhiISO4EEmDUA0QSAxHJnUACjHqASGIgIrkTSIBRDxBJDEQkdwIJMOoBIomBiOROIAFG%0APUAkMRCR3AkkwKgHiCQGIpI7gQQY9QCRxEBEcieQAKMeIJIYiEjuBBJg1ANEEgMRyZ1AAox6%0AgEhiICK5E0iAUQ8QSQxEJHcCCTDqASKJgV2LNK/n+HdEMnuASGJgzyLN+1/mz78gktUDRBID%0AEcmdQAKMeoBIYmD3Iq0QqcQDRBID+xZp99roS6R/NnPr2P9iymucBCg//+/FUEyVuS3S3iIe%0AkTyTBCg/zyOSBOgRaYVIpR4gkhiISO4EEmDUA0QSAxHJnUACjHqASGIgIrkTSIBRDxBJDOxZ%0AJH6yodwDRBIDuxbp56kdoYvPWdQDRBIDEcmdQAKMeoBIYiAiuRNIgFEPEEkMRCR3Agkw6gEi%0AiYGI5E4gAUY9QCQxEJHcCSTAqAeIJAYikjuBBBj1AJHEQERyJ5AAox4gkhiISO4EEmDUA0QS%0AAxHJnUACjHqASGIgIrkTSIBRDxBJDEQkdwIJMOoBIomBiOROIAFGPUAkMRCR3AkkwKgHiCQG%0AIpI7gQQY9QCRxEBEcieQAKMeIJIYiEjuBBJg1ANEEgMRyZ1AAox6gEhiICK5E0iAUQ8QSQxE%0AJHcCCTDqASKJgYjkTiABRj1AJDEQkdwJJMCoB4gkBiKSO4EEGPUAkcRARHInkACjHiCSGIhI%0A7gQSYNQDRBIDEcmdQAKMeoBIYiAiuRNIgFEPEEkMRCR3Agkw6gEiiYGI5E4gAUY9QCQxEJHc%0ACSTAqAeIJAYikjuBBBj1AJHEQERyJ5AAox4gkhiISO4EEmDUA0QSAxHJnUACjHqASGIgIrkT%0ASIBRDxBJDEQkdwIJMOoBIomBiOROIAFGPUAkMRCR3AkkwKgHiCQGIpI7gQQY9QCRxEBEcieQ%0AAKMeIJIYiEjuBBJg1ANEEgMRyZ1AAox6gEhiICK5E0iAUQ8QSQxEJHcCCTDqASKJgYjkTiAB%0ARj1AJDEQkdwJJMCoB4gkBiKSO4EEGPUAkcRARHInkACjHiCSGIhI7gQSYNQDRBIDEcmdQAKM%0AeoBIYiAiuRNIgFEPEEkMRCR3Agkw6gEiiYGI5E4gAUY9QCQxEJHcCSTAqAeIJAYikjuBBBj1%0AAJHEQERyJ5AAox4gkhiISO4EEmDUA0QSAxHJnUACjHqASGIgIrkTSIBRDxBJDEQkdwIJMOoB%0AIomBiOROIAFGPUAkMRCR3AkkwKgHiCQGIpI7gQQY9QCRxEBEcieQAKMeIJIYiEjuBBJg1ANE%0AEgMRyZ1AAox6gEhiICK5E0iAUQ8QSQxEJHcCCTDqASKJgYjkTiABRj1AJDEQkdwJJMCoB4gk%0ABiKSO4EEGPUAkcRARHInkACjHiCSGIhI7gQSYNQDRBIDEcmdQAKMeoBIYiAiuRNIgFEPEEkM%0ARCR3Agkw6gEiiYGI5E4gAUY9QCQxEJHcCSTAqAeIJAYOKNJdTHmNkwDl5/+9GIqpMjwiXZny%0AGicBys/ziCQBIpJpoh4gkhiISO4EEmDUA0QSAxHJnUACjHqASGIgIrkTSIBRDxBJDEQkdwIJ%0AMOoBIomBiOROIAFGPUAkMRCR3AkkwKgHiCQGIpI7gQQY9QCRxEBEcieQAKMeIJIYiEjuBBJg%0A1ANEEgMRyZ1AAox6gEhiICK5E0iAUQ8QSQxEJHcCCTDqASKJgYjkTiABRj1AJDEQkdwJJMCo%0AB4gkBiKSO4FlYjUMAcIbZCTovvddtsYFRKTKPQ5vkJGg+9532RoXEJEq9zi8QUaC7nvfZWtc%0AQESq3OPwBhkJuu99l61xARGpco/DG2Qk6L73XbbGBUSkyj0Ob5CRoPved9kaFxCRKvc4vEFG%0Agu5732VrXEBEqtzj8AYZCbrvfZetcQERqXKPwxtkJFglJEj/pFQFIpI7gWViNQwBwhtkJECk%0A6kBEMtUoXMN2CyCSBIhIphqFa9huAUSSABHJVKNwDdstgEgSICKZahSuYbsFEEkCRCRTjcI1%0AbLcAIkmAiGSqUbiG7RZAJAkQkUw1Ctew3QKIJAEikqlG4Rq2WwCRJEBEMtUoXMN2CyCSBIhI%0AphqFa9huAUSSABHJVKNwDdstgEgSICKZahSuYbsFEEkCRCRTjcI1bLcAIkmAiGSqUbiG7RZA%0AJAkQkUw1Ctew3QKIJAEikqlG4Rq2WwCRJEBEMtUoXMN2CyCSBIhIphqFa9huAUSSABHJVKNw%0ADdstgEgSICKZahSuYbsFEEkCRCRTjcI1bLcAIkmAiGSqUbiG7RZAJAkQkUw1Ctew3QKIJAEi%0AkqlG4Rq2WwCRJEBEMtUoXMN2CyCSBIhIphqFa9huAUSSABHJVKNwDdstgEgSICKZahSuYbsF%0AEEkCRCRTjcI1bLcAIkmAiGSqUbiG7RZAJAkQkUw1Ctew3QKIJAEikqlG4Rq2WwCRJEBEMtUo%0AXMN2CyCSBIhIphqFa9huAUSSABHJVKNwDdstgEgSICKZahSuYbsFEEkCRCRTjcI1bLcAIkmA%0AiGSqUbiG7RZAJAkQkUw1Ctew3QKIJAEikqlG4Rq2WwCRJMCLIr0h0lGNwjVstwAiSYAXRZqm%0AXzaXakdApHgCRKoOvCjS07Sep9cPRIoBwhtkJECk6sCLIq1Wf142Mj2+IhIi1fikVAX2JNJm%0AntcqIRIi1fikVAV2JdL78zxND78RCZFqfFKqAvsR6W1jEa+REAmRTMCLIq2f0z3ftgiRjDVs%0AtwAiSYAXRXrfPSL9RaQYILxBRgJEqg68KNLBpYdnREKkGp+UqsCuRFqtPn7zVbsgILxBRgJE%0Aqg68ItKf34+T4ecbakdApHgCRKoOvCjSr9liESIZa9huAUSSAC+K9LNF8/aX9SCSr4btFkAk%0ACfCiSD8+Fm0Fmj+NQiRzDdstgEgS4EWRVqvNj9qtHv+cPB4hUmEN2y2ASBLgRZE+HjY//r1+%0Ahvd+/LwOkQpr2G4BRJIAL4r0a3peW7R6nR4vi/TPZs4fyHqcQA2jgPAGvSRgjPPtiw1f/9vN%0AvOIRqfQ8j0g3psfWuIB2kT79QaSCGrZbAJEkwIsi7Z/aPU+/DiLtBpGKathuAUSSAC+K9LH5%0Ahux65pMfW+URqbCG7RZAJAnwokir1e+HzY+snv6rFIhUWMN2CyCSBHhFpJ+Gn2worGG7BRBJ%0AAnSK9MPUjoBI8QSIVB14WaTn/Ysk/jWKECC8QUYCRKoOvCjS8zQh0r+IhEg24EWR5unlhkGI%0A5KhhuwUQSQK8KNLNRyJE8tSw3QKIJAFeFOlpMv1/CCGSrYbtFkAkCfCiSH/nx9v/D0KIZK1h%0AuwUQSQK8KNLEFxuOahSuYbsFEEkCRCRTjcI1bLcAIkmAF0UyT+0IiBRPgEjVgYhkqlG4hu0W%0AQCQJEJFMNQrXsN0CiCQBIpKpRuEatlsAkSRARDLVKFzDdgsgkgSISKYahWvYbgFEkgARyVSj%0AcA3bLYBIEiAimWoUrmG7BRBJAkQkU43CNWy3ACJJgIhkqlG4hu0WQCQJEJFMNQrXsN0CiCQB%0AIpKpRuEatlsAkSRARDLVKFzDdgsgkgSISKYahWvYbgFEkgARyVSjcA3bLYBIEiAimWoUrmG7%0ABRBJAkQkU43CNWy3ACJJgIhkqlG4hu0WQCQJEJFMNQrXsN0CiCQBIpKpRuEatlsAkSRARDLV%0AKFzDdgsgkgSISKYahWvYbgFEkgARyVSjcA3bLYBIEiAimWoUrmG7BRBJAkQkU43CNWy3ACJJ%0AgIhkqlG4hu0WQCQJEJFMNQrXsN0CiCQBIpKpRuEatlsAkSRARDLVKFzDdgsgkgSISKYahWvY%0AbgFEkgARyVSjcA3bLYBIEiAimWoUrmG7BRBJAkQkU43CNWy3ACJJgIhkqlG4hu0WQCQJEJFM%0ANQrXsN0CiCQBIpKpRuEatlsAkSRARDLVKFzDdgsgkgSISKYahWvYbgFEkgARyVSjcA3bLYBI%0AEiAimWoUrmG7BRBJAkQkU43CNWy3ACJJgIhkqlG4hu0WQCQJEJFMNQrXsN0CiCQBIpKpRuEa%0AtlsAkSRARDLVKFzDdgtkiRQGVP8s1+Qh0nbKS4BIiGQCIpKpBeEWtVsAkSRARDK1INyidgsg%0AkgSISKYWhFvUbgFEkgARydSCcIvaLYBIEiAimVoQblG7BRBJAkQkUwvCLWq3ACJJgIhkakG4%0ARe0WQCQJEJFMLQi3qN0CiCQBIpKpBeEWtVsAkSRARDK1INyidgsgkgSISKYWhFvUbgFEkgAR%0AydSCcIvaLYBIEiAimVoQblG7BRBJAkQkUwvCLWq3ACJJgIhkakG4Re0WQCQJEJFMLQi3qN0C%0AiCQBIpKpBeEWtVsAkSRARDK1INyidgsgkgSISKYWhFvUbgFEkgARydSCcIvaLYBIEiAimVoQ%0AblG7BRBJAkQkUwvCLWq3ACJJgIhkakG4Re0WQCQJMC7SCFNegn+jgPAG/5cEdzM8Iv04UUB4%0Ag4wEPCJVByKSqQXhFrVbAJEkQEQytSDconYLIJIEiEimFoRb1G4BRJIAEcnUgnCL2i2ASBIg%0AIplaEG5RuwUQSQJEJFMLwi1qtwAiSYCIZGpBuEXtFkAkCRCRTC0It6jdAogkASKSqQXhFrVb%0AAJEkQEQytSDconYLIJIEiEimFoRb1G4BRJIAEcnUgnCL2i2ASBIgIplaEG5RuwUQSQJEJFML%0Awi1qtwAiSYCIZGpBuEXtFkAkCRCRTC0It6jdAogkASKSqQXhFrVbAJEkQEQytSDconYLIJIE%0AiEimFoRb1G4BRJIAEcnUgnCL2i2ASBIgIplaEG5RuwUQSQJEJFMLwi1qtwAiSYCIZGpBuEXt%0AFkAkCRCRTC0It6jdAogkASKSqQXhFrVbAJEkQEQytSDconYLIJIEiEimFoRb1G4BRJIAEcnU%0AgnCL2i2ASBIgIplaEG5RuwUQSQJEJFMLwi1qtwAiSYCIZGpBuEXtFkAkCRCRTC0It6jdAogk%0AASKSqQXhFrVbAJEkQEQytSDconYLIJIEiEimFoRb1G4BRJIAEcnUgnCL2i2ASBIgIplaEG5R%0AuwUQSQJEJFMLwi1qtwAiSYCIZGpBuEXtFkAkCRCRTC0It6jdAogkASKSqQXhFrVbAJEkQEQy%0AtSDconYLIJIEiEimFoRb1G4BRJIAEcnUgnCL2i2ASBIgIplaEG5RuwUQSQJEJFMLwi1qtwAi%0ASYCIZGpBuEXtFkAkCRCRTC0It6jdAogkASKSqQXhFrVbAJEkQEQytSDconYLIJIEiEimFoRb%0A1G4BRJIAEcnUgnCL2i2ASBIgIplaEG5RuwUQSQJEJFMLwi1qtwAiSYCIZGpBuEXtFkAkCRCR%0ATC0It6jdAogkASKSqQXhFrVbAJEkwDFEin4Sy88jEiKZgIhUGbDkAMrPI5IEiEiVAUsOoPw8%0AIkmAiFQZsOQAys8jkgSISJUBSw6g/DwiSYCIVBmw5ADKzyOSBIhIlQFLDqD8PCJJgIhUGbDk%0AAMrPI5IEiEiVAUsOoPw8IkmAiFQZsOQAys8jkgSISJUBSw6g/DwiSYCIVBmw5ADKzyOSBIhI%0AlQFLDqD8PCJJgIhUGbDkAMrPI5IEiEiVAUsOoPw8IkmAiFQZsOQAys8jkgSISJUBSw6g/Dwi%0ASYCIVBmw5ADKzyOSBIhIlQFLDqD8PCJJgIhUGbDkAMrPI5IEiEiVAUsOoPw8IkmAiFQZsOQA%0Ays8jkgToEWlez/HviGQ+j0iGz3Lm9CzSvP9l/vwLIlnPI5Lhs5w5iPRjgugnsfw8IiGSCegQ%0A6WATIvnPI5Lhs5w5Y4n0z2ZMx4JT/kmMng8DSHAA3MuYRJpXPCKVnOcRyfBZzpzeH5EQqew8%0AIhk+y5nTuUjz6S+IZD2PSIbPcub0LdJ8ZhMiWc8jkuGznDldizSfPywhkvU8Ihk+y5nTs0jz%0AvP+RBn6ywX0ekQyf5czpWaQLUzsCIsUTIFJ1ICJVBiw5gPLziCQBIlJlwJIDKD+PSBIgIlUG%0ALDmA8vOIJAEiUmXAkgMoP49IEiAiVQYsOYDy84gkASJSZcCSAyg/j0gSICJVBiw5gPLziCQB%0AIlJlwJIDKD+PSBIgIlUGLDmA8vOIJAEiUmXAkgMoP49IEiAiVQYsOYDy84gkASJSZcCSAyg/%0Aj0gSICJVBiw5gPLziCQBIlJlwJIDKD+PSBIgIlUGLDmA8vOIJAEiUmXAkgMoP49IEiAiVQYs%0AOYDy84gkASJSZcCSAyg/j0gSICJVBiw5gPLziCQBIlJlwJIDKD+PSBIgIlUGLDmA8vOIJAEi%0AUmXAkgMoP49IEiAiVQYsOYDy84gkASJSZcCSAyg/j0gSICJVBiw5gPLziCQBIlJlwJIDKD+P%0ASBIgIlUGLDmA8vOIJAEiUmXAkgMoP49IEiAiVQYsOYDy84gkASJSZcCSAyg/j0gSICJVBiw5%0AgPLziCQBIlJlwJIDKD+PSBIgIlUGLDmA8vOIJAEiUmXAkgMoP49IEiAiVQYsOYDy84gkASJS%0AZcCSAyg/j0gSICJVBiw5gPLziCQBIlJlwJIDKD+PSBIgIlUGLDmA8vOIJAEiUmXAkgMoP49I%0AEiAiVQYsOYDy84gkASJSZcCSAyg/j0gSICJVBiw5gPLz3YgUBnhbkzqItITOIxIimYCIVBmw%0A5ADKzyNSyiDSEjqPSIhkAiJSZcCSAyg/j0gpg0hL6DwiIZIJiEiVAUsOoPw8IqUMIi2h84iE%0ASCYgIlUGLDmA8vOIlDKItITOIxIimYCIVBmw5ADKzyNSyiDSEjqPSIhkAiJSZcCSAyg/j0gp%0Ag0hL6DwiIZIJiEiVAUsOoPw8IqUMIi2h84iESCYgIlUGLDmA8vOIlDKItITOIxIimYCIVBmw%0A5ADKzyNSyiDSEjqPSIhkAiJSZcCSAyg/j0gpg0hL6DwiIZIJiEiVAUsOoPw8IqUMIi2h84iE%0ASCYgIlUGLDmA8vOIlDKItITOIxIimYCIVBmw5ADKzyNSyiDSEjqPSIhkAiJSZcCSAyg/j0gp%0Ag0hL6DwiIXX5XGsAAAe5SURBVJIJiEiVAUsOoPw8IqUMIi2h84iESCYgIlUGLDmA8vOIlDKI%0AtITOIxIimYBxkQxTfguTAOXnwwASpAEGmYqPSOW3cOERKSMBj0jh6eKpXegWIlI8ASKFB5Fi%0ACyASIpmAiFQZkHELwgkQKTyIFFsAkRDJBESkyoCMWxBOgEjhQaTYAoiESCYgIlUGZNyCcAJE%0ACg8ixRZAJEQyARGpMiDjFoQTIFJ4ECm2ACIhkgmISJUBGbcgnACRwoNIsQUQCZFMQESqDMi4%0ABeEEiBQeRIotgEiIZAIiUmVAxi0IJ0Ck8CBSbAFEQiQTEJEqAzJuQTgBIoUHkWILIBIimYCI%0AVBmQcQvCCRApPIgUWwCREMkERKTKgIxbEE6ASOFBpNgCiIRIJiAiVQZk3IJwAkQKDyLFFkAk%0ARDIBEakyIOMWhBMgUngQKbYAIiGSCYhIlQEZtyCcAJHCg0ixBRAJkUxARKoMyLgF4QSIFB5E%0Aii2ASIhkAiJSZUDGLQgnQKTwIFJsAURCJBMQkSoDMm5BOAEipdTw6gURqS4g4xaEEyBSSg2v%0AXhCR6gIybkE4ASKl1PDqBRGpLiDjFoQTIFJKDa9eEJHqAjJuQTgBIqXU8OoFEakuIOMWhBMg%0AUkoNr14QkeoCMm5BOAEipdTw6gURqS4g4xaEEyBSSg2vXhCR6gIybkE4ASKl1PDqBRGpLiDj%0AFoQTIFJKDa9eEJHqAjJuQTgBIqXU8OoFEakuIOMWhBMgUkoNr14QkeoCMm5BOAEipdTw6gUR%0AqS4g4xaEEyBSSg2vXhCR6gIybkE4ASKl1PDqBRGpLiDjFoQTIFJKDa9eEJHqAjJuQTgBIqXU%0A8OoFEakuIOMWhBMgUkoNr14QkeoCMm5BOAEipdTw6gURqS4g4xaEEyBSSg2vXhCR6gIybkE4%0AASKl1PDqBRGpLiDjFoQTIFJKDa9eEJHqAjJuQTgBIqXU8OoFEakuIOMWhBMgUkoNr14QkeoC%0AMm5BOAEipdTw6gURqS4g4xaEEyBSSg2vXhCR6gIybkE4ASKl1PDqBRGpLiDjFoQTIFJKDa9e%0AEJHqAjJuQTgBIqXU8OoFEakuIOMWhBMgUkoNr14QkeoCMm5BOAEipdTw6gURqS4g4xaEEyBS%0ASg2vXhCR6gIybkE4ASKl1PDqBRGpLiDjFoQTIFJKDa9eEJHqAjJuQTgBIqXU8OoFEakuIOMW%0AhBMgUkoNr14QkeoCMm5BOAEipdTw6gURqS4g4xaEEyBSSg2vXhCR6gIybkE4ASKl1PDqBRGp%0ALiDjFoQTIFJKDa9eEJHqAjJuQTgBIqXU8OoFEakuIOMWhBMgUkoNr14QkeoCMm5BOAEipdTw%0A6gURqS4g4xaEEyBSSg2vXhCR6gIybkE4ASKl1PDqBRGpLiDjFoQTIFJKDa9eEJHqAjJuQTgB%0AIqXU8OoFC0Sa14NIrvOIhEjfPfr8BZGs5xEJkRApYQFEQiRESlgAkRDpikj/bMZ6jGHuYio+%0AIoUmmwfwHoDyBREJ4P8RiEjuBAAB1uchEsC7BCKSOwFAgPV5ffxkQ9UEAAHW5/Xxs3ZVEwAE%0AWJ+HSADvEohI7gQAAdbnIRLAuwQikjsBQID1eYgE8C6BiOROABBgfR4iAbxLICK5EwAEWJ+H%0ASADvEohI7gQAAdbnIRLAuwQikjsBQID1eYgE8C6BiOROABBgfR4iAbxLICK5EwAEWJ+HSADv%0AEohI7gQAAdbnIRLAuwQikjsBQID1eYgE8C6BiOROABBgfR4iAbxLICK5EwAEWJ+HSADvEohI%0A7gQAAdbnCUSqPf3/JwHZMD7db2hfEJFKhw3j0/2GiFR/2DA+3W+ISPWHDePT/Ybji8QwQw0i%0AMUzCIBLDJAwiMUzCIBLDJAwiMUzC9CTS4b+bfvzfT59P/2Pqjed4w/nsbX3Mfpv5fMOOVlwv%0AtP3l7LPcbp3vU1DDjkSa97/Mn385+r2L+dxs/uFtXcyP966b7fYzf97C/WZ93cMfFjQsh0iO%0AGUuk+eSN/cy86lyk7wuOJdJ2zjbv5+5+zny6VWcl2MzZvetrucP961ek+fs/gwy79S5Sb8+e%0Ad/f5+AXI5y/dzHz066q7ezikSLdvYVcindVz1dstPmzYbwmO7uFpGbrZcF51LtLxU3dHDbsS%0AafXzY2kvt3g3Pz357HHD+Ye3tZ/Tl5kd3sPLL41GFKnPEuxmEJHmH97WwcyHL8x3K9LRdw48%0ANexIpPn0l9UPf208h216LcEI93AzPT8ibefco7Ge2p0+6n+9rZ87fCxSn6/iLonUzYLbGUMk%0AXw07Eunbd+Xnr7d1Muc/e9Hvhscd7WvD1aGn3d7Dr50cNexJJIYZdhCJYRIGkRgmYRCJYRIG%0AkRgmYRCJYRIGkRgmYRCJYRIGkRgmYRCJYRIGkRgmYRCJYRIGkcacp+nPavVnemy9B7MfRBpz%0APqaH1epxYxPTxSDSoPN7enudnltvwRwGkUad3v4dnjsfRBp1XqfptfUOzOcg0qiDSF0NIo06%0A88MDT+36GUQadH5Pb2/T79ZbMIdBpDFn++Xvh+mj9R7MfhBpzNl/Q/ap9R7MfhCJYRIGkRgm%0AYRCJYRIGkRgmYRCJYRIGkRgmYRCJYRIGkRgmYRCJYRIGkRgmYRCJYRIGkRgmYf4Dp+bf49Af%0AU2IAAAAASUVORK5CYII=)

In \[48\]:

    cotton<-as.tibble(cotton) %>% mutate(new=obs-lag(obs))
    cotton[1,"new"]<-cotton[1,"obs"]
    ggplot(cotton,aes(x,new))+geom_histogram(stat="identity")+ggtitle("Histogram of Fiber Strength")

    Warning message:
    "Ignoring unknown parameters: binwidth, bins, pad"

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzNNTU1ZWVlo%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///8Yrk7HAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2di1ZbORJF77iB8Eb4/z92/ARDsFGFUqnqsM9a%0AE+jk2PuqpB0/cPcsa0LIj7PMvgBCFIJIhDgEkQhxCCIR4hBEIsQhiESIQxCJEIcgEiEOQSRC%0AHIJIhDjk5yIty4fvlk/3+Prnx4Qv8/pnWa7eycecuYpzefyz2tzN7ev+Hq1XsDaQiHaGizTq%0AoN1srLl+h/ybSDfHmz3+w5Xu+4hEtnEX6fwf+2ZZni9A+qj3y2pr0Mv97s4Qifx7Cot0CdJH%0AXS0v+2/ulz+IRH6SQU/tnjevYFZ/ng5Puva/s9r8zuFR5PF6Wd2v90/EXq+Wm81v3V5tnqo9%0AHe7mbrVcbR4r7jdf7k9Q7/exvN3vp2t4v4rNfVw/7n/r6Xo5fP+G+/vS354ZHhqnN1rf7i/o%0A9NLfn0luLnN/5eT3ZoxIT28vPY4H/vHkxcj6bvft7f4cbl6n3G4fHE5eq+z//OnP7su7SSf3%0A8b1It3vG9ncelrfvj7h9rt+/Xb+LsW98uNH128W9X/pbf3+ZmPS7M0akq+VhvT2LV2+PUBs7%0AXtevmwP4svuHzR8/rvbn8Hr7ltndcrf79Xp3D9tXLrfLav/l7a25D/dx/qnd4SpWe8Tz9nbb%0Ax4vn672AO9whL9sHuIfnD3dyaHy80epp/XqzvbiPl74+Xu3+D8kvjodIp++Z/fWm2f7b28Pf%0A/X+2X2/3jzIP+9O4+6v8anldv994+1f/6+HR6/2uPtzHZ5H+uoCtyZtzv8Pt393ePmH79Mjx%0Asn/b7uaEdGh8vNHhgj5f+nr94Q/JL84YkW62f9O/HP54++vV4WX9y/YB5up4SJdTH54f764P%0A9/Au1Wnhw318K9L+gWeP+/Te+Gled9TDI+EJ74sbbb/8fel/XSb5lRnz1O5l95Jn90bBp/e2%0ATk/z6ff3q48mfHFCP9zHt0/tThEXRNrl6Xr7zPKTGF+K9PelIxLZZtC7duvH3Uvwz8fzrEj3%0AmweF24eXQSJ9XTz5h9dldUEMRCLfZpRI691b4G/H87undlfL0+d7sD21+/T921O7ZfuG4POX%0AxfX18nj6+x/E+OJGPLUj5zNQpNPjebvsP8h2+mbD/aenTbv3t9fnRfpwH9+KtEM8bV/8/Nnf%0A7nn7/YdbPby9I/jw/meHxhc32n75+9IRiWwzRqT929+3+7e8to8jmydtt/u3rp+P7yE/nL7+%0AuNoe0P27ymdF+nAf34q0Q6y2D3TPu58GPa/272SfXvr1snrYPMK83B5+NnXytvoXN9p++Xjp%0AJ31E+uUZI9LhB7Krl/3bX+svfyB7KtL98Xeezov08T6+E+nu7eepx9v99Ti2frk+Yrc/xbr6%0A8HbE3zfafTm59A99RPrlGfTU7mn3EaHt39jPV9tXSn9/RGi5eTo9t/fbP3163D+Gnd7v6Qk9%0AvY9v32y4e//gzsvt8bNGn8/7483uX6PY3eX+St8af93o8PTz7dI/9BHpl2fi/r++f+itWgpf%0AOhmTGSLtP3bwdL3cf1vNlsKXToZmhkjH1xkFP55W+NLJ0Ex5ave4+9dbH2agf5rCl05GhtfI%0AhDgEkQhxCCIR4hBEIsQhiESIQxCJEIcgEiEOQSRCHIJIhDjkpyK195x+PzyhMGWa8NICaIgE%0AbQZMjoZI0GbA5GiIBG0GTI6GSNBmwORoiARtBkyOhkjQZsDkaIgEbQZMjoZI0GbA5GiIBG0G%0ATI6GSNBmwORoiARtBkyOhkjQZsDkaIgEbQZMjoZI0GbA5GiIBG0GTI6GSNBmwORoiARtBkyO%0AhkjQZsDkaIgEbQZMjoZI0GbA5GiIBG0GTI6GSNBmwORoiARtBkyOhkjQZsDkaIgEbQZMjoZI%0A0GbA5GiIBG0GTI6GSNBmwORoiARtBkyOhkjQZsDkaIgEbQZMjoZI0GbA5GiIBG0GTI6mINJ/%0AnfGhRQaRytAQyUqLDCKVoSGSlRYZRCpDQyQrLTKIVIaGSFZaZBCpDA2RrLTIIFIZGiJZaZFB%0ApDI0RLLSIoNIZWiIZKVFBpHK0BDJSosMIpWhIZKVFhlEKkNDJCstMohUhoZIVlpkEKkMDZGs%0AtMggUhkaIllpkUGkMjREstIig0hlaIhkpUUGkcrQEMlKiwwilaEhkpUWGUQqQ0MkKy0yiFSG%0AhkhWWmQQqQwNkay0yCBSGRoiWWmRQaQyNESy0iKDSGVoiGSlRQaRytAQyUqLDCKVoSGSlRYZ%0ARCpDQyQrLTKIVIaGSFZaZBCpDA2RrLTIIFIZGiJZaZFBpDI0RLLSIoNIZWiIZKVFBpHK0BDJ%0ASosMIpWhIZKVFhlEKkNDJCstMohUhoZIVlpkEKkMDZGstMggUhkaIllpkUGkMjREstIig0hl%0AaIhkpUUGkcrQEMlKiwwilaEhkpUWGUQqQ0MkKy0yiFSGhkhWWmQQqQwNkay0yCBSGRoiWWmR%0AQaQyNESy0iKDSGVoiGSlRQaRytAQyUqLDCKVoSGSlRYZRCpD+20i/VQ6tf2fBZOjIRIizYDJ%0A0RAJkWbA5GiIhEgzYHI0REKkGTA5GiIh0gyYHA2REGkGTI6GSIg0AyZHQyREmgGToyESIs2A%0AydEQCZFmwORoiIRIM2ByND+R5qVXDmuXkH8Ij0ifu1/TIsMjUhkaIiHSDJgcDZEQaQZMjoZI%0AiDQDJkdDJESaAZOjIRIizYDJ0RAJkWbA5GiIhEgzYHI0REKkGTA5GiIh0gyYHA2REGkGTI6G%0ASIg0AyZHSytS94FHpIowORoiIdIMmBwNkRBpBkyOhkiINAMmR0MkRJoBk6MhEiLNgMnREAmR%0AZsDkaIiESDNgcjREQqQZMDkaIiHSDJgcDZEQaQZMjoZIiDQDJkdDJESaAZOjIRIizYDJ0RAJ%0AkWbA5GiIhEgzYHI0REKkGTA5GiIh0gyYHA2REGkGTI6GSIg0AyZHQyREmgGToyESIs2AydEQ%0ACZFmwORoiIRIM2ByNERCpBkwORoiIdIMmBwNkRBpBkyOhkiINAMmR0MkRJoBk6MhEiLNgMnR%0AEAmRZsDkaIiESDNgcjREQqQZMDkaIiHSDJgcDZEQaQZMjoZIiDQDJkdDJESaAZOjIRIizYDJ%0A0RAJkWbA5GiIhEgzYHI0REKkGTA5GiIh0gyYHA2REGkGTI6GSIg0AyZHQyREmgGToyESIs2A%0AydEQCZFmwORoiIRIM2ByNERCpBkwORoiIdIMmBwNkRBpBkyOhkiINAMmR0MkRJoBk6MhEiLN%0AgMnREAmRZsDkaIiESDNgcjREQqQZMDkaIiHSDJgcDZEQaQZMjoZIiDQDJkdDJESaAZOjIRIi%0AzYDJ0RAJkWbA5GiIhEgzYHI0REKkGTA5GiIh0gyYHA2REGkGTI6GSIg0AyZHQyREmgGToyES%0AIs2AydEQCZFmwORoiIRIM2ByNERCpBkwORoiIdIMmBwNkRBpBkyOhkiINAMmR0MkRJoBk6Mh%0AEiLNgMnREAmRZsDkaIiESDNgcjREQqQZMDkaIvl0xwSRytAQyac7JohUhoZIPt0xQaQyNETy%0A6Y4JIpWhIZJPd0wQqQwNkXy6Y4JIZWiI5NMdE0QqQ0Mkn+6YIFIZGiL5dMcEkcrQEMmnOyaI%0AVIaGSD7dMUGkMjRE8umOCSKVoSGST3dMEKkMDZF8umOCSGVoiOTTHRNEKkNDJJ/umCBSGRoi%0A+XTHBJHK0BDJpzsmiFSGhkg+3TFBpDI0RPLpjgkilaEhkk93TBCpDA2RfLpjgkhlaIjk0x0T%0ARCpDQySf7pggUhkaIvl0xwSRytCMIq12v2xy+hWRRgWRytBsIu3EWR2MOn5FpGFBpDI0k0ir%0ANSKd6Y4JIpWhWUQ6yINIX3THBJHK0FxE+t82397cls5DbKgO7RJyyLcirdY8Ip3tjgmPSGVo%0A/SK9eYNIX3THBJHK0Awi7YNIX3bHBJHK0PpFentYQqQvumOCSGVoiOTTHRNEKkP7B5H4ZMMX%0A3TFBpDI0o0gX4nzZ3YcYkSrC5GiI5NMdE0QqQ0Mkn+6YIFIZGiL5dMcEkcrQEMmnOyaIVIaG%0ASD7dMUGkMjRE8umOCSKVoSGST3dMEKkMDZF8umOCSGVoiOTTHRNEKkNDJJ/umCBSGRoi+XTH%0ABJHK0BDJpzsmiFSGhkg+3TFBpDI0RPLpjgkilaEhkk93TBCpDA2RfLpjgkhlaIjk0x0TRCpD%0AQySf7pggUhkaIvl0xwSRytAQyac7JohUhoZIPt0xQaQyNETy6Y4JIpWhIZJPd0wQqQwNkXy6%0AY4JIZWiI5NMdE0QqQ0Mkn+6YIFIZGiL5dMcEkcrQEMmnOyaIVIaGSD7dMUGkMjRE8umOCSKV%0AoSGST3dMEKkMDZF8umOCSGVoiOTTHRNEKkNDJJ/umCBSGRoi+XTHBJHK0BDJpzsmiFSGhkg+%0A3TFBpDI0RPLpjgkilaEhkk93TBCpDA2RfLpjgkhlaIjk0x0TRCpDQySf7pggUhkaIvl0xwSR%0AytAQyac7JohUhoZIPt0xQaQyNETy6Y4JIpWhIZJPd0wQqQwNkXy6Y4JIZWiI5NMdE0QqQ0Mk%0An+6YIFIZGiL5dMcEkcrQEMmnOyaIVIaGSD7dMUGkMjRE8umOCSKVoSGST3dMEKkMDZF8umOC%0ASGVoiOTTHRNEKkNDJJ/umCBSGRoi+XTHBJHK0BDJpzsmiFSGhkg+3TFBpDI0RPLpjgkilaEh%0Akk93TBCpDA2RfLpjgkhlaIjk0x0TRCpDQySf7pggUhkaIvl0xwSRytAQyac7JohUhoZIPt0x%0AQaQyNETy6Y4JIpWhIZJPd0wQqQwNkXy6Y4JIZWiI5NMdE0QqQ0Mkn+6YIFIZGiL5dMcEkcrQ%0AEMmnOyaIVIaGSD7dMUGkMjRE8umOCSKVoSGST3dMEKkMDZF8umOCSGVoiOTTHRNEKkNDJJ/u%0AmCBSGRoi+XTHBJHK0BDJpzsmiFSGhkg+3TFBpDI0RPLpjgkilaEhkk93TBCpDA2RfLpjgkhl%0AaIjk0x0TRCpDQySf7pggUhkaIvl0xwSRytAQyac7JohUhoZIPt0xQaQyNETy6Y4JIpWhIZJP%0Ad0wQqQwNkXy6Y4JIZWiI5NMdE0QqQ0Mkn+6YIFIZGiL5dMcEkcrQEMmnOyaIVIaGSPFdw+44%0ADDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6G%0ASPFdw+44DDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6GSPFdw+44%0ADDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6GSPFdw+44DDInTI6G%0ASPFdw+44DDInTI7mJ5JzOg+moZqmS6TDI1JU1/DXnMMgc8LkaIgU3zXsjsMgc8LkaIgU3zXs%0AjsMgc8LkaIgU3zXsjsMgc8LkaIgU3zXsjsMgc8LkaIgU3zXsjsMgc8LkaIgU3zXsjsMgc8Lk%0AaIgU3zXsjsMgc8LkaIgU3zXsjsMgc8LkaIgU3zXsjsMgc8LkaIgU3zXsjsMgc8LkaGdFekSk%0AUV3D7jgMMidMjnZWpGX5Y3PJ+bK7DyYiOUXtaMfSzop0s2xy8/CKSO5dw+44DDInTI52VqT1%0A+vl+K9P1AyI5dw274zDInDA52gWRtrndqIRIzl3D7jgMMidMjnZJpKfb1bJc3SGSc9ewOw6D%0AzAmTo50V6XFrEa+RRnQNu+MwyJwwOdpZkTbP6W77LUIkQ9ewOw6DzAmTo50V6Wn/iPSCSO5d%0Aw+44DDInTI52VqSjS1e3iOTcNeyOwyBzwuRol0Rar1/veNfOv2vYHYdB5oTJ0S6I9Hx3vRg+%0A3+B82d0HE5Gcona0Y2lnRfqzsliESIauYXccBpkTJkc7KxKftUMkaAbCOZH49DciQTMQzom0%0AXm8/are+fkYk765hdxwGmRMmRzsr0uvV9uPfm2d4T4jk3DXsjsMgc8LkaGdF+rPcbixaPyzX%0AiOTcNeyOwyBzwuRoZ0Xa/vzo+D9Ecu0adsdhkDlhcjREiu8adsdhkDlhcrSzIh2e2t0ufxDJ%0AuWvYHYdB5oTJ0c6K9Lr9gewmq96PrTpfdvfBRCSnqB3tWNpZkdbru6vtR1a7/1UK58vuPpiI%0A5BS1ox1LuyCSMc6X3X0wEckpakc7loZI8V3D7jgMMidMjnZepNvDiyTetfPuGnbHYZA5YXK0%0AsyLdLgsijekadsdhkDlhcrSzIq2W+06DEMnYNeyOwyBzwuRoZ0XqfiRCJGvXsDsOg8wJk6Od%0AFelmMf03hBCpv2vYHYdB5oTJ0c6K9LK67v8vCCGSpWvYHYdB5oTJ0c6KtPBmw6iuYXccBpkT%0AJkdDpPiuYXccBpkTJkc7K5I5zpfdfTARySlqRzuWhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yO%0AhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPu%0AOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOhkjxXcPuOAwyJ0yOZhFptclXXxEJkX49%0AzSDS6vDL56+IZOwadsdhkDlhcjREiu8adsdhkDlhcjSDSEebEOmHXcPuOAwyJ0yO5iLS/7bp%0Aunl/Og+moZqmS6TTJdJqzSNSVNdpkN1Re4yIpSFS2q7TILujdrRjaTaRVqe/INLYrtMgu6N2%0AtGNpJpFWH2xCpLFdp0F2R+1ox9IsIq0+Piwh0tiu0yC7o3a0Y2kGkVarw0cZ+GRDSNdpkN1R%0AO9qxNMsj0uU4X3b3aUMkp6gd7VgaIqXtOg2yO2pHO5aGSGm7ToPsjtrRjqUhUtqu0yC7o3a0%0AY2mIlLbrNMjuqB3tWBoipe06DbI7akc7loZIabtOg+yO2tGOpSFS2q7TILujdrRjaYiUtus0%0AyO6oHe1YGiKl7ToNsjtqRzuWhkhpu06D7I7a0Y6lIVLartMgu6N2tGNpiJS26zTI7qgd7Vga%0AIqXtOg2yO2pHO5aGSGm7ToPsjtrRjqUhUtqu0yC7o3a0Y2mIlLbrNMjuqB3tWBoipe06DbI7%0Aakc7loZIabtOg+yO2tGOpSFS2q7TILujdrRjaYiUtus0yO6oHe1YWqhIhsOGSIhUioZIabsX%0ABjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7%0AloZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsX%0ABjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7%0AloZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsX%0ABjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7%0AloZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsX%0ABjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7loZIabsXBjkkakc7%0AloZIabsXBjkkakc7loZIabvNsDaXo+BxJ7+Whkhpuw2RCtEQKW23IVIhGiKl7TZEKkRDpLTd%0AhkiFaIiUttsQqRANkdJ2GyIVoiFS2m5DpEI0RErbbYhUiIZIabsNkQrREClttyFSIRoipe02%0ARCpEQ6S03YZIhWiIlLbbEKkQDZHSdhsiFaIhUtpuQ6RCNERK222IVIiGSGm7DZEK0RApbbch%0AUiEaIqXtNkQqREOktN2GSIVoiJS22xCpEA2R0nYbIhWiIVLabkOkQjREStttiFSIhkhpuw2R%0ACtEQKW23IVIhGiKl7TZEKkRDpLTdhkiFaIiUttsQqRANkdJ2GyIVoiFS2m5DpEI0RErbbYhU%0AiIZIabsNkQrREClttyFSIRoipe02RCpEQ6S03YZIhWiIlLbbEKkQDZHSdhsiFaIhUtpuQ6RC%0ANERK222IVIiGSGm7DZEK0RApbbchUiEaIqXtNkQqREOktN2GSIVoiJS22xCpEA2R0nYbIhWi%0AIVLabkOkQjREStttiFSIhkhpuw2RCtEQKW23IVIhGiKl7TZEKkRDpLTdhkiFaIiUttsQqRAN%0AkdJ2GyIVoiFS2m5DpEI0RErbbYhUiIZIabsNkQrREClttyFSIRoipe02RCpE8xOpI52nwtrt%0ArFbrmkdGUoRHpGTdxiNSIRoipe02RCpEQ6S03YZIhWiIlLbbEKkQDZHSdhsiFaIhUtpuQ6RC%0ANERK222IVIiGSGm7DZEK0RApbbchUiEaIqXtNkQqREOktN2GSIVoiJS22xCpEA2R0nYbIhWi%0AIVLabkOkQjREStttiFSIhkhpuw2RCtEQKW23IVIhGiKl7TZEKkRDpLTdhkiFaIiUttsQqRAN%0AkdJ2GyIVoiFS2m5DpEI0RErbbYhUiIZIabsNkQrREClttyFSIRoipe02RCpEQ6S03YZIhWiI%0AlLbbEKkQDZHSdhsiFaIhUtpuQ6RCNERK222IVIiGSGm7zSqSpfvVUfjhUbJFjYZIabsNkQrR%0AEClttyFSIRoipe02RCpEQ6S03YZIhWiIlLbbEKkQDZHSdhsiFaIhUtpuQ6RCNERK222IVIiG%0ASGm7DZEK0RApbbchUiEaIqXtNkQqREOktN2GSIVoiJS22xCpEA2R0nYbIhWiIVLabkOkQjRE%0AStttiFSIhkhpuw2RCtEQKW23IVIhGiKl7TZEKkRDpLTdhkiFaIiUttsQqRANkdJ2GyIVoiFS%0A2m5DpEI0RErbbYhUiIZIabsNkQrREClttyFSIRoipe02RCpEQ6S03YZIhWiIlLbbEKkQDZHS%0AdhsiFaIhUtpuQ6RCNERK222IVIiGSGm7DZEK0RApbbchUiEaIqXttpEiWbpDgkiIFNRtiFSI%0Ahkhpuw2RCtEQKW23IVIhGiKl7TZEKkRDpLTdhkiFaIiUttsQqRANkdJ2GyIVoiFS2m5DpEI0%0ARErbbYhUiIZIabsNkQrREClttyFSIRoipe02RCpEQ6S03YZIhWiIlLbbEKkQDZHSdhsiFaIh%0AUtpuQ6RCNERK222IVIiGSGm7DZEK0RApbbchUiEaIqXttiwiWbr9B89W/2EQ6fsuIv2HSPNp%0AiJS22xDJLYj0fReR/kOk+TREStttiOQWRPq+i0j/IdJ8GiKl7TZEcgsifd9FpP8QaT4NkdJ2%0AW0GROqv/IRIihXUbIrkFkXp2pH/3psuBSMcuIiFSULchklsQqWdH+ndvuhyIdOwiEiIFdRsi%0AuQWRenakf/emy4FIxy4iIVJQtyGSWxCpZ0f6d2+6HIh07CISIgV1GyK5BZF6dqR/96bLgUjH%0ALiIhUlC3IZJbEKlnR/p3b7ociHTsIhIiBXUbIrkFkXp2pH/3psuBSMcuIiFSULchklsQqWdH%0A+ndvuhyIdOwiEiIFdRsiuQWRenakf/emy4FIxy4iIVJQtyGSWxCpZ0f6d2+6HIh07CISIgV1%0Am7hI/V2HY+5xJ5cJiJS12xDp/Rp+esw97uQyAZGydhsi/cP92rp++YlIq00QCZFOup3VNNvm%0Alx+ItHr7BZEQCZEQKWm3IdI/3C8ifVphdzfHjgzoNkT6h/sdtRUhIv1vG+vNCZFMwCPSkKj9%0AZG8aTXhpud/+RiQtmvDSEKlnBZEwZZrw0hCpZwWRMGWa8NIQqWcFkTBlmvDScotk/2TDmBVE%0AwpRpwktLLtLHhF72LJgyTXhpiNSzgkiYMk14aYjUs4JImDJNeGmI1LOCSJgyTXhpiNSzgkiY%0AMk14aYjUs4JImDJNeGmI1LOCSJgyTXhpiNSzgkiYMk14aYjUs4JImDJNeGmI1LOCSJgyTXhp%0AiNSzgkiYMuQAEIsAAAJESURBVE14aYjUs4JImDJNeGmI1LOCSJgyTXhpiNSzgkiYMk14aYjU%0As4JImDJNeGmI1LOCSJgyTXhpiNSzgkiYMk14aYjUs4JImDJNeGmI1LOCSJgyTXhpiNSzgkiY%0AMk14aYjUs4JImDJNeGmI1LOCSJgyTXhpiNSzgkiYMk14aYjUs4JImDJNeGmI1LOCSJgyTXhp%0AiNSzgkiYMk14aYjUs4JImDJNeGmlRDqJ8P97H0srmci1IVJPWFrJIFK2sLSSQaRsYWklU1Qk%0AQn5vEIkQhyASIQ5BJEIcgkiEOASRCHGIm0gf/8/OpbJSXdt+UZqre19bzOK8RFq9/aIX0WWt%0AV++bJrfEgz5x60Kk7yO7LF2RVmtEShfNVW2jK1L8shDp28i+RPoFIsVtHSJ9G92l6Yv09svw%0AIFJfJJcmL9Kn70YGkfoiuTRE8gsifRvdpcmLxFO7TFmtlVemunNvyyr2ZoPoz8d3kV3a7/hk%0AQwyPz9oR4hBEIsQhiESIQxCJEIcgEiEOQSRCHIJIhDgEkQhxCCIR4hBEIsQhiESIQxCJEIcg%0AUt3cLM/r9fNyPfs6yBqRKud1uVqvr7c2kelBpMK5Wx4fltvZV0G2QaTK0fxXiUoGkSrnYVke%0AZl8D2QWRKgeR0gSRKmd1dcVTuxxBpMK5Wx4fl7vZV0G2QaS62b39fbW8zr4Oskakyjn8QPZm%0A9nWQNSIR4hJEIsQhiESIQxCJEIcgEiEOQSRCHIJIhDgEkQhxCCIR4hBEIsQhiESIQxCJEIf8%0AH4B/Og2GkkqjAAAAAElFTkSuQmCC)

-   히스토그램을 통해 왼쪽으로 살짝 치우치고 오른쪽으로 꼬리를 갖는
    분포임을 알 수 있다.
-   따라서 EVD는 Gumbel, Weibull, Frechet 분포를 따를 가능성이 높다고
    생각한다.

(b) 3-parameter Weibull 분포의 모수 추정<a href="#(b)-3-parameter-Weibull-분포의-모수-추정" class="anchor-link">¶</a>
---------------------------------------------------------------------------------------------------------------------

In \[49\]:

    n1<-389
    n2<-3000

방법1<a href="#방법1" class="anchor-link">¶</a>
-----------------------------------------------

\$\\alpha\$값 변화 -\> \$x\$축 \$\[-ln(1-p\_r)\]^{1/\\alpha}\$, \$y\$축
\$X\_{(r)}\$인 Q-Q plot

In \[50\]:

    lm1_steel<-function(a) {
      steel_new<-steel %>% filter(x>a) %>% mutate(pr=obs/(n1+1),probit=(-log(1-pr))^(1/a))
      lm1<-lm(x~probit,data=steel_new)
      intercept1<-as.numeric(format(summary(lm1)$coefficients[1],digits=4))
      slope1<-as.numeric(format(summary(lm1)$coefficients[2],digits=4))
      rsqr1<-summary(lm1)$r.squared
      return(list(rsqr1,intercept1,slope1))
    }

    lm1_cotton<-function(a) {
      cotton_new<-cotton %>% filter(x>a) %>% mutate(pr=obs/(n2+1),probit=(-log(1-pr))^(1/a))
      lm3<-lm(x~probit,data=cotton_new)
      intercept3<-as.numeric(format(summary(lm3)$coefficients[1],digits=4))
      slope3<-as.numeric(format(summary(lm3)$coefficients[2],digits=4))
      rsqr3<-summary(lm3)$r.squared
      return(list(rsqr3,intercept3,slope3))
    }

In \[51\]:

    interval<-0.01
    a1<-seq(interval,10,by=interval)
    Rsquared1<-rep(NA,length(a1))
    Rsquared3<-rep(NA,length(a1))

In \[52\]:

    for (i in 1:length(a1)) {
      Rsquared1[i]<-lm1_steel(a1[i])[[1]]
      Rsquared3[i]<-lm1_cotton(a1[i])[[1]]
    }

In \[53\]:

    alpha_steel<-interval*which.max(Rsquared1)
    a_steel<-lm1_steel(alpha_steel)[[2]]
    b_steel<-lm1_steel(alpha_steel)[[3]]

In \[54\]:

    alpha_cotton<-interval*which.max(Rsquared3)
    a_cotton<-lm1_cotton(alpha_cotton)[[2]]
    b_cotton<-lm1_cotton(alpha_cotton)[[3]]

In \[55\]:

    steel_new<-steel %>% mutate(pr=obs/(n1+1),probit=(-log(1-pr))^(1/alpha_steel))
    cotton_new<-cotton %>% mutate(pr=obs/(n2+1),probit=(-log(1-pr))^(1/alpha_cotton))

In \[56\]:

    ggplot(steel_new,aes(probit,x))+
      geom_point()+
      geom_smooth(method="lm",se=F)+
      ggtitle(paste("Steel",",","alpha=",alpha_steel,",",
                    "R2=",format(Rsquared1[which.max(Rsquared1)],digits=3),",",
                    "a=",a_steel,",","b=",b_steel))

    `geom_smooth()` using formula 'y ~ x'

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzMzZv9NTU1o%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///9uyY2wAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2diXajuhJFeZdMnc7Q9v9/7LPBA9gMAkrilLTP%0AurdDEsKmJLYNMkN1JIRsTrX3ChCSQxCJEIMgEiEGQSRCDIJIhBgEkQgxCCIRYhBEIsQgiESI%0AQRCJEINYifT1XlfVy8fhPH14nyTOIh/nmP+Lez5fT6vx2fnBoWrTfvdRV/XHKLbNpYqBZfXz%0Affr161cP/lJV77/P0z9v9+nF+fNyWuXDGHJq0XPt1m+aa77bH1S39KfJcIya5u3a0udenm7v%0AqCK9tmvxev/JV2cT+K2byXoMe019GF5WL9/tr//cf/LR/uT7cfqnnV5nUrsW9e8wcnLRc+3W%0AbZp7Xh5EqvvTZDg2In1W9dmg38+q+tlTpI/q9bRNfdedbe2jur+Cv1afh+PvazX8nnTFfLcz%0ADCyrl5fq7/H4t7NuX1V98ub3tX6YPr3KfJ6b6C20im4+z2txeG//+Ak5vei5dus2TQfY+7Ov%0Ac4cOTJOH2IhUX18TP6v3PUWq2zl/Oi+dL9XhOvnTbnCHkRfWG6b984FldfPdCvZx3rbbvLbv%0AP9/nn3Snj9eX+NAqurmsfvPHz8jpRc8RO01zy6F66f7ZT/U5OE0eYyPSve1PU7fdhe4e/X16%0ArHs/Tj34+n2d4/T/Z33/i/P098NsQetz2jBuP/0z9Ao8UsTQj/ur2748/1TvjzOede1OX5Qc%0A3S8Kqaj542fk9KJPK/Gnfjim6qTbNLe8Vd/dguuX4WnyGBuRentLV5H+tvvVzW860yMbZt0/%0AxjodCXT+4v12wNGbbfQY+L6JfFUfp43ppXkNf5s+THl4R+ouawD12k51tsabPHV/+iRws/81%0AsosYUNFpb/TvIHJ60afW67Th06K7TXPN98n8DvxPZ2fuDzt2U7ER6XQUX7//vTZ02xM/zcvs%0Az+t5E+lOD4v0p9kY/jSH9q1I9amHv+r2mOt0BHZ4O/+uP9uYSF/3Lety0H9V+Oulqkc26Psx%0AUt19WWiWNYC6qXL7yUu7pX2df9KdPp7fF6oxjwIqumg2gJxedL8NnxbdbZp7DYfO4n87B1+/%0A647xionRqN1vO2z31unwj3YP/HDugO70sEjdY4FWpPbd4zxY3W5Hh3YDvc82npfuIdL5Fff3%0AonD7LjcyDnff1LozvIyO8fW/XsYFzttt1Z8+v4Cc22b47XC+op/Xt3YsZ0CkyUVX7cHUz8jg%0ASrdpbiV8dhf/1nkTeuMNaTJmnwwcvv68XrbBtideOi+B3enRTeanWcLl768b89vL7S8uX26z%0AjWZgF+67Vfi8PZ62nOGD5usq9j46Gt0dHNiq25Hqj+YnnemfRqqf13psx3K+okMzePGMnF50%0AdRlMGDoWuuf7/lbT7jTeFn/ovKIcRj8GIE1sP2L7fm13hJol9+SZFemzvu14NHNdOv+l6ovU%0AmW0sg6O6Lbn5xc/IhtUs9FD3/npwWfe5j/1yPptDjvYn9+n36wje8OfUARU1xgwhpxc9tIoT%0AszUjDd3vu4MzMwM1xESk7iBZfRdpaIaxbv08vZl9/P19Eum1L1J/tsEjiuFtv6vw5Gjx9+VT%0A1N6yBlBv14U9vlL/dA4mfnojeAGFD1Z0/eNn5PSiu7+dWfR1qjdP3R2+s33FzS8m7dPZzb5v%0AsHVnp7o7PXaMdBv5bpdx2Rofdu36sw1sG4fXenAkuXoJFOn02lsfHpc1gBoYi27z2Tkkaaan%0AsbMV3f94YsR9RKTD7bdTi+6MO3bnYahhSUxE+nvri7/3XZD3tr+bnZLu9NSr51dnsKEdnqg/%0AeiJ1ZxvKb/1wuFC3y/k+b9GXg6PvmRMBXi4b6tOyerl8OtrZ5bmgmt2t7vTrdRUGjzJmKros%0A6Ou8Ts/IuUV/Tvy21zTPa3QZeTg+T5Oh2Lxjv1b131Ov/H5U7ejYeQP8aQZWf+p2+Ps+PfbC%0A/Hkb5WqNOQ8MfDWd3XtHus82kEP9OMb2Ub0dmsGtw3mTac/bGdndvy70pznPaWBZj2vcnK9z%0An+mjej0cD5/NZtud/mrG/sewMxX9OS/oPCL/M4ScXnTVDNv9ravhT3u7TdP/s/YrY3ZLYiPS%0A7+t1n+D8mvnS7htczolsXu86051NpjP5eV3A91WkdkC9M+x7OUa6zTaQ9+6+SftnlzX77EB6%0AHx4PTL83b7C9ZQ3kcr7o39ufHi6nxP4+TF8/sBnGzlR0Xf8/g8jpRXc/1J5Y9OfTn7VfXzoD%0Ali8rT7ktJ1bHkF9vzWUU7U78S/ui+ftR38aS79PDIp0Hr+r376/r0fnp/9NG8v7Tme06aned%0AbaiaZ5Ga6xmup8n8vD+Mbg+PgtTtxynTInXPgGrnOZxrvGy13enj1+PVDyOFD+a8/m/fI8jJ%0ARVftKULjZx91mmagV+ZHiMg9+zbQOD1Vz+1Uf0Qsm/wu2bXZ/45/UphIpIk1cIrdqaLis6tI%0AH+NDQYlEmlgDp9idKio+qjsC7JQTV2F7JcQgiESIQRCJEIMgEiEGQSRCDBIkUnOeQl3fTz6r%0Au98QQkJEapxpZbr+JNr6EOIyASLVR0QiZDrzItUdb+reF0LIJetEuh0i/e+cOGtGiKPMinTb%0ArXv8en9b+heUwNk2JD4hiyIg2BHCRRo/NEIkCMUTFohU3/fj+odGiASheEK4SB1n6v73iASh%0AeMIaker+951vLddsSxAJQmLCCpFuu3jXbxAJQvGEhSLNxHLNtgSRICQmIJIsAoInAiLJIiB4%0AIiCSLAKCJwIiySIgeCIgkiwCgicCIskiIHgiIJIsAoInAiLJIiB4IiCSLAKCJwIiySIgeCIg%0AkiwCgicCIskiIHgiIJIsAoInAiLJIiB4IiCSLAKCJwIiySIgeCIgkiwCgicCIskiIHgiIJIs%0AAoInAiLJIiB4IiCSLAKCJwIiySIgeCIgkiwCgicCIskiIMgQqqqanQuRVBEQVAhVNW8SIski%0AIIgQqirAJESSRUAQISCSbwQEEQIi+UZAUCFwjOQaAUGGwKidZwQETwREkkVA8ERAJFkEBE8E%0ARJJFQPBEQCRZBARPBESSRUDwREAkWQQETwREkkVA8ERAJFkEBE8ERJJFQFAh/PdfwFyIpIqA%0AIEL4778AkxBJFgFBgxDkESLpIiBIEMI8QiRdBAQFQqBHiKSLgCBACPUIkXQREPYnnD1i+Ns5%0AAsLehP+a9yNEco6AsDOh9QiRvCMg7Eu4Hh4hknMEhF0Jt2EGRHKOgLAn4T5ch0jOERB2JHSG%0AvRHJOQLCfoTux0eI5BwBYTdC72NYRHKOgLAXoX86AyI5R0DYifBwWhAiOUdA2IfweHodIjlH%0AQNiF8HSaKiI5R0DYg/B8ujciOUdA2IEwcNkEIjlHQEhPGLr8CJGcIyAkJwxexodIzhEQUhOG%0AL4dFJOcICGkJ/41cVo5IzhEQkhLGPEIk7wgIKQnjdzlBJOcICAkJE3cLQiTnCAjpCFN33UIk%0A5wgIyQiTd69DJOcICKkI03eBRCTnCAiJCDN3U0Uk5wgIaQhzdyVGJOcICEkIs3f3RiTnCAgp%0ACPN3yUck5wgICQgBT5tAJOcICPEJIU9t2UMkQjyleWqLdXhHEkNAiE0IfDps2HogkioCQmRC%0A6NNhw9YDkVQREOISgp8OG7YeiKSKgBCVEP502LC5EEkVASEmIdgjRPKOgBCREO4RInlHQIhH%0AWOARInlHQIhGWOIRInlHQIhFWOQRInlHQIhEWOYRInlHQIhDWOgRInlHQIhBGL193UYCIski%0AIEQgLPcIkbwjINgTlmuESO4REMwJazxCJO8ICNaEVR4hkncEBGPCOo8QyTsCgi1hpUeI5B0B%0AwZSw1iNE8o6AYEhYMey9jIBIsggIdoQNHiGSdwQEM8IGjRDJPQKCFWGTR4jkHQHBiLDNI0Ty%0AjoBgQ9joESJ5R0AwIWz1CJG8IyBYEDZ7hEjeERAMCNs9QiTvCAjbCQYeIZJ3BITNBAuPEMk7%0AAsJWgolHiOQdAWEjwcYjRPKOgLCNYOQRInlHQNhEsPIIkbwjIGwhmHmESN4REDYQ7DxCJO8I%0ACOsJhh4hkncEhNUES48QyTsCwlqCqUeI5B0BYSXB1iNE8o6AsI5g7BEieUdAWEWw9giRvCMg%0ArCGYe4RI3hEQVhDsPUIk7wgIiwlbbl8XRhifC5FUERCWEqJ4hEjeERAWEqJohEjuERCWESJ5%0AhEjeERAWEWJ5hEjeERCWEKJ5hEjeERAWEOJ5hEjeERDCCRE9QiTvCAjBhJgeIZJ3BIRQQlSP%0AEMk7AkIgIa5HiOQdASGMENkjRPKOgBBEiO0RInlHQAhJdI8QyTsCQkDie4RI3hEQ5pPAI0Ty%0AjoAwmxQeIZJ3BISZNJcfqdSASLIICNNpL+NTqQGRZBEQJnPZrVOpAZFkERCmcj08UqkBkWQR%0AECZyG2ZQqQGRZBEQxnMfrlOpAZFkERBG0xn2VqkBkWQREMbS/fhIpQZEkkVAGE7/7nUqNSCS%0ALALCYB7uAqlSAyLJIiAM5fGsIJUaEEkWAWEgT2fXqdSASLIICM95PktVpQZEkkVAeMrA2d4q%0ANSCSLALCY4aumlCpAZFkERAeMnj1kUoNiCSLgNDP8FV8KjUgkiwCQi8jV8Oq1IBIsggI3Yxd%0AVa5Sw0KR6uafU24/6EwjEoRYhNG7M6jUsEykRpr6ZlR/GpEgxCL0Paqqypwwnggi1UdESoeA%0AcMuTR3eTVGpYIlJ9RKSECAjXDHh0M0mlBjOR/nfO7CIIWZzmrlv3XETabXXmMrtm9ZF3pJQI%0ACG0exxm8vyPdnEGkNAgITZ7H65wfI9VtECkVAsI5Q+Pe7kfteEdKiYDwL+Du3io1IJIsAkLI%0AXfJValgh0u1shrozjUgQ7AkBT5tQqWGhSDOxXLMtQaQsCCFPbVGpAZFkEcUTgp5+pFIDIski%0ASieEPUVMpQZEkkWUTfgvzCOZGhBJFlE0IdQjmRoQSRZRMiH84bAqNSCSLKJgwoKHLKvUgEiy%0AiHIJSx5WrlIDIskiiiUs8UimBkSSRZRKWOSRTA2IJIsolLDMI5kaEEkWUSQheNh7NWFxEMk5%0AokTCYo9kakAkWUSBhMUa6dSASLKI8ggrPJKpAZFkEcUR1ngkUwMiySJKI6zySKYGRJJFFEZY%0A55FMDYgkiyiLsNIjmRoQSRZRFGGtRzI1IJIsoiTCao9kakAkWURBhPUeydSASLKIcggbPJKp%0AAZFkEcUQtnikUgMi6SJKIWzySKQGRBJGFELY5pFGDf8QSRhRBmGjRxI1NHMhkiqiCMJWjxRq%0AaOdCJFVECYTNHgnUcJkLkVQRBRC2e7R/Dde5EEkVkT/BwKPda7jNhUiqiOwJFh7tXcN9LkRS%0AReROMPFIppUQSRaROcHGI5lWQiRZRN4EI49kWgmRZBFZE6w8kmklRJJFZExYfvu6pQTDIJJz%0ARL4EQ49kWgmRZBHZEgw10mklRJJF5Eow9UimlRBJFpEpwdYjmVZCJFlEngRjj2RaCZFkEVkS%0ArD2SaSVEkkXkSDD3SKaVEEkWkSHB3iOZVkIkWUR+hAgeybQSIskisiPE8EimlRBJFpEbIYpH%0AMq2ESLKIzAhxPJJpJUSSReRFiOSRTCshkiwiK0Isj2RaCZFkETkRonkk00qIJIvIiBDPI5lW%0AQiRZRDYEy8uPhglRg0jOEbkQonok00qIJIvIhBBVI51WQiRZRB6EyB7JtBIiySKyIMT2SKaV%0AEEkWkQPh4lFVVbEIKq2ESLKIDAh3j6KZpNJKiCSL8E/oehTLJJVWQiRZhHvC9fgIkRBpT4R3%0AwtmjhoBIiLQnwjmheT9qCRwjIdKOCN+Edr/uQmDUDpH2Q7gmXI6PXNewiIBIsgjPhOs4g+ca%0AlhEQSRbhmHA7n8FxDQsJiCSL8Eu4nxfkt4alBESSRbgldM6vc1vDYgIiySK8ErrnqXqtYTkB%0AkWQRTgm9872d1rCCgEiyCJ+E/nUTPmtYQ0AkWYRLwsP1Ry5rWEVAJFmER8LjdXwea1hHQCRZ%0AhEPC0/WwDmtYSUAkWYQ/wvN15f5qWEtAJFmEO8LA/Rnc1bCaYCsSKTfN7ev2XgmF8I4khvBF%0AGL4NpK8athAQSRbhijBy2y1XNWwiIJIswhNh7PZ1nmrYRkAkWYQjwuhtIB3VsJGASLIIP4Tx%0A26n6qWErAZFkEW4IE7cldlPDZgIiySK8EKZu7+2lhu0ERJJF+CBMP/3IRw0WBESSRbggzDxF%0AzEUNJgREkkV4IMw9tcVDDTYERJJFOCDMPv3IQQ1GBESSRegT5p8ipl+DFQGRZBHyhICn8cnX%0AYEZAJFmEOiHkqZbqNdgREEkWIU4IejqseA2GBESSRUgTZoa9DQiBUSEgkixCmhD4tHLpGkwJ%0AiCSLUCaEaaRdgy0BkWQRwoTA9yPpGowJiCSL0CUEeyRcgzUBkWQRsoRwj3RrMCcgkixClbDA%0AI9ka7AmIJIsQJSzxSLWGCAREkkVoEhZ5JFpDDAIiySIkCcs80qwhCgGRZBGKhIUeSdYQh4BI%0AsghBwlKPFGuIREAkWYQeYbFHgjXEIiCSLEKOsNwjvRqiERBJFqFGWOGRXA3xCIgkixAjrPFI%0ArYaIBESSRWgRVnkkVkNMAiLJIpQIgdfxbSCsjQoBkWQRQoS1HinVEJmASLIIHcJajZRqiE1A%0AJFmEDGG9Rzo1RCcgkixChbDBI5ka4hMQSRYhQtjikUoNCQiIJIvQIGzySKSGFAREkkVIELZ5%0ApFFDEgIiySIECKuHvYMJm6NCQCRZxP6EzR4J1JCKgEiyiN0JmzUSqCEZAZFkEXsTDDzavYZ0%0ABESSRexMsPBo7xoSEhBJFrEvwcSj7FupMxciqSJ2Jdh4lHsrdedCJFXEPoSqqv6ZeZRtKw3M%0AhUiqiF0I1TlmHuXaSkNzIZIqYg9C41Fl5lGmrTQ4FyKpInYTyc6jTFtpcC5EUkXsJZKhR5m2%0A0uBciKSK2OkYydKjXFtpaC5EUkXsQzD1KNtWGpgLkVQRuxBsPcq1lYbmQiRVxB4EY48ybaXB%0AuRBJFbEDwdqjPFtpeC5EUkUkJ2y//GiOECEqBESSRaQmRPAow1YanQuRVBGJCRE0yrCVxudC%0AJFVEWkIUj7JrpYm5EEkVkZQQx6PcWmlqLkRSRaQkRPIos1aanAuRVBEJCbE8yquVpudCJFVE%0AOkI0j7JqpZm5EEkVkYwQz6OcWmluLkRSRaQiRPQoo1aanQuRVBGJCDE9yqeV5udCJFVEGkJU%0Aj7JppYC5EEkVkYQQ16NcWilkLkRSRaQgRPYok1YKmguRVBEJCLE9yqOVwuZCJFVEfEJ0j7Jo%0AJURyjohOiO9RDq2ESN4RsQkJPMqglRDJPSIyIYVH/lspmIBIsoi4hLNH3mtQIiCSLCIqoXk/%0Acl6DFAGRZBExCe1+ne8atAhLRKpPuXxpJq4/q29zWK7ZliDSdC7HR65rECMsEKm+/TPwFZEc%0AEa7jDJ5rUCOsFKnu/RCRXBFu43WOa5AjLBCpK87wGxIiOSB0bl/ntgZBwjqR7m9It0Ok/50T%0AtAiyZxqP9l6JrBMg0m1cob9nx2CDH0LvY1inNUgSFol0fHpHevjOcs22BJFG0j+dwWcNmoRV%0AItUDP0MkB4SH04Jc1iBKWCBS3f9nyCvLNdsSRBrM4+l1HmtQJRiI1Hl/slyzLUGkoTydpuqw%0ABlnCApGOtxG6zpBD78QGRFImPJ/u7a8GXcISkeZjuWZbgkhPGXr6kbcalAmIJIuwFykuYSjl%0AEBBJFmFKGLyKz1kN0gREkkVYEoYvh/VVgzYBkWQRhoSRy8pd1SBOQCRZhB1h7PYMnmpQJyCS%0ALMKMMHqbE0c1yBMQSRZhRRi/XZCfGvQJiCSLMCJM3HbLTQ0OCIgki7AhTN2+zksNHgiIJIsw%0AIUzeBtJJDS4IiCSLsCBM307VRw0+CIgkizAgzNyW2EUNTgiIJIvYTpi7vbeHGrwQEEkWsZkw%0Ae5t8BzW4ISCSLGIrYf5xE/o1+CEgkixiIyHgsS3yNTgiIJIsYhNh6Do+W0JQyiEgkixiCyHI%0AI/EafBEQSRaxgRD4ND7pGpwREEkWsZ4Q+lRL5Rq8ERBJFrGaEPx0WOEa3BEQSRaxlhD+lGXd%0AGvwREEkWsZKw4GnlsjU4JIyK9PrTfj28IdI+iKWEqqr+LfJIsAa/hFGRqurj/OVPteSdynLN%0AtqREkapzwoa9VxJWpBzCqEg/r1X9929dvfwg0j6IZYQVHsnV4JkwKtLxeLKoevlaoBEi7Ui4%0AehSPsCblECZE+kKkXRGLRVrqkVwNngmjIv28sGu3L2LpMdJij/RqcEwYFamq/py/MNiwG2Ih%0AYblHejU4JoyK9HZovzL8vRdiGWGFR3I1eCaMirQqlmu2JeWJtMYjtRpcExBJFrGAsHDYewVh%0AZcohIJIsIpyw0iOpGrwTEEkWEUxYqZFUDe4JiCSLCCWs9kioBv8ERJJFBBLWe6RTQwYERJJF%0AhBE2eCRTQw4ERJJFBBG2eKRSQxYERJJFhBA2eSRSQx4ERJJFBBC2eaRRQyYERJJFzBM2eiRR%0AQy4ERJJFzBK2eqRQQzYERJJFzBE2eyRQQz4ERJJFzBC2e7R/DRkREEkWMU0w8Gj3GnIiIJIs%0AYpJg4dHeNWRFQCRZxBTBxKMsOkKFgEiyiAmCjUdZdIQKAZFkEeMEI4+y6AgVAiLJIkYJVh5l%0A0REqBESSRYwRzDzKoiNUCIgkixgh2HmURUeoEBBJFjFMMPQoi45QISCSLGKQYOlRFh2hQkAk%0AWcQQwdSjLDpChYBIsogBgq1HWXSECgGRZBFPhLW3rwsnmKccAiLJIh4J5h5l0REqBESSRTwQ%0AzDXKoyNUCIgki+gTIniURUeoEBBJFtEjxPAoi45QISCSLKJLiOJRFh2hQkAkWUSHEMejLDpC%0AhYBIsog7IZJHWXSECgGRZBE3QiyPsugIFQIiySKuhGgeZdERKgREkkVcCPE8yqIjVAiIJIto%0ACRE9yqIjVAiIJItoCDE9yqIjVAiIJIs4E6J6lEVHqBAQSRZxjO1RFh2hQkAkWcQxtkdZdIQK%0AAZFkEcfYHmXRESoERFJF2F9+9JQcOkKFgEiiiAQeZdERKgRE0kQk0CiPjlAh2IpEjHL2aO91%0AIOvCO5IOonk/yqGZyiEgkiCi3a/LoZnKISCSHuJyfJRDM5VDQCQ5xHWcIYdmKoeASGKI+7B3%0ADs1UDgGRxBD3ce8cmqkcAiJpITofH+XQTOUQEEkK8R8iOSUgkhKidz5DDs1UDgGRhBD984Jy%0AaKZyCIikg3g4vy6HZiqHgEgyiMfzVHNopnIIiKSCeDrfO4dmKoeASHsjqqo6f3m+biKHZiqH%0AgEg7I6qqMWng+qMcmqkcAiLti6jaDF3Hl0MzlUNApH0R4x5l0UzlEBBpX8S4R1k0UzkERNoZ%0AMepRFs1UDgGR9kaMeZRFM5VDQKS9EaP3C8qhmcohINK+iInb1+XQTOUQEGlXxNRtIHNopnII%0AiLQnYvI2kDk0UzkERNoRMX071RyaqRwCIu2HmLktcQ7NVA4BkXZDzN3eO4dmKoeASHshZm+T%0An0MzlUNApJ0Q84+byKGZyiEg0i6IkKcf5dBM5RAQaQ9E0FPEcmimcgiItAMi7CliOTRTOQRE%0ASo8IfBpfDs1UDgGRkiNCn2qZQzOVQ0Ck1Ijgp8Pm0EzlEBApMSL8Kcs5NFM5BERKi1jwtPIc%0AmqkcAiIlRSzwKItmKoeASCkRSzzKopnKISBSQsQij7JopnIIiJQOscyjLJqpHAIiJUMs9CiL%0AZiqHgEipEEs9yqKZyiEgUiLEYo+yaKZyCIiUBrHcoyyaqRwCIiVBrPAoi2Yqh4BIKRBrPMqi%0AmcohIFICxCqPsmimcgiIFB+xzqMsmqkcAiJFR6z0KItmKoeASLERaz3KopnKISBSZMRqj7Jo%0ApnIIiBQXsd6jLJqpHAIiRUVs8CiLZiqHgEgREUG3r9tE2BYIdgREiofY5lEWzVQOAZGiIbZp%0AlEczlUNApFiIrR5l0UzlEBApEmKzR1k0UzkERIqD2O5RFs1UDgGRoiAMPMqimcohIFIMhIVH%0AWTRTOQREskdsHPYOIBgFgh0BkcwRRh5l0UzlEBDJGmGkUR7NVA4BkYwRZh5l0UzlEBDJFmHn%0AURbNVA4BkUwRhh5l0UzlEBDJEmHpURbNVA4BkQwRph5l0UzlEBDJDmHrURbNVA4BkcwQxh5l%0A0UzlEBDJCmHtURbNVA4BkYwQ5h5l0UzlEBDJBmHvURbNVA4BkUwQETzKopnKISCSBSKGR1k0%0AUzkERDJARPEoi2Yqh4BI2xFxPMqimcohINJWhNXlR+OEaIFgR1giUn3K9Wv98LNiRYrmURbN%0AVA5hgUj19Z964GelinTWqKqqiISIgWBHQKRNiItHUUzKoZnKISwQ6WpO3f+2ZJFuHsUwKYdm%0AKoewRqTuIdLtn/+dE7SIfHL26HgRae91ISoJ2BRagbrvQkW/I7XDDLwjQfi36h2p+7Vkka7D%0AdRwjQUCk9Yj7sDejdhDWjNqxa3dOtI+PbsmhmcohrBTpebChMJHie5RFM5VDWCBS78yGq0CF%0AntmQwKMcmqkgwhKR5mO5ZlsSm5DCowyaqSQCIq1I8/lRXMS/DJqpKAIiLU/zfuS9CAi2BERa%0AnHa/znkREIwJiLQ0l+Mj30VAsCYg0sJcxxlcFwHBnIBIy3Ibr/NcBAR7AiItyn3c23ERECIQ%0AEGlJOp8f+S0CQgwCIi1I93NYt0VAiEJApPD0zmfwWgSEOARECk7/vCCnRUCIRECk0DycX+ez%0ACAixCIgUlqfb13ksAkI8AiIF5fk2kA6LgBCRgEghGbhswl8REGISECkgQ5cfuSsCQlQCIs1n%0A8DI+b0VAiEtApNkMXw7rrAgIkQmINJeRy8p9FQEhNgGRZjJ2ewZXRUCITkCkyYw//chRERAS%0AEBBpKhNPEfNTBIQUBESayNRdt9wUASEJAZHGM3n3Oi9FQEhDQKTRTN8F0kkREBIREGksM3dT%0A9VEEhFQERHWmpWUAAApcSURBVBrJ3F2JXRQBIRkBkYYze3dvD0VASEdApMHM3yXfQREQEhIQ%0AaSgBT5vQLwJCSgIiDSTkqS3yRUBISkCk5wQ9/Ui9CAhpCYj0lLCniIkXASExAZEeE/g0Pu0i%0AIKQmINJDQp9qKV0EhOQEROon+OmwykVASE9ApF7Cn7IsXASEHQiI1M2Cp5XrFgFhDwIidbLA%0AI90iIOxCQKR7lngkWwSEfQiIdMsij1SLgLATAZGuWeaRaBEQ9iIg0iULPdIsAsJuBERqs9Qj%0AySIg7EdApCaLPVIsAsKOBET6N3n7OivEikDwRECkdR7JFQFhXwIirditW4xYFQieCIi0ziOx%0AIiDsTShepJUeaRUBYXdC6SKt9UiqCAj7EwoXabVHSkVAECCULdJ6j4SKgKBAKFqkDR7pFAFB%0AglCySFs8kikCggahYJE2eaRSBAQRQrkibfNIpAgIKoTCRKqq6jK10SOZDoSgQShLpKq6mrTV%0AI5kOhKBBKEqkqrqatNkjmQ6EoEEoU6TtHsl0IAQNQpEiGXgk04EQNAhFidSatOryo2CEXSB4%0AIpQl0j8zj2Q6EIIGoTCRTA6P5hBGgeCJUJpIVh7JdCAEDUJhIpl5JNOBEDQIZYlk55FMB0LQ%0AIBQlkqFHMh0IQYNQkkiWHsl0IAQNQjkiGQ17TyFsA8EToRiRjD2S6UAIGoRSRDLWSKcDIWgQ%0AChHJ3COZDoSgQShDJHuPZDoQggahCJEieCTTgRA0CLYiaebs0d7rQIpJtu9IMd6PdF4JIWgQ%0A8hcpjkcyHQhBg5C9SJE8kulACBqE3EWK5ZFMB0LQIGQuUjSPZDoQggYhb5HieSTTgRA0CFmL%0AFNEjmQ6EoEHIWaSYHsl0IAQNQsYiRfVIpgMhaBDyFSmuRzIdCEGDkK1IkT2S6UAIGoRcRYrt%0AkUwHQtAgZCpSdI9kOhCCBiFPkeJ7JNOBEDQIWYqUwCOZDoSgQchRpBQeyXQgBA1ChiIl8Uim%0AAyFoEPITqbkeNiqhiUoHQtAg5CZSe/s6RIKQmJCZSJfbQCIShMSEvES6Hh4hEoTEhKxEug0z%0AIBKExIScRLoP1yEShMSEjETqDHsjEoTEhHxE6n58hEgQEhOyEan3MSwiQUhMyESkh6cfIRKE%0AxISMRIpLeIxKB0LQIOQh0uPZdYgEITEhC5GeTlNFJAiJCTmI9Hy6NyJBSEzIQKSByyYQCUJi%0Agn+Rhi4/QiQIiQnuRRq8jA+RICQmeBdp+HJYRIKQmOBcpJHLyhEJQmKCb5HGbs+ASBASE1yL%0ANHqbE0SCkJjgWaTx2wUhEoTEBMciTdx2C5EgJCb4FWnq9nWIBCExwa1Ik7eBRCQIiQleRZq+%0AnSoiQUhM8CnSf9MeIRKE1ASXIs15hEgQUhM8ijR/l3xEgpCY4FCkgKdNIBKExAQ3IlVV1U6E%0APLUFkSAkJngRqaouJgU9/QiRICQmOBGpqi4mhT1FDJEgJCY4EynwaXyIBCExwZVIs8PeGwhL%0Ao9KBEDQITkT6t8gjRIKQmuBFpH+tR4EzIxKExAQ3Ii16WDkiQUhMcCPSEo8QCUJqgheRFnmE%0ASBBSE5yItMwjRIKQmuBDpIUeIRKE1AQXIi31CJEgpCZ4EGmxR4gEITXBgUjLPUIkCKkJ+iKt%0A8AiRIKQmyIu0xiNEgpCaoC7SKo8QCUJqgrhI6zxCJAipCdoirfQIkSCkJkiLtNYjRIKQmiAs%0AUvjlR2sJW6LSgRA0CLoibfAIkSCkJsiKtEEjRIKQnKAq0iaPEAlCaoKoSNs8QiQIqQmaIm30%0ACJEgpCZIirTVI0SCkJqgKNJmjxAJQmqCoEjbPUIkCKkJeiIZeIRIEFIT5ESy8AiRIKQmqIlk%0A4hEiQUhNWCJSfUr363X69s12kWw8QiQIqQkLRKov/9S3bzpfbUQy8giRIKQmSIlk5REiQUhN%0AWCBST5x6yKONIpl5hEgQUhO2inQ7RPrfOUGLGMvZo00LIEQgASLdxxV6X20GG+zej3hHgpCc%0AsEik4/AunYlIlh4hEoTUhHUi1QM/2yaSqUeIBCE1YYFIdf+f48C3q9fM1iNEgpCasEakuv+z%0Azrdr18zYI0SCkJqwQKTrGQ31daiuvv9sm0jWHiEShNSEJSLNZ92amXuESBBSEwREsvcIkSCk%0AJuwu0pbb14UR4kSlAyFoEPYWKYpHiAQhNWFnkaJohEgQkhP2FSmSR4gEITVhV5FieYRIEFIT%0A9hQpmkeIBCE1YUeR4nmESBBSE/YTKaJHiAQhNWEvkeIMe3cJkaPSgRA0CDuJFNcjRIKQmrCP%0ASHE1QiQIyQm7iBTbI0SCkJqwh0jRPUIkCKkJO4gU3yNEgpCakF6kBB4hEoTUhOQipfAIkSCk%0AJiQWqRn2VqldHAHBEyGxSGeTZGoXR0DwREgtUviabQkiQUhMQCRZBARPBESSRUDwREAkWQQE%0AT4TEIlVVpVO7OAKCJ0JakapzZGoXR0DwREgqUtVGpXZxBARPBESSRUDwREAkWQQET4SkInGM%0ABCFXQlqRGLWDkCkhsUgL1mxLEAlCYgIiySIgeCIgkiwCgicCIskiIHgiIJIsAoInAiLJIiB4%0AIiCSLAKCJwIiySIgeCIgkiwCgicCIskiIHgiIJIsAoInAiLJIiB4IiCSLAKCJwIiySIgeCIg%0AkiwCgicCIskiIHgiIJIsAoInAiLJIiB4IiCSLAKCJwIiySIgeCIgkiwCgicCIskiIHgiIJIs%0AAoInAiLJIiB4IiCSLAKCJwIiySIgeCIgkiwCgicCIskiIHgiIJIsAoInAiLJIiB4ItiKpJL/%0A7b0CFsmhiBJrQCSx5FBEiTUgklhyKKLEGhBJLDkUUWINOYlEyG5BJEIMgkiEGASRCDEIIhFi%0AEEQixCA5iFSf0p2up2bWzX21c6nBYxGPG1Po32UgUn3759jtSW/p9N/tH2/pbHcu17/f9ov6%0AAZFEUh/9i1QfEclxnmp3Gv8iPdbgNYjkds/8nLxEctwRiOR6G8xMpN53vtI7TChUpOPjtKPk%0AsBHWE9+5CSJ1f+AviKSRdf2QmUiOt8G8RPJbw8PAY8Eiuey+YxYbYb8GlyXUD5MFiXT7ALru%0ATDtMffvXbxHua6ivw41La8hBJEJ2DyIRYhBEIsQgiESIQRCJEIMgEiEGQSRCDIJIhBgEkQgx%0ACCLlkKrqTn16PKXAexAph/RFqujU9KHNc0hfHUTaIbS5n5wEeatef5upn/r1ePx9r6r334ff%0AnP7DpPShyf3kZM3Jkfpwnnqt3o+Hurp9/3abQqRdQpP7yUmfw/G1+jhPnf45flSnd6XL9/ff%0AsGu3S2hzPznt0J1256qX89R5N+7l/O/9+8sUIu0S2txPWkHuqgx9j0g7hTb3E0QSDm3uJ5cd%0AuNerKuzaCYU295Pq5NDhtfpzVaU32ND5DSLtENrcT86D3lVVH6+qjAx/H9t5SNIgkp+cduBe%0Abx/AntP/QPbt+ptPREofRPITdtmEQ9/4CSIJh77xE0QSDn3jJ4gkHPqGEIMgEiEGQSRCDIJI%0AhBgEkQgxCCIRYhBEIsQgiESIQf4PcSh4WPVuJfQAAAAASUVORK5CYII=)

In \[57\]:

    ggplot(cotton_new,aes(probit,x))+
      geom_point()+
      geom_smooth(method="lm",se=F)+
      ggtitle(paste("Cotton",",","alpha=",alpha_cotton,",",
                    "R2=",format(Rsquared3[which.max(Rsquared3)],digits=3),",",
                    "a=",a_cotton,",","b=",b_cotton))

    `geom_smooth()` using formula 'y ~ x'

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzMzZv9NTU1o%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///9uyY2wAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2dAXuayhZFeTVJk9w0bfL/f+wT0YgKCsMMZ+9h%0A7e/e1iZmuTm4IqJC800IWZwmugAhNQSRCMkQRCIkQxCJkAxBJEIyBJEIyRBEIiRDEImQDEEk%0AQjIEkQjJkEUi/ffSNM3Lx+03vn6f/0xL09z/94O8XVz9q+kydlNdnl6/ui+8P+//8T7K/rP/%0A9vPFQr8/Nc3vv/cvz87tzbRfPC3E21Oz6/o2PxmiPJzbGXRKv3W/xOfLgqWpPgtE+twdV+Dz%0ALbU5/5mWZSJ9XF79Y5JITbM73KGex5aqy5/u22/nr7x2X/lz7/Ls3N5Mm6fjQnQdd3/79XfD%0AC3f/ZnqggaXpl/jsLmPScNLv6/vBvrTT3v/Sermhxor0fiXNazPwqHmD3i/I6+HKz/s7y5/d%0A9X34lKfmv/1jce8GPprdfg5/n3d3Ls/Pzc20OS3Ye9vx63d/8B/N572FG8kt6KJ1v8RL895e%0A/2Zdk0PS7+tPzWnr5/nmfhoq0udz83R59afma+zKffTn4Zf6run94zZ/OsFe27tYl+fTb+//%0Axi/Pzu3NfLebqMcFOy5Qbyk/m+Ft0QdzuwX1W1+UWL5Oq07yXD7Ov5s+D7/Jvz9/75rd7/b3%0AYrcd9bM1df5Gux5ed83T2APE6/6O8vzneMX2//fdcRP9ePnP1dUGF2lf52J97+9/95bkfN2L%0Anxq5y7x2v/k/m9/XV2wHMnZ5kHRvKW5v5rt9UPhzWfHs+m5kEffXf9vdPte6vtYZ1G99UaL7%0ABTPy64Uki/Ry/TB0eiLycS1S7xv7rz3/XL7Nrn/F9v+3w79fD//+/bPpfnG126farbIX97eP%0A5nV/Z3oae2C4ekT6+fLTIP65u9ST8+eutxu//GhhJ9xM+wDx0l+wv8/nh6u34Q27w2+V3gxv%0Ah3UNumh9UeKt27Qb2eDdfJJF2l1tLu2fMr19fX+9Hp6O9jcDrr6x3wL/ehl+Jv92WEtvh292%0AIu32K/hj1xzM2H0cf/DyasP3jYt/Hp8+d4+bt/l5jrTrX+OjvZUB/M8d7ecrT92d+LCDY+zy%0Ao4WdcDPddlj/GudfR39Hn7pcznBwWJe/1/qtL0u8terj0UiSRbq+e7we74W/27/7Il19o11n%0AX8P3rf4Ge7ceu11Eu8sfvN2uf9Du8JS5/b07/Dh4vnv19X4a2Ya5vYcfnrC399Vm/PL9hZ10%0AM3vee//h8/ml/d3S5WXsAWl//cODzefYb5Er0GXrixKfh22JF/baDSebSE/He/3fbpPofJWB%0Ab4xL8Pnx9nwW6XjHfrn5wZ+rTW3X5s/Ir+2TRhcvHb2M7ecduId3W6uvh6+MXR7KvaW4vZlu%0AA6t//a/jjoH9t8b21e+v32043H2a+AO6Wpp+ic+DYJ/PO0waTLJI13vCftbweQ1c7efprZqR%0Au8/77mfD43Dlp9NNXf5g72pjGfzmyE8cvvy1u3i8Gt9hPrTN9X54BnbcMz1y+Sb3l+L2Zl4O%0Ad/aL638eBXob370/VPc2n30Tz637P/z7tDdvwavsNSdZpN/9lfc3i0jv+42r1//+3oj0fPmD%0Al1eb8Bzp7hdPX/7Tf+X05NEA/uV057p+EPjsPeCNXR5d2Ak3M7Skx8u78bXYd2FsWN8Dozm0%0A7peYZuR2kzyW3u7vv7vnLJt2T83Pnu9uxR9v4WrT7vJqc0Qa3rw5Xvft+MaGdktn9+f0rRv8%0A4H7pNu+9pyFjl0cXdsLNjIs0vquht2k3U6RD634JRLqf9LHsTvtM/x62il6Pq3xgZ8PNN0bW%0ARvfVj97OhsOd4Gt3+YP9q40uVv+bxx2Mf0aecJ+u+3Qs+nd373nA8UXK3tbUEX/Y9hm7fH9h%0AJ91M/8eO6I+u8fvIi7Hd9d873PCzqEvQ9dL0Szyfhjj6dGzbSRfpT/cWoa/9JnX7C3G/kfLa%0A7eU+7Gjt3gX29/Yb3a2OPCK9/+zl6oxp9yZ9HFbtxSPS+/jOsNNi9b/52rx8HfY6Db+/4XTd%0Az0PDvbf3X3M8vm3mfKXX5nm/fO+He9jY5fsLO+lm+mXfWnS7v/7wiDG+z+7wKNRydiPv+LsE%0AXS9Nv8TH4cXj0V2fm8+CB+rTC63HDYv+665PTXePP6z2yxdku1s932zv4vsJ+Ock0svQD/av%0ANrpYFxsix7ehvt9+/+Ly78O23+/R7Z8uxzdv/vfzo1/dXoPDw9jY5fsLO+lmLgjHJepe1Xm6%0A3sN4cf3zi9pD6YNuluaixPHVuNHd6BvPki3er3bvbfP79Cuq906gz6f2t1j35/VbhHp/XV1s%0A9xftfv/5OL2/Zv//fu3d/mDvaqOLdblF334y4OJtMoMi7Tdr3oefi1yk99GC7jpf7buejnew%0Asct3F3bazVxcbJfo5c/39bdvvtB0bxEa/53TA90uzcXHKD6GPtdBjol/6vh4j9O6t1o0K91s%0A/FrdXMJH/t/4K4UFRbpzqyWz0s0GLd2mEy7S6709ThG3WjIr3WzQ0m064SLdCS9ZEJtwXyUk%0AQxCJkAxBJEIyBJEIyRBEIiRDEImQDEEkQjIEkQjJEEQiJENSRfrX5fR3gRREu7KprcZGJEs2%0AtdXYiGTJprYaG5Es2dRWYyOSJZvaamxEsmRTW42NSJZsaquxEcmSTW01NiJZsqmtxkYkSza1%0A1diIZMmmthobkSzZ1FZjI5Ilm9pqbESyZFNbjY1Ilmxqq7ERyZJNbTU2Ilmyqa3GRiRLNrXV%0A2Ihkyaa2GhuRLNnUVmMjkiWb2mpsRLJkU1uNjUiWbGqrsRHJkk1tNTYiWbKprcZGJEs2tdXY%0AiGTJprYaG5Es2dRWYyOSJZvaamxEsmRTW42NSJZsaquxEcmSTW01NiJZsqmtxp4n0q77cx9E%0ACmVTW409S6TOn/MfiBTFpvaK7F+/mubxzc8QafeNSCJsaq/H/vVrikmzHpGuHUKkKDa1V2N3%0AHj00KUmk03Ok/7WZ8GOEuObk0fR9cdNF2n2zaRfMpvZK7B+PijwiIVI0m9rrsFuP/k3xCJE8%0A2dRehX3w6N+/CR4hkieb2iuwfx09KvWCLCLFs6ldnn32iHc2VMumdnH2WSPea1cvm9ql2X2P%0AEKlaNrULsy88QqRq2dQuy770CJGqZVO7KPvKI0Sqlk3tkuxrjxCpWja1C7JvPEKkatnULse+%0A9QiRqmVTuxh7wCNEqpZN7VLsIY8QqVo2tQuxBz1CpGrZ1C7DHvYIkaplU7sIe8QjRKqWTe0S%0A7DGPEKlaNrULsEc9QqRq2dTOzv417hEiVcumdm72PY8QqVo2tTOz72mESPWyqZ2Xfd8jRKqW%0ATe2s7AceIVK1bGrnZD/yCJGqZVM7I/uhR4hULZva+diPPUKkatnUzsae4BEiVcumdi72FI8Q%0AqVo2tTOxJ3mESNWyqZ2HPc0jRKqWTe0s7IkeIVK1bGrnYE/1CJGqZVM7A3uyR4hULZvay9nT%0APUKkatnUXsye4REiVcum9kL23Y8fpbARyZJN7YXsWR4hUrVsai9jz9IIkeplU3sRe97jESLV%0Ay6b2EvbBo6ZpcrIRyZJN7QXsk0fTTUKkWtnUTmefPZpsEiLVyqZ2Mrt7foRIsAujK6993M+A%0ASLALo+uu/bO/judIsMuiq67d2+/NXjvY1E5kz339aAYbkSzZ1E5hp3qESNWyqZ3ATvYIkapl%0AU3s+O90jRKqWTe3Z7AUeIVK1bGrPZS/xCJGqZVN7JnuRR4hULZva89jLPEKkatnUnsVe6BEi%0AVcum9hz2Uo8QqVo2tWewF3uESNWyqT2dvdwjRKqWTe3J7AweIVK1bGpPZefwCJGqZVN7IjuL%0AR4hULZvak9jzjqc6j319FURyZFN7CjuXR4hULZvaE9i5NEKketnUfszO5xEiVcum9kN2Ro8Q%0AqVo2tR+xc3qESNWyqf2AndUjRKqWTe377LweIVK1bGrfZWf2CJGqZVP7Hju3R4hULZvad9jZ%0APUKkatnUHmfn9wiRqmVTe5RdwCNEqpZN7TF2CY8QqVo2tUfYRTxCpGrZ1B5ml/EIkaplU3uQ%0AXcgjRKqWTe0BdraPHw2wH18FkRzZ1L5ll/MIkaplU/sm5TRCpHrZ1L5OSY8QqVo2ta9S1CNE%0AqpZN7cuU9QiRqmVvuXbTNNdfaj0KHgkiWbI3XLtpbkw6PB4hUo7F2Bp7u7Wb5sakbrvOXiRC%0AVsxRpN5XDtt1QuERyYi93do3j0in/Qz2j0jTb6vgYmyNveHaIx5FjwSRLNlbrj3sUfRIEMmS%0ATe0uvdePECnHYmyNTe1D+q/DIlKOxdgam9ptLt7PgEg5FmNrbGr/u35fECLlWIytsal98zE+%0ARMqxGFtjU/vmY3yIlGMxtsbefO3bt3sjUo7F2Bp767UHPjaBSDkWY2vsjdce+vgRIuVYjK2x%0At1178GN8iJRjMbbG3nTt4Y/DIlKOxdgae8u1Rz5Wjkg5FmNr7A3XHjs8AyLlWIytsbdbe/Qw%0AJ4iUYzG2xt5s7fHDBSFSjsXYGnurte8cdguRcizG1tgbrX3v8HWIlGMxtsbeZu27h4FEpByL%0AsTX2JmvfP5wqIuVYjK2xt1j7wWGJESnHYmyNvcHajw7vjUg5FmNr7M3VfnwWMUTKsRhbY2+t%0A9oSz8SFSjsXYGntjtaectQWRcizG1tjbqj3p7EeIlGMxtsbeVO1pZxFDpByLsTX2lmpPPBsf%0AIuVYjK2xN1R76lktESnHYmyNvZ3ak88Oi0g5FmNr7M3Unn6WZUTKsRhbY2+l9oyzlSNSjsXY%0AGnsjtWd4FD0SRLJkb6P2HI+iR4JIluxN1J7lUfRIEMmSvYXa8zyKHgkiWbI3UHumR9EjQSRL%0Adv2153oUPRJEsmRXX3u2R9EjQSRLdr21m6b5l+JR9EgQyZJdbe2mTYpH0SNBJEt2rbUPHjUp%0AHkWPBJEs2bXWTvcoeiSIZMmutXa6R9EjQSRLdrW1kz2KHgkiWbLrrZ3qUfRIEMmSXW3tZI+i%0AR4JIluxaa6d7FD0SRLJk11l7wmEgk9mLgki1squsvcij6JEgkiW7xtqLNAofCSJZsiusvdCj%0A6JEgkiW7vtpLPYoeCSJZsqurvdij6JEgkiW7ttrLPYoeCSJZsiurncGj6JEgkiW7rto5PIoe%0ACSJZsquqncWj6JEgkiW7ptp5PIoeCSJZsiuqncmj6JEgkiW7ntq5PIoeCSJZsqupnc2j6JEg%0AkiW7ltr5PIoeCSJZsiupndGj6JEgkiW7jto5PYoeCSJZsquondWj6JEgkiW7htp5PYoeCSJZ%0Asiuondmj6JEgkiXbv3Zuj6JHgkiWbPva2T2KHgkiWbLda+f3KHokiGTJNq9dwKPokSCSJdu7%0AdgmPokeCSJZs69pFPIoeCSJZsp1rl/EoeiSIZMk2rl3Io+iRIJIl27d2KY+iR4JIlmzb2sU8%0Aih4JIlmyXWuX8yh6JIhkyTatXdCj6JEgkiXbs3ZJj6JHgkiWbMvaRT2KHgkiWbINay87jdjj%0AWIm06/7cB5FC2X61Dx751Z7MniVS58/5D0SKYtvV7h6Ovv81TVMC/y98JHNE2n0jkgjbrfZx%0As+67aYqZZCRSXx9ECmWb1T49PWqaciY5i/S/NhN+jGw7h6dHbY4iBdcpnFkisbMhlm1V+7y7%0AjkckRNJiO9Xu7fbmOdKVPz2PECmCbVS7//IRe+0uBep7hEgRbJ/aFy/D+tSezU4R6cIjRIpg%0A29S+fDuDTe357ASRdruLtzaUX46Kxy+Izsq+eluQS+0E9jyRhlJ+OSoevyA6J/v67XUmtVPY%0AiGTJ9qh98zZVj9pJbESyZFvUvn27t0XtNDYiWbIdag98bMKhdiIbkSzZBrWHPn5kUDuVjUiW%0AbPnawx/jk6+dzkYkS7Z67ZGPw6rXXsBGJEu2eO2xT5WL117CRiRLtnbt0aMzaNdexEYkS7Z0%0A7fGjnEjXXsZGJEu2cu07RwtSrr2QjUiWbOHa9466JVx7KRuRLNm6te8evU639mI2IlmyZWvf%0APwqkbO3lbESyZKvWfnA0VdXaGdiIZMkWrf3oqMSitXOwEcmSrVn74dG9NWtnYSOSJVuy9uOj%0A5EvWzsNGJEu2Yu0JZ5tQrJ2JjUiWbMHaU87aIlg7FxuRLNl6tSed/UivdjY2Ilmy5WpPO4uY%0AXO18bESyZKvVnng2PrXaGdmIZMkWqz31rJZitXOyEcmSrVV78tlhtWpnZSOSJVuq9vSzLEvV%0AzstGJEu2Uu0ZZytXqp2ZjUiWbKHaMzxSqp2bjUiWbJ3aczwSqp2djUiWbJnaszzSqZ2fjUiW%0AbJXa8zySqV2AjUiWbI3aI4eBzIGeH0TKsRhbY0vUnu2RRu0ybESyZCvUnq2RRu1CbESyZAvU%0ATvBIoXYpNiJZsqNqN01zvJTikeu0EaladlDtpjmZlOSR67QRqVp2TO2mOZmU5pHrtBGpWnaw%0ASIkeuU4bkaplx4qU6pHrtBGpWnboc6Rkj1ynjUjVsiP32qV75DptRKqWHVh7gUeu00akatlx%0AtZd45DptRKqWHVZ7kUeu00akatlRtZd55DptRKqWHVR7oUeu00akatkxtZd65DptRKqWHVF7%0A/sePJqNzBJFyLMbW2AG1M3jkOm1Eqpa9fu0MGtlOG5GqZa9eO4tHrtNGpGrZa9fO45HrtBGp%0AWvbKtTN55DptRKqWvW7tXB65ThuRqmWvWjubR67TRqRq2WvWzueR67QRqVr2irUzeuQ6bUSq%0Alr1e7ZweuU4bkaplr1Y7q0eu00akatlr1c7rkeu0Eala9kq1M3vkOm1Eqpa9Tu3cHrlOG5Gq%0AZa9SO7tHrtNGpGrZa9TO75HrtBGpWnb52jk+fjSCLhNEyrEYW2MXr13EI9dpI1K17NK1i2hk%0AO21EqpZduHYhj1ynjUjVssvWLuWR67QRqVp20drFPHKdNiJVyy5Zu5xHrtNGpGrZBdEFPXKd%0ANiJVyy6HLumR67QRqVp2MXRRj1ynjUjVskuhW48MawuwEcmSXQh9eDzyq63ARiRLdhl0t11n%0AV1uCjUiW7CLo4/Mjt9oabESyZJdAn/YzmNUWYSOSJTsvumma3v46m9pSbESyZGdFN23O+71d%0AamuxEcmSnRN90uj0+pFJbTH2cpGIeU4iRfeoJTwiGbEzPyLtNWqKsK9iOm027apl532OdPm+%0AIJfaWmxEsmRnRV+9v86lthYbkSzZOdHX71M1qS3GRiRLdkb0zfu9PWqrsRHJkp0Pffu5CYva%0AcmxEsmRnQw98/sihth4bkSzZudBDn+MzqC3IRiRLdib04Odh9WsrshHJkp0HPfy5cvnakmxE%0AsmRnQY8cn0G9tiYbkSzZOdBjxzkRry3KRiRLdgb06PGCtGurshHJkr0cPX7cLenasmxEsmQv%0ARt85fp1ybV02Ilmyl6LvHQdSuLYwG5Es2QvRd4+nqltbmY1Iluxl6PvHJZatLc1GJEv2IvSD%0A43ur1tZmI5Ilewn60XHyRWuLsxHJkr0A/fB8E5q11dmIZMlORz8+b4tkbXk2Ilmyk9ETzn+k%0AWFufjUiW7FT0lPOICdY2YCOSJTsN/WuKR3q1LdiIZMlOQk/zSK62BxuRLNkp6KmnhxWrbcJG%0AJEt2AnryaZa1aruwEcmSPR89/XTlUrVt2IhkyZ6Nnu6RVG0fNiJZsueiZ3ikVNuIjUiW7Jno%0AOR4J1XZiI5Ilex56lkc6ta3YiGTJnoWe55FMbS82Ilmy56BneqRS24yNSJbsGei5HonUdmMj%0AkiV7Onq2Rxq17diIZMmejJ7vkURtPzYiWbKnohM8UqhtyEYkS/ZEdIpHArUd2YhkyZ6GTvIo%0AvrYlG5Es2VPQEz9+lMROjOm0Eala9gR0qkesySQ2IlmyH6NbjZqmKcJOjum0Eala9kP00SNE%0AWouNSJbsR+iTR4i0FhuRLNkP0Es8Yk0msRHJkn0f3e1mSPSINZnERiRL9l106u66KexlMZ02%0AIlXLvode6BFrMomNSJbsO+ilHrEmk9iIZMkeRy/2iDWZxEYkS/YoerlHrMkkNiJZssfQGTxi%0ATSaxEcmSPYLO4RFrMomNSJbsYXQWj1iTSWxEsmQPovN4xJpMYiOSJXsInckj1mQSG5Es2bfo%0A5I8fTWBni+m0Eala9g06n0esySQ2Ilmyr9H5NGJNprFHRXr+7P7+ekEkPfYVOqdHrMkk9qhI%0ATfPa/vXWPHqkKr8cFY8/DzqrR6zJJPaoSJ/Pze6//3bN0yci6bEv0Hk9Yk0msUdF+v7eW9Q8%0AfTzQCJFC2H10Zo9Yk0nsOyJ9IJIsu4fO7RFrMok9KtLnE5t2uuwzOrtHrMkk9qhITfPW/sXO%0ABkn2Dzq/R6zJJPaoSC9f3d/s/lZkn9AFPGJNJrFHRZqc8stR8fgXokt4xJpMYiOSJbtDF/GI%0ANZnERiRL9gFdxiPWZBIbkSzZLbqQR6zJJDYiWbK/y3nEmkxiI5Il+7ucR6zJJDYiWbIzfvzo%0AJqzJFDYiObJLesSaTGIjkiG7pEasyTQ2Ivmxy3rEmkxiI5Idu/XIsHZhdDQbkdzYh8cjv9ql%0A0dFsRDJjd9t1drWLo6PZiOTFPj4/cqtdHh3NRiQr9mk/g1ntFdDRbERyYv/sr/OqvQY6mp0i%0A0m4fRApgn/d7W9VeBR3NThBp9/MHIq3KPnjUNE0BdD+syRQ2ItmwTx61JhnVXgkdzUYkF/bZ%0Ao71JPrXXQkezF4n0vzZTf4wsyuH9DN9HkaLLkDthZ4My+7ifgUckVXaCSGzarc/+2V/HcyRR%0ANiI5sHvv92avnSYbkQzYt5+bsKi9KjqajUj67IHPHznUXhcdzU4QiZ0N67KHPsdnUHtldDQ7%0ARaTLlF+Oisc/IYOfh9WvvTY6mo1I4uzhz5XL114dHc1GJG32yPEZ1Guvj45mI5I0e+w4J+K1%0AA9DRbERSZo8eL0i7dgQ6mo1Iuuw7h4FUrh2DjmYjkiz73uFUhWsHoaPZiKTKvnsYSN3aUeho%0ANiKJsu8fTlW2dhg6mo1ImuwHhyVWrR2HjmYjkiT70eG9RWsHoqPZiKTIfniYfM3akehoNiIJ%0Ash+fbkKydig6mo1IeuwJp21RrB2LjmYjkhx7yumPBGsHo6PZiKTGnnQaMb3a0ehoNiKJsaed%0Ajk+udjg6mo1IWuyJp7VUqx2PjmYjkhR76ulhxWoLoKPZiKTEnnyaZa3aCuhoNiIJsaefrlyq%0AtgQ6mo1IOuzpHknV1kBHsxFJhj3DI6XaIuhoNiKpsOd4JFRbBR3NRiQR9iyPdGrLoKPZiKTB%0AnueRTG0ddDQbkSTYMz1SqS2EjmYjkgJ7rkcitZXQ0WxEEmDP9kijthQ6mo1I8ez5HknU1kJH%0AsxEpnJ3gkUJtMXQ0G5Gi2SkeCdRWQ0ezESmYneRRfG05dDQbkWLZaR6F19ZDR7MRKZSd6FF0%0AbUF0NBuR4thNk+qR60hMayOSNHuBR64jMa2NSMrszqOmBHpZWJMpbESKYnceIVIlbEQKYv/q%0APEKkStiIFMNe5pHrSExrI5Is+7CbId0j15GY1kYkVXb67rqH6OVhTaawESmAvdgj15GY1kYk%0ATfZyj1xHYlobkSTZGTxyHYlpbURSZOfwyHUkprURSZCdxSPXkZjWRiQ9dh6PXEdiWhuR5NiZ%0APHIdiWltRFJj5/LIdSSmtRFJjJ3NI9eRmNZGJC12Po9cR2JaG5Gk2Bk9ch2JaW1EUmLn9Mh1%0AJKa1EUmIndUj15GY1kYkGfavvB65jsS0NiKpsHN75DoS09qIJMLOrZHtSExrI5IGO79HriMx%0ArY1IEuwCHrmOxLQ2IimwS3jkOhLT2ogkwC7iketITGsjUjy7jEeuIzGtjUjh7EIeuY7EtDYi%0ARbNLeeQ6EtPaiBTMLuaR60hMayNSLLucR64jMa2NSKHsgh65jsS0NiJFskt6ZDoS19qIFMgu%0A6pHnSGxrI1Icu/WIkayIjmYjUpEcHo8YyYroaDYilUi3XcdIVkRHsxGpQI7PjxjJiuhoNiLl%0Az2k/AyNZER3NRqTs+dlfx0hWREezESl3zvu9GcmK6Gg2ImVO7/UjRrIiOpqNSFnT9F+HZSQr%0AoqPZiJQzFx4xkjXR0WxEypiDR01ThH0Zm5Gsho5mLxeJnPKr84hBbjs8Ii3MySMekSLQ0WxE%0AypWTRj8eMZI10dFsRMqUbjdD36PNj2RVdDQbkfJk6ONHGx/JuuhoNiJlyeDH+LY9kpXR0WxE%0AypHhj8NueiRro6PZiJQhIx8r3/JIVkdHsxFpecYOz7DhkayPjmYj0uKMHuZkuyMJQEezEWlp%0Axg8XtNmRRKCj2Yi0MHcOu7XVkYSgo9mItCz3Dl+30ZHEoKPZiLQodw8Duc2RBKGj2Yi0JPcP%0Ap7rJkUSho9mItCAPDku8xZGEoaPZiJSeR4f33uBI4tDRbERKza+Hh8nf3Egi0dFsRErMY482%0AN5JQdDQbkdIy5awtGxtJLDqajUhJmXT2o22NJBgdzUaklEw7i9imRhKNjmYjUkImno1vSyMJ%0AR0ezEWl+pp7VckMjiUdHsxFpdiafHXY7IxFAR7MRaW6mn2V5MyNRQEezEWlmZpytfCsjkUBH%0AsxFpXmZ4tJWRaKCj2Yg0K3M82shIRNDRbESak1kebWMkKuhoNiLNyDyPNjESGXQ0G5GmZ6ZH%0AWxiJDjqajUiTM9ejDYxECB3NRqSpme1R/SNRQkezEWlaJnz8KJmdEIWRaKGj2Yg0KSkeVT4S%0AMXQ0G5GmJEWjykeiho5mI9KEpHlU9Ujk0NFsRHqcRI9qHokeOpqNSA+T6lHFIxFER7MR6VGS%0APap3JIroaDYiPUi6R9WORBIdzUake2maBR7VORJVdDQbke5kmUdVjkQWHc1GpPF0HjVF2Atj%0Aeo80rY1Iy9idR4hkgY5mI9J4Oo8QyQIdzUak0Sz1qMKRCKOj2Yg0lsUe1TcSZXQ0G5FGsmh/%0A3QP28pjeI01rI1I6O4NHtY1EGx3NRqTB5PCospGIo6PZiDSULB7VNRJ1dDQbkQaSx6OqRiKP%0AjmYj0m0yebz5iCAAAAt+SURBVFTTSPTR0WxEukkujyoaiQE6mo1I18nmUT0jcUBHsxHpKvk8%0AqmYkFuhoNiJdJqNHtYzEAx3NRqSL5PSokpGYoKPZiNRPVo/qGIkLOpqNSL3k9aiKkdigo9mI%0AdE5mj2oYiQ86mo1IP8ntUQUjMUJHsxHplOwe+Y/ECR3NRqRj8ntkPxIrdDQbkboU8Mh9JF7o%0AaDYiHVLCI/ORmKGj2YjUpohH3iNxQ0ezEelfKY+sR2KHjmYjUjGPnEfih45mI1Ixj4xHYoiO%0AZiNSMY98R+KIjmZvXqRyHtmOxBIdzd66SAU9ch0JtVPYGxeppEemI6F2EnvbIhX1yHMk1E5j%0Ab1mkX2U9chxJYbZpbUS6m4NHpquW2mrs7YrUPRyZrlpqq7E3K9Jxs8501VJbjb1VkU5Pj0xX%0ALbXV2BsV6Wc3g+mqpbYae5sinXfXma5aaquxNylSb7e36aqlthp7iyL1Xz4yXbXUVmMnirQz%0AFuniZVjTVUttNXaaSDtPkZqmuX5bkOmqpbYaO0mknecjUtPm6m1BpquW2mrsFJF2npt2Qx5F%0Aj18QTe0k9iKR/tdm8o9F5+RRdA9SdyaLtPu2fUS6fbu36e9Iaqux54u0+/nDTKR/Ax5Fj18Q%0ATe0kdoJIXQxFGvr4kemqpbYae75I54clL5GGP8ZnumqprcbejEgjH4c1XbXUVmNvRaSxT5Wb%0Arlpqq7ETReql/HJkQI8encF01VJbjb0NkcaPcmK6aqmtxt6ESHeOFmS6aqmtxt6CSPeOumW6%0Aaqmtxt6ASHePXme6aqmtxq5fpPtHgTRdtdRWY1cv0oOjqZquWmqrsWsX6dFRiU1XLbXV2JWL%0A9PDo3qarltpq7LpFenyUfNNVS201dtUiTTjbhOmqpbYau2aRppy1xXTVUluNXbFIk85+ZLpq%0Aqa3GrlekaWcRM1211FZj1yrS1LPxma5aaquxKxVp8lktTVcttdXYdYo0/eSwpquW2mrsKkWa%0AcZJl01VLbTV2jSLNOVm56aqlthq7QpHmeBQ9fkE0tZPY9Yk0y6Po8QuiqZ3Erk6keR5Fj18Q%0ATe0kdm0izfQoevyCaGonsSsTaa5H0eMXRFM7iV2XSLM9ih6/IJraSeyqRJrvUfT4BdHUTmLX%0AJFKCR9HjF0RTO4ldkUgpHkWPXxBN7SR2PSIleRQ9fkE0tZPY1YiU5lH0+AXR1E5i1yJSokfR%0A4xdEUzuJXYdIkz9+lMBOD/fIFdHR7CpESvcoevyCaGonsWsQKV2j8PELoqmdxK5ApCUeRY9f%0AEE3tJLa3SE3TLPMoevyCaGonsa1F2nvULPMoevyCaGonsZ1FyuBR9PgF0dROYpuLtNSj6PEL%0AoqmdxPYWqfWoKcLOEe6RK6Kj2c4i/VvuUfT4BdHUTmI7i5TBo+jxC6KpncQ2Fmnx86M77Czh%0AHrkiOprtK1IWj6LHL4imdhLbVqQ8HkWPXxBN7SS2q0iZPIoevyCa2klsU5FyeRQ9fkE0tZPY%0AniJl8yh6/IJoaiexLUXK51H0+AXR1E5iO4qU0aPo8QuiqZ3ENhQpp0fR4xdEUzuJ7SdSVo+i%0Axy+IpnYS206kvB5Fj18QTe0ktptImT2KHr8gmtpJbDORcnsUPX5BNLWT2F4iZfcoevyCaGon%0Asa1Eyu9R9PgF0dROYhuJtOAwkA/ZRcI9ckV0NNtHpCIeRY9fEE3tJLaNSEU0Ch+/IJraSWwX%0AkQp5FD1+QTS1k9gmIpXyKHr8gmhqJ7E9RCrmUfT4BdHUTmJbiFTOo+jxC6KpncR2EKmgR9Hj%0AF0RTO4ltIFJJj6LHL4imdhJbX6SiHkWPXxBN7SS2vEhlPYoevyCa2klsdZFajyoevyCa2kls%0AcZEOj0cVj18QTe0ktrZI3XZdxeMXRFM7iS0t0vH5UcXjF0RTO4mtLNJpP0PF4xdEUzuJLSzS%0Az/66iscviKZ2EltXpPN+74rHL4imdhJbVqTe60cVj18QTe0ktqpI/ddhKx6/IJraSWxRkS7e%0Az1Dx+AXR1E5ia4p0+b6giscviKZ2EltSpKv311U8fkE0tZPYciI1TXP9PtWKxy+IpnYSW02k%0AAY+iR6TIprYaW0ykIY+iR6TIprYae7lIWdN5JFGFkKRoPCIdPGpm/z5Ijimb2mpsMZGGPIoe%0AkSKb2mpsLZEGPYoekSKb2mpsKZFGjs9Q8fgF0dROYiuJNHack4rHL4imdhJbSKTR4wVVPH5B%0ANLWT2DoijR93q+LxC6KpncRWEeneacQqHr8gmtpJbBGR7p6Or+LxC6KpncTWEOn+4VQrHr8g%0AmtpJbAmRHhyWuOLxC6KpncRWEOnR4b0rHr8gmtpJbAGRHh4mv+LxC6KpncSOF+nx6SYqHr8g%0AmtpJ7HCRJpy2peLxC6KpncSOFmnK6Y8qHr8gmtpJ7GCRJp1GrOLxC6KpncSOFWna6fgqHr8g%0AmtpJ7FCRJp7WsuLxC6KpncSOFGnq6WErHr8gmtpJ7ECRJp9mueLxC6KpncSOE2n66corHr8g%0AmtpJ7DCRpnsUPSJFNrXV2FEizfAoekSKbGqrsWNEuvvxo5TFSI4pm9pq7BCR5nkUPSJFNrXV%0A2BEizdMofESKbGqrsQNEmutR9IgU2dRWY68v0myPokekyKa2Gnt1keZ7FD0iRTa11dhri5Tg%0AUfSIFNnUVmOvLFKKR9EjUmRTW429rkhJHkWPSJFNbTX2qiKleRQ9IkU2tdXYa4qU6FH0iBTZ%0A1FZjryhSqkfRI1JkU1uNvZ5IyR5Fj0iRTW019moipXsUPSJFNrXV2GuJtMCj6BEpsqmtxl5J%0ApCUeRY9IkU1tNfY6Ii3yKHpEimxqq7HXEKlZ5lH0iBTZ1FZjryDSUo+iR6TIprYau7xITbM3%0AqWnKLsbW2NRWY68iUpuyi7E1NrXV2Ihkyaa2Gru8SP+WehQ9IkU2tdXYK4j0b6FH0SNSZFNb%0Ajb2GSCssxtbY1FZjI5Ilm9pqbESyZFNbjY1Ilmxqq7ERyZJNbTU2Ilmyqa3GRiRLNrXV2Ihk%0Ayaa2GhuRLNnUVmMjkiWb2mpsRLJkU1uNjUiWbGqrsRHJkk1tNTYiWbKprcZGJEs2tdXYiGTJ%0AprYaG5Es2dRWYyOSJZvaamxEsmRTW42NSJZsaquxEcmSTW01NiJZsqmtxkYkSza11diIZMmm%0AthobkSzZ1FZjI5Ilm9pqbESyZFNbjY1Ilmxqq7ERyZJNbTU2Ilmyqa3GXi7SMf9b9uNRofaa%0AMa09rzciGYXaqwaRHobaa8a0NiI9DrXXjGntVUUihLRBJEIyBJEIyRBEIiRDEImQDEEkQjJk%0AkUi7fXIVWTWWrU2nbVq7zZzeS0Tazb0xlViuWdNpm9ZuM+tuskWRdo6lXadtWvt77t1kiyKZ%0Alm5jWtyx9o5Nu8exLN3GtLhjbUSaEMvS36a9PXc27L4R6XEsS3/Te73Mvm8jklFMazsW33WZ%0A/gOI5BPL1r53Eh6RHofSq8X3TrKeSL4vWju2nrutoRLP1oesJhIhpAsiEZIhiERIhiASIRmC%0ASIRkCCIRkiGIREiGIBIhGYJIhGQIInmnafqX3l3fQ+AfRPLOpUgNqzMqTN47l+ogUliYvEP2%0Agrw0z38Plz53z9/ff383ze+/V9/Z/4dJUWHwDtlbs3dk99Veem5+f3/tmp9/v/xcQqTAMHiH%0A7PX5+n5uXttL+z++X5v9o9Lx3+fvsGkXGCbvkP0G3X5zrnlqL7WbcU/tn+d/Hy8hUmCYvEM6%0AQc6qDP0bkULD5B2CSPJh8g45bsA9n1Rh004uTN4hzd6hr+fm7aTKxc6G3ncQKSxM3iHtTu+m%0A2X2fVBnZ/f3dXYcEBJEcst+Ae/55AbbN5QuyL6fvvCNSVBDJIWyyyYc15BBEkg9ryCGIJB/W%0AkEMQST6sIUIyBJEIyRBEIiRDEImQDEEkQjIEkQjJEEQiJEMQiZAM+T9g0jjrxSaLPQAAAABJ%0ARU5ErkJggg==)

방법2<a href="#방법2" class="anchor-link">¶</a>
-----------------------------------------------

\$a\$값 변화 -\> \$x\$축 \$ln\[-ln(1-p\_r)\]\$, \$y\$축
\$ln(X\_{(r)}-a)\$인 Q-Q Plot

In \[58\]:

    lm2_steel<-function(aaa) {
      steel_new<-steel %>% filter(x>aaa) %>% mutate(pr=obs/(n1+1),probit=log(-log(1-pr)))
      lm1<-lm(log(x-aaa)~probit,data=steel_new)
      intercept1<-as.numeric(format(summary(lm1)$coefficients[1],digits=4))
      slope1<-as.numeric(format(summary(lm1)$coefficients[2],digits=4))
      rsqr1<-summary(lm1)$r.squared
      return(list(rsqr1,intercept1,slope1))
    }

    lm2_cotton<-function(aaa) {
      cotton_new<-cotton %>% filter(x>aaa) %>% mutate(pr=obs/(n2+1),probit=log(-log(1-pr)))
      lm3<-lm(log(x-aaa)~probit,data=cotton_new)
      intercept3<-as.numeric(format(summary(lm3)$coefficients[1],digits=4))
      slope3<-as.numeric(format(summary(lm3)$coefficients[2],digits=4))
      rsqr3<-summary(lm3)$r.squared
      return(list(rsqr3,intercept3,slope3))
    }

In \[59\]:

    initial1<-0.1
    a2<-seq(initial1,32,by=initial1)
    Rsquared11<-rep(NA,length(a2))

    initial2<-0.01
    a3<-seq(initial2,1,by=initial2)
    Rsquared33<-rep(NA,length(a3))

In \[60\]:

    for (i in 1:length(a2)) {
      Rsquared11[i]<-lm2_steel(a2[i])[[1]]
    }

    for (j in 1:length(a3)) {
      Rsquared33[j]<-lm2_cotton(a3[j])[[1]]
    }

In \[61\]:

    a_steel1<-initial1*which.max(Rsquared11)
    mu_steel1<-lm2_steel(a_steel1)[[2]]
    sigma_steel1<-lm2_steel(a_steel1)[[3]]

In \[62\]:

    a_cotton3<-initial2*which.max(Rsquared33)
    mu_cotton3<-lm2_cotton(a_cotton3)[[2]]
    sigma_cotton3<-lm2_cotton(a_cotton3)[[3]]

In \[63\]:

    steel_new1<-steel %>% filter(x>a_steel1) %>% mutate(pr=obs/(n1+1),probit=log(-log(1-pr)))
    cotton_new3<-cotton %>% filter(x>a_cotton3) %>% mutate(pr=obs/(n2+1),probit=log(-log(1-pr)))

In \[65\]:

    ggplot(steel_new1,aes(probit,log(x-a_steel1)))+
      geom_point()+
      geom_smooth(method="lm",se=F)+
      ggtitle(paste("Steel",",","a=",a_steel1,",",
                    "R2=",format(Rsquared11[which.max(Rsquared11)],digits=3),",",
                    "mu=",mu_steel1,",","sigma=",sigma_steel1))

    `geom_smooth()` using formula 'y ~ x'

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzMzZv9NTU1o%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///9uyY2wAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2di3qiyhJG2YMmxkni6Ps/7BbwgkJDN91Q1dWr%0AvnMmZGZTLP52KSCX6kJRVHRV0gAUZaEQiaISFCJRVIJCJIpKUIhEUQkKkSgqQSESRSUoRKKo%0ABIVIFJWgEImiElSsSN+fdVXtDudm+vw5uaRli/raVXXX/nL52VfV/vvtP/hxNq5utbvPf7zO%0Avzs6FzVsf9xV1edpejpJffXXoXpU89vvx3NZ/TAmamHULzUMo7fwc4+vP+0LaLAiI/+4D3kT%0A+fT4LRvdfdu9bl9IP92ivl7/i92sSNf5z89e1d7xnw/bH7q/+ZmaTlLf1ahI9fWX327y9FyB%0AetbgBCINw+gv/LsnT3/aF9BgxUV+rOrGoNOxqn5XEelY7U/XT7rqo/llV/29XP6+9TlWEyJ1%0AP69vrodL8/K/9rr81O8m3mvQ/ruqr66c9vXEdJIaX4fvJtPrW9Wx+Q8+Lm9hrFyDMF4Wfqie%0An1W96S0BtVWcSHV139apPlcRaVedH/P+dAYcmiF+1Lma+kS6Tfy27+111ftlWMP2++4z56f5%0AG9d0gvrdj67Db3V8rkT7Zz+MlWsYxsvCb79c3qY3BFRXcev8jOw69fiA729eP6dd8R6ur6L9%0AzFZS+9o/tO/Q1xdYf1fso5raRxpOuUGG7e//YfMO65oeXepXXe2u63ysuz2y23/v4qyun5cj%0A/1bvuh+dSD35HW8El9/P61bV589zSd/7qj62vwyZZnIfzfqx8Ou71+Mv+tMzgKYrTqRum+ne%0A6ibS326buf2X3rTjdVT397Fe9rIfddq3b4z77m9fRu7n+mL2/UR6/PVudFHD9g9havf02FK/%0Auj2oz/bH8UWksVX8HN0q/upeytef7abdY3P0Fsawfp57q123DuPQLfadaSb3sayfC/+uDo2X%0ALUd/ehrQdsWJdKqv74F/f++9uqNM7fvcdXvl+3V6/PX+1b5EvtojAA6RbsP9fP32/rXZmJjf%0AR6r7vn83SxxZ1LD9rnstt0cCXNNjS232Gw9V3f3YzYr0vk5tnR4fd191f6f/Hsaw2r2a6zvX%0AfYG/VfMX33W32HemmdzHsn4u/Haw5bbn2XvfnAS0XZGbs6fusN1H75V+6DaUz80roT89/rKb%0A36z+3X90hzRGBvfYe8Mf1vP10T9Qt3NseAzbt/vOt5eia3qsTwN77tk/s2k3+m8ftw+k5l2o%0ACfh0/+0WxmSLdvLQ7WP97QjemWZyHxXpsfDO2VP7/tifnga0XdH7hefvr/3tpXrfEX2+vfWn%0Ana+j37bD5DLa3fvh4HZbHvPfI/W/OvqoHAdnR147++fGkXN6pE//JbpQpPPd/d9W2t/985Dy%0AeT9+3P2j2Tg4Pbvt7m9i1RjTZTr38U+kt4X/9HYSe9MuQNuV5gDLz77bXmo7vsgzK9KxHtnS%0Aea/f5nU1HNyP6ucyKdL1j3P9sqlxcG54jL12ju3Wf/c3rmlHnziRvu6Yn/cjhM+9/t/xb8JO%0A9eNNo+3WX58h00zuLpFeF97/1960A9B2RYnU/2yonyKN/Qeu19Hx+mF2+Hu6Df34DsRt5o/7%0A4O6ff+v673tL/Ol/c3r3aGRRw/a3+u298bqmB0t1iOS7j1RXb/8wn2RzlknV7U55iDSTuzMM%0Ar+GdeVs0WVGrvH++vT+Hq75v3b9Nu/aRfh7/OCPS2PFpD5Gu7+717ZuO877+eZ/zMavziO+x%0Ad6jCNT1capRIz0MNISJd2kPg93e01027d6aZ3J1hvILsHNOIFFZ/H+n9fW57fXbht5/v/emp%0ADSHHAbDGxHP375+PLwm/3jfOpjftLs2Lpns5nOqpk1eG7W8Lb7euXNPOpb7Yc55ax5F1OFb3%0A3bp9t6yfJsF+GJOd2m63gw3HcZFmcneG0S68vkMdRqenAY1W3HvHvqr/XrM7HW5fXzSv09/2%0AUOhv3R3+fk67PpGO7gNg7fHZc3v8+vfyOG3l/ajbrEi/7QlM172l6S8KB+0PzcLPx3bzxjXt%0AXGrvx776OE+s48g6PI7ZXV+U+5/7QbGXMMbxr2z3A6Td4e+/1bhIM7kPw+gv/NCsUHM88fw6%0APQ1ou+JEOu3v2wTNG9iu2za4ncXYbvf0pntj1ps83hs4DvTsn+3v52/+fWvx8mIY3QT6bD85%0AP6c3BIftz93+ePsx5poeLnb4ou2+KnUc5Hud//Zz9zy0eOil2Q9jsNzbF7INVfe3X8/VHd1H%0Amsx9mHV/4bfpo2PacTKj6Yrdmv3+aC+j6Laod9072OnwOA+lN+146R+b73R/vp0nOjZXLHzc%0ARrt37lGQSNdtjuPsHtVI+3MDf9sRck0PFjt80V5+rsF8Te45vInU/0+/eydc9cMY/Ic/7SlC%0Ap+ffNnN+/IyLNJv7MOv+wpvpPpQTsJiS2S1ccalC+7lSu9ezyz0XeS729iXyAvg7OM8xh9YK%0AFzu13G677PrB4r6SkUpXIiId1hvbFVsrXOzUcu+7SAV+OSpRBR7xL6W+m/Mg9yWeiS1RiERR%0ACQqRKCpBIRJFJShEoqgEhUgUlaAQiaISFCJRVIJCJIpKUIhEUQkqRqR/i2rhbOkaAABAKgBE%0AAgCABA0QCQAAEjRAJAAASNAAkQAAIEEDRAIAgAQNEAkAABI0QCQAAEjQAJEAACBBA0QCAIAE%0ADRAJAAASNEAkAABI0ACRAAAgQYMQker6efP5uv8LIgFQOkCASPXjj95PRAIAgH+IBAAASRoE%0AiNQX6O35QrLrIDY/AADEivTYRfqvKa9ZKaqE8hLpZcuOgw0AALDsE6ke/0V2HcTmBwCAZSLV%0Ajt9k10FsfgAAWCRS/TqFSAAAsOQL2dfJ3q+y6yA2PwAALPke6X6orr68nuWASAAUDxDyieQu%0A2XUQmx8AABAJAAASAiASAAAkaIBIAACQoAEiAQBAggaIBAAACRogEgAAJGiASAAAMFNVVc02%0AQCQAAJiuqpoyCZEAAMBn/qqaNAmRAAAAkZI0AAAARErQAAAA2EdK0AAAADhql6ABAADwPVKC%0ABgAAgEgJGgAAACIlaAAAAIiUoAEAACBSggYAAIBICRoAAAAiJWgAAAAe8//549EAkQAAYLL+%0A/Jk0CZEAAMBj/hmPEAkAADzmn/MIkQAAYH7+WY8QCQAAZuef9wiRAABgbn4PjxAJAACm5//j%0A4xEiAQDA9PxeHiESAABMzu+lESIBAMDk/H6fR4gEAABT8/t6hEgAAOCe39sjRAIAAOf8/h4h%0AEgAAuOYP8AiRAADAMX+IR4gEAADj8wd5hEgAADA6f5hHiAQAAGPzB3qESAAAMDJ/qEeIBAAA%0Aw/mDPUIkAAAYzB/uESIBAMD7/As8QiQAAHibf4lHiAQAAC/z+13H52qASAAA0NRCjxAJAAB6%0A8y/UCJEAAKA3/2KPEAkAAB7zL/cIkQAA4D5/hEeIBAAAt/ljPEIkAADo5o/yCJEAAKCZf+lh%0A7zcARAKgaIBYjxAJAABiN+t6AIgEQMEA8R4hEgAAJPAIkQAoHqDxKBUAIgFQKkD7eYRICRoA%0AUDJAt12HSAkaAFAwwG3/CJESNACgXID7cQZEStAAgGIBHsfrEClBAwBKBXge90akBA0AKBSg%0A9/0RIiVoAECZAP3vYREpQQMAigR4OZ8BkRI0AKBEgNfzghApQQMACgR4O78OkRI0AKA4gMHl%0AR4iUoAEApQG0HlVrACASAOUAtBpdawUARAKgGIC7R32TEClBAwCKAnh6hEhpGwBQEkB7mAGR%0A1mgAQEEAt8N1FftI6RsAUA7A47D3q0fKRKIo1dUe9t5oWXwiAWAVYOIukLo+kaIQotdBbH4A%0A8gCYuusWIiVoAEARAJN3r0OkBA0AKAFg+i6QiJSgAQAFAMzcTRWREjQAwD7A3F2JESlBAwDM%0AA8ze3RuREjQAwDrA/F3yESlBAwCMA3g8bQKREjQAwDaAz1NbEClBAwBMA3g9/QiREjQAwDKA%0A31PEEClBAwAMA3g+jQ+REjQAwC6A71MtESlBAwDMAng/HRaREjQAwCqA/1OWESlBAwBsAkxc%0AfrQaACIBYA4gwCNEStEAAJMAARohUpIGAFgECPk8QqQkDQAwCBDmESKlaACAPYBAjxApRQMA%0AzAGEeoRIKRoAYA0g2CNEStEAAGMA4R4hUooGANgCWOARIqVoAIApgCUeIVKKBgBYAljkESKl%0AaACAIYBlHiFSigYA2AFY6BEipWgAgBmApR4hUooGAFgB6B4PKwiASAAYALg/ZlkMAJEAyB/g%0Az+Nx5UIA7Q9EAiBvgKdHiBS3DmLzA6AAoDvMsNQjRErRAID8Ae6H6xZ6hEgpGgCQPcDiw96p%0AABAJAAMA0R4hUooGAGQOEO8RIqVoAEDeAI1HWhJAJAAyBehuA6klAUQCIE+A2+1UtSSASABk%0ACXDfPdKSACIBkCPA4zCDlgQQCYAMAZ6H67QkgEgA5AfQO+ytJQFEAiA7gP7XR1oSQCQAcgN4%0A+RpWSwKIBEBmAK+nM2hJAJEAyAvg7bQgLQkgEgBZAbyfXqclAUQCICeAwWmqWhJAJAAyAhie%0A7q0lAUQCIB+AkcsmtCSASABkAzB2+ZGWBBAJgFwARi/j05IAIgGQCcD45bBaEkAkALIA+OO4%0ArFxLAogEQA4ALo/UJIBIAGQA4L7LiZYEEAkA/QATdwvSkgAiAaAeYOquW1oSQCQAtANM3r1O%0ASwKIBIBygOm7QGpJAJEA0A0wczdVLQkgEgCaAZyHvbcC8G2ASAAoBpj1SE0CiASAXgCPm+Rr%0ASQCRAFAL4POwCS0JIBIAWgG8HtqiJQFEAkApgN/Dj7QkgEgAKASoqsrzIWJaEkAkAPQB+Huk%0AJgFEAkAdQIBHahJAJADUAXQeVXIACxogEgDqADqPEGnrdRCbH4B1AAI8UpMAIgGgDSDEIzUJ%0AIBIAygC8jzOsBbCoASIBoAsgzCM1CYSIVF9rbBqRAEg2f6BHahIIEKl+/PE6jUgAJJs/1CM1%0ACSASAHoA5i8/WhlgeYMAkZ4GIRIAawAs8EhNAtEi/deU16wUNVmNRtIMCcpLpLr/k08kABIC%0ALPg4SgsQ1QCRANABsMwjNQmEiVS/TCASAMkAFnqkJoEgkerXKUQCIBXAUo/UJBAiUv02iUgA%0AJAJY7JGaBAJEquvb6Qz1hTMbAEgIsOSwd1KABA1CPpHcJbsOYvMDkE4kUYAEDRAJAGmACI30%0AJIBIAAgDxHwe6UkAkQCQBYjzSE0CiASAKECkR2oSQCQAJBvEeqQmAUQCQLBBtEdqEkAkAOQa%0AxHukJgFEAkCsQQKP1CSASABINWivP5IESDE/IgEgDNB+HplJAJEAkGnQbdeZSQCRABBpcNs/%0AMpMAIgEg0eB+nMFMAogEgECDx/E6MwkgEgDbN3ge9zaTACIBsHWD/nV8ZhJAJAA2bvByPayZ%0ABBAJgG0bvJ7OYCYBRAJg0wZvpwWZSQCRANiywfvpdWYSQCQANmwwOE3VTAKIBMB2DYane5tJ%0AAJEA2KzByGUTZhJAJAC2ajB2+ZGZBBAJgI0ajF7GZyYBRAJgmwbjl8OaSQCRANikgeOycjMJ%0AIBIA6zeoKtftGcwkgEgArN7A7ZGdBBAJgLUbdB5VcgBrzo9IAGwF0HmESIgEQFSDziNEQiQA%0AIhr8mfLITgKIBMCqDaY9spMAIgGwZoO5uxKbSQCRAFixwezdvc0kgEgArNdg/i75ZhJAJABW%0Aa+DxtAkzCSASAGs18Hlqi5kEEAmAlRp4Pf3ITAKIBMAqDf54eWQnAUQCYI0Gnh7ZSQCRAFih%0AgfdDLc0kgEgApG/g/3BYMwkgEgDJGwQ8ZNlMAogEQOoGIQ8rN5MAIgGQuEGIR3YSQCQA0jYI%0A8shOAogEQNIGYR7ZSQCRAEjZINAjOwkgEgAJG4R6ZCcBRAIgXYNgj+wkgEgAJGsQ7pGdBBAJ%0AgFQNFnhkJwFEAiBRgyUe2UkAkQBI02CRR3YSQCQAkjRY5pGdBBAJgAQNfC8/Wg1AbH5EAiAd%0AwGKPzCSASADEN1iskZkEEAmA+AYRHhlJ4B8iARDdIMYjGwm0PxAJgKgGUR6ZSKD7gUgAxDSI%0A88hCArcfiARAREV6ZCABRAIgev7lh70TAYgngEgAJJg/2qPsE0AkAOLnv2pUeAL/EAmA6Pn/%0AINI/RAIgdv52u67oBPoNEAmAZdXtH5WcwEsDRAJgUd2OMxScwGsDRAJgSd2P15WbwFsDRAJg%0AQT2OexebwHsDRAIgvJ7fH5WawKABIgEQXL3vYQtNYNgAkQAIrf75DGUmMNIgjUhUQdV+D0s5%0Ai08kAHzq9fy6EhMYbYBIAATV23mqBSYw3gCRAAip9/O9y0vA0QCRAAiowXUTxSXgaoBIAHjX%0AyHV8hSXgboBIAPjW2PWwZSUw0QCRAPCs0cthi0pgqgEiAeBXjUeVJMA6DRApQQMA/OdvPaoG%0AJhWUwHQDRALAp24eIZKrASIB4FF3jxDJ1QCRAJgvp0fFJDDbAJEAmKvbYe8xjwpJwKMBIgEw%0AU5O3Uy0iAZ8GiATAdE3fTbWEBLwaIBIAkzVzV+ICEvBrgEgATNXc3b3tJ+DZAJEAmKjZu+Sb%0AT8C3ASIB4K75p01YT8C7ASIB4CyPp7YYT8C/ASIB4Cqfpx/ZTiCgASIB4Civp4iZTiCkASIB%0AMF5+T+OznEBQA0QCYLQ8n2ppOIGwBogEwFj5Ph3WbgKBDRAJgJHyfsqy2QRCGyASAMPyf1q5%0A1QSCGyASAIPy98hqAuENEAmA9wrwyGgCCxogEgCvNXn50RYA2zZApAQNABjOH+aRxQSWNUAk%0AAPoVppHFBBY2QCQAehXqkb0EljZAJACeFeyRuQQWN0AkAB4V7pG1BJY3cIt0/vqoqurj64xI%0AhQAs8MhYAhENnCIdqnsdEKkIgCUe2UogpoFDpJ+6PnyfrhOn70NV/yCSfYBFHplKIKrBuEjf%0A9bH327H+RiTrAMs8spRAXINxkT7fRHn/HZGsASz0yFACkQ3GRQot2XUQm98OwFKP7CQQ2wCR%0AAIjwyEwC0Q0QCYAIj6wkEN9gXKSqX4hkHCDCIyMJJGgwLtIBkcoBiPHIRgIpGoyLdDVp9rsj%0ARLIBEOWRiQSSNHCIdPmsEakEgMDLj9IDRM6vBsAl0qX2ODMIkXIHiPUo/wQ4apegQfEAsRrl%0AnwAipWhQOkC8R7kngEhJGhQOkMCjzBNICDAuEt8jFQDQeFR2AikBEKlUgPbzqOgEkgKMixRa%0AsusgNn/OAN12XckJpAVApCIB7oe9y00gNcCESMeP62bd/heRDALcjzOUm0BqAKdI5127f1R5%0AnSskuw5i82cL8DhcV2wCyQGcIn1Wh6tFl7/VHpGsAfxBpOQATpGao3X3/yOSKYDe90eFJrAC%0AACIVB9D/HrbMBNYAcIp027Q7VHM3PkGkvABezmcoMoFVAJwinevu69j6hEiWAF7PCyoxgXUA%0AnCJdLl+7qtodfO5YjEjZALydX1dgAisBTIgUULLrIDZ/fgDv56mWl8BaAIhUEsDgfO/iElgN%0AYEIkzmywBjC8bqK0BNYDcIrEmQ3mAEauPyosgRUBnCJxZoM1gLHr+MpKYE0Ap0h8IWsMYPR6%0A2KISWBUAkQoBGL+uvKQE1gVwisSZDaYAHPdnKCiBlQGcInFmgyEA5+3riklgdQCnSJzZYAfA%0AfRvIUhJYH2BCpICSXQex+TMBeGp03cKQAFhxfjUAiGQe4MWjN5PKSGALAKdI96N1tc/d9GXX%0AQWz+LADePHo1qYgENgEYF6nmvnZWAHq7R4i0IsC4SMeeR0dEyhigf5gBkVYEGBfp4vlFLCIp%0AB3g9XMc+0noATpGCSnYdxObXDjA47M1Ru9UA3CId68vlp6q/EClXAI+niBlPYEMAp0jX3aTL%0AqTno0DfpeQSvbgqRNAP4PLXFdgJbAjhF2lU/1/8ff6ve4e+eOq8HxWXXQWx+1QBeTz8yncCm%0AAE6Rrh9I39Xu5aBDfUGkbAD8niJmOYFtAZwi1dXps/pt9pL6fznuESKpA/B8Gp/hBDYGcIr0%0A1Zz53Xwg9Z9u/hTpsYv0X1MXSle1T+OjROr9qN2hqr+vxvQ9evtE4mCDVgDvp8OaTWBzALdI%0AY1U7fpNdB7H5tQL4P2XZagLbA3iJ9DjggEg5AAQ8rdxoAgIAC0Vi004vQIBHRhOQAIgQqffx%0AJLsOYvOrBAjxyGYCIgBLRGot6p/YgEh6AII8MpmADECYSK6SXQex+RUChHlkMQEhAEQyBRDo%0AkcEEpAAQyRJAqEf2EhADQCRDAMEemUtADgCRzAB4XH60LoDE/GoAEMkKwBKPbCUgCuAl0mzJ%0AroPY/JoAlmhkKwFZALdIB27HlRHAMo8sJSAM4BTp4REiZQCw0CNDCUgDOEWqq999dTrvefRl%0ABgBLPbKTgDiAU6TrJ9FX9X058+hL/QCLPTKTgDzAlEjfzV1W2bRTD7DcIysJKABwivRR/T1V%0Au8sPIikHWHTYOyWA6PxqAJwiNQbtm2MNPPpSNUCURyYS0AHgFOnyvWseJPty7xNEUgcQpZGJ%0ABJQAuEUKKdl1EJtfHiDSIwMJaAFApJwBYj3KPwE1AIiUMUC0R9knoAcAkfIFiPco9wQUASBS%0AtgDt3VQlAVI0MAOASLkCtJ9HRSegCgCRMgXotutKTkAXwKRI3s+RlV0HsfkFAW77RwUnoAwA%0AkbIEuB9nKDcBbQCIlCPA43hdsQmoA0CkDAGex71LTUAfACLlB9D7/qjQBBQCTIrkXbLrIDa/%0ADED/e9gyE9AIgEi5Abycz1BkAioBECkvgLfLjwpMQCkAImUF8H4ZX3kJaAVApJwABqepFpeA%0AWgBEyghgeLp3aQnoBXCKtP/tfp4/EEkJwMhlE4UloBjAKdLtbg1f3EVIC8DY5UdlJaAZwCnS%0A776q//6tq90vIqkAGL2Mr6gEVAM4RbpcrhZVu28PjRBpg/nHL4ctKQHdABMifSOSGgDX3evK%0ASUA7gFOk3x2bdnoAXLdnKCcB7QBOkarqq/nBwQYNAM67nBSTgHoAp0gf5+4nh7/lAdy3Cyol%0AAf0A4yK93+977v7fsusgNv9GABO33SokgQwAxkX6ro+934713CEH2XUQm38bgKnb15WRQA4A%0A4yJdfur68H26Tpy+D1U9+9A+2XUQm38TgMnbQBaRQBYADpH6z5D1eB6F7DqIzb8FwPTtVEtI%0AIA8Ap0iX89fH1aKPr/O8R4i02vwztyUuIIFMANwihZTsOojNvz7A3O297SeQCwAiaQaYvU2+%0A+QSyAXCK9NhFqurPEyKJAMw/bsJ6AvkAeIh0rTmTZNdBbP6VATwe22I8gYwAnCJ9V/urPqd9%0A9fdymH0gs+w6iM2/LoDP449sJ5ATgFOkXdUdrqt2HjeKlF0HsflXBfB6jJjpBLICcIp0l6f5%0AiUjbA/g9js9yAnkBOEXa3zft9pef5lMJkTYF8HyspeEEMgNwinSqb8fsTtcPpONlumTXQWz+%0A1QBc1/FtBrBZAzMATpEu569dVe0O1z2l+ZOEZNdBbP61ALw9MptAfgBukUJKdh3E5l8JIOBp%0A5UYTyBAAkdQBBHhkNIEcAdwinQ/3TTtE2hIgxCObCWQJ4BSpd7ABkTYECPLIZAJ5AjhF+rwf%0A/p47qwGRUs4f5pHFBDIFcIrU/0IWkbYCCPTIYAK5AiCSJoBQj+wlkC2AUyQ27bYHCPbIXAL5%0AAjhF4mDD5gDhHllLIGMAp0gc/t4K4Ppu1f6ywCMjCVgAcIsUUrLrIDZ/EoD2c//fMo9sJGAC%0AAJGEAW7XIC/zyEQCNgDGRXq5zhyR1gS4hbzMIxMJ2ABAJGGAKI9MJGADYFyk0JJdB7H5k+0j%0ALfXIRgImABBJHKDyv/xoHQDJBmYAEEkcIMIjIwlYAEAkaYAIjYwkYAIAkYQBojwykYANAESS%0ABYjzyEICRgAQSRQg0iMDCVgBQCTJBrEe5Z+AGYA0IlGLqvFImoFKXXwibdwg5rB3EoAE8wPA%0App00QAqP8k7AFAAiCTVoNSo6AVsAiCTToPs4KjkBYwCIJNLgtllXcALWABBJosF996jcBMwB%0AIJJAg8dhhmITsAeASNs3eB6uKzUBgwCItHmD3mHvQhOwCIBIWzfof31UZgImARBp4wYvX8MW%0AmYBNAETatsHr6QwlJmAUAJE2bfB2WlCBCVgFQKQtG7yfXldeAmYBEGnDBoPTVItLwC4AIm3X%0AYHi6d2kJGAZApM0ajFw2UVgClgEQaaMGo5cfFZWAbQBE2qbB+GV8JSVgHACRNmnguBq2oASs%0AAyDSFg1cV5WXk4B5AETaoIHz7gzFJGAfAJHWb+C+y0kpCRQAgEirN5i4W1AhCZQAgEhrN5i6%0A61YZCRQBgEjrNpi+e10JCRQCgEjrNpi+C2QJCRQCgEirNpi5mWoBCZQCgEhrNpi7K7H9BIoB%0AQKQVG8ze3dt8AuUAINJ6Debvkm89gYIAEGm1Bh5PmzCeQEkAiLRWA5+ntthOoCgARFqpgdfT%0Aj0wnUBYAIq3TwO8pYpYTKAwAkVZp4Pk0PsMJlAaASGs08H2qpd0EigNApBUaeD8d1mwC5QEg%0AUvoG/k9ZtppAgQCIlLxBwNPKjSZQIgAipW4Q4JHRBIoEQKTEDUI8splAmQCIlLTB9HV8GwBs%0APD8AiLQGQKBHBhMoFgCREjYI1MhgAuUCIFK6BsEemUugYABEStYg3CNrCZQMgEipGizwyFgC%0ARQMgUqIGSzyylUDZAIiUpsEij0wlUDgAIqVoEHrYOzmA1PwAIFJCgKUe2UkAAESKb7BUIzsJ%0AAIBI8Q2We2QlAQAQKb5BhEdGEgDgHyJFN4jxyEYCALQ/ECmqQZRHJhIAoPuBSDEN4jyykAAA%0Atx+IFFGRHhlIAABEip8/1qP8EwAAkeLnj/Yo+wQAQKT4+eM9yj0BAP4hUvT8jUdaRhEAcQBE%0AWljt55GWUQRAHACRllW3XadlFAEQB0CkRXXbP9IyigCIAyDSkrofZ9AyigCIAyBSeD0vP9Iy%0AigCIAyBScIoxTskAAAwCSURBVPUu49MyigCIAyBSaPW/PtIyigCIAyBSYL18DatlFAEQB0Ck%0AsHo9nUHLKAIgDoBIQfV2WpCWUQRAHACRQur99DotowiAOAAiBdTgNFUtowiAOAAi+dfwdG8t%0AowiAOAAiedfIZRNaRhEAcQBE8q2xy4+0jCIA4gCI5Fmjl/FpGUUAxAEQya/GL4fVMooAiAMg%0Aklc5LivXMooAiAOEiVQ/p65Vjkiu2zNoGUUAxAGCRHq6U79qJbsOq8/vvM2JllEEQBwgRKT6%0AUqZI7tsFaRlFAMQBgj6RihRp6iliWkYRAHGAaJH+a8pj1lyr9Ugagsqo+EQaq+m7QGp5OwRA%0AHACRJmvmbqpaRhEAcQBEmqq5uxJrGUUAxAEQaaJm7+6tZRQBEAdAJHfN3yVfyygCIA6wRKTm%0AzwLObPB42oSWUQRAHCBMJFfJrsMq8099fbQJwCYNAECkBA3mRBIF2KQBAIiUoMHE/H4PEdMy%0AigCIAyDSaHk+jU/LKAIgDoBIY+X7VEstowiAOAAiNVVVVf/vvZ8Oq2UUARAHQKR/rUd9k/yf%0AsqxlFAEQB0Ckm0dPkwKeVq5lFAEQB0Ckd5ECPFIzigCIAyDSm0ghHqkZRQDEARDp38s+UpBH%0AakYRAHEARGpqoUdqRhEAcQBE6legR2pGEQBxAETqVahHakYRAHEARHpWsEdqRhEAcQBEelS4%0AR2pGEQBxAES61wKP1IwiAOIAiNSV13V8awLINAAAkRI0eM6/zCM1owiAOAAiNbVMIz2jCIA4%0AACL9W+6RmlEEQBwAkSI8UjOKAIgDIFKER2pGEQBxAESK8EjNKAIgDlC8SDEeqRlFAMQBChdp%0A4WHvdADCDQBApAQNLpEeqRlFAMQBihYpUiM9owiAOEDJIkV7pGYUARAHKFikeI/UjCIA4gDl%0AipTAIzWjCIA4QLEitQ8rlwRIMT8AagBKFan9PDIzigCIAxQqUrddZ2YUARAHKFOk2/6RmVEE%0AQBygSJHuxxnMjCIA4gAlivQ4XmdmFAEQByhQpOdxbzOjCIA4QHki9b4/MjOKAIgDFCdS/3tY%0AM6MIgDhAaSK9nM9gZhQBEAcoTKTX84LMjCIA4gBFifR++ZGZUQRAHKAkkQaX8ZkZRQDEAQoS%0AaXi6t5lRBEAcoByRRi6bMDOKAIgDFCPS2OVHZkYRAHGAUkQavYzPzCgCIA5QiEjjl8OaGUUA%0AxAHKEMlxWbmZUQRAHKAEkZx3rzMzigCIAxQgkvsukGZGEQBxAPsiTdwtyMwoAiAOYF6kqbtu%0AmRlFAMQBrIs0efc6M6MIgDiAcZGm7wJpZhQBEAewLdLM3VTNjCIA4gCmRZq7K7GZUQRAHMCy%0ASLN39zYzigCIAxgWaf4u+WZGEQBxALsieTxtwswoAiAOYFYkn6e2mBlFAMQBrIrk9fQjM6MI%0AgDiAUZH8niJmZhQBEAewKZLn0/jMjCIA4gAmRfJ9qqWZUQRAHMCiSN5PhzUzigCIA9gTyX35%0A0UYAG84PgBoAcyIFeGRnFAEQB7AmUoBGhkYRAHEAYyIFeWRnFAEQB7AlUphHdkYRAHEAUyIF%0AemRnFAEQB7AkUqhHdkYRAHEAQyIFe2RnFAEQB7AjUrhHdkYRAHEAMyIt8MjOKAIgDmBFpCUe%0A2RlFAMQB0ogkXo1H0gwU1VTOn0iLPo8MvR0CIA5gQqSFHtkZRQDEASyItNQjO6MIgDiAAZEW%0Ae2RnFAEQB8hfpOUe2RlFAMQBshcpwiM7owiAOEDuIsV4ZGcUARAHyFykKI/sjCIA4gB5ixTn%0AkZ1RBEAcIGuRIj2yM4oAiAPkLFKsR3ZGEQBxgIxFivbIzigCIA6Qr0jxHtkZRQDEAXIVqb19%0AnZYQAQAgU5G620BqCREAAPIU6bZZpyVEAADIUqT77pGWEAEAIEeRHocZtIQIAAAZivQ8XKcl%0ARAAAyE+k3mFvLSECAEB2IvW/PtISIgAA5CbSy9ewWkIEAIDMRHo9nUFLiAAAkJdIb6cFaQkR%0AAACyEun99DotIQIAQE4iDU5T1RIiAABkJNLwdG8tIQIAQD4ijVw2oSVEAADIRqSxy4+0hAgA%0AALmINHoZn5YQAQAgD5H+jF8OqyVEAADIQiSHR2pCBACAHERy3p1BS4gAAJCBSO67nGgJEQAA%0A9Is0cbcgLSECAIB6kabuuqUlRAAA0C7S5N3rtIQIAADKRZq+C6SWEAEAQLVIrsPe3g1iAdZu%0AAIAZAM0izXmkJkQAAFAs0vzNvbWECAAAekXyuEm+lhABAECtSD4Pm9ASIgAAaBXJ66EtWkIE%0AAAClIvk9/EhLiAAAoFMkz4eIaQkRAABUiuT7MD4tIQIAgEaRvB9qqSVEAABQKJL/w2G1hAgA%0AAPpECnjIspYQAQBAnUghDyvXEiIAAGgTKcQjNSECAIAykYI8UhMiAADoEinMIzUhAgCAKpEC%0APVITIgAAKBJp9vKjuQaxANs3AMAMgB6Rwj1SEyIAAKgRKVwjPSECAIAWkZZ4pCZEAABQItIi%0Aj9SECAAAOkRa5pGaEAEAQIVICz1SEyIAAGgQaalHakIEAAB5kRYc9n5bh8VlZhQBEAfQIVLc%0AOiwuM6MIgDiAuEjLNdITIgAASIsU8XmkJ0QAABAWKcojNSECAICsSHEeqQkRAABERYr0SE2I%0AAAAgKVKsR2pCBAAAQZGiPVITIgAAyIkU75GaEAEAQEykBB6pCREAAKREajwyEyIAAAiJ1H4e%0AmQkRAABkROq268yECAAAIiLd9o/MhAgAABIi3Y8zmAkRAAAERHocrzMTIgAAbC5S7zo+MyEC%0AAECISPW1+tOPXwJg+tfDmgkRAAACRKoff/R+Bor08jWsmRABAGBbkV5PZzATIgAALBTp1SNf%0AmLfTgsyECAAAS0V67CL919TcrF21pwVRlO0K/0QKPNgwOE3VzLsRAAAsFOnyOu2zwOHp3mZC%0ABACAzUQauWzCTIgAALBQpOBNu7HLj8yECAAAESL1Ppw8ljdyGZ+ZEAEAIECkx5kNdW/aW6QV%0A10FsfgAAWCKSu+YXV1XVausgNj8AAGwsUlWNmGQmRAAA2EakqhozyUyIAACASAAAkKABIgEA%0AQIIG24jEPhIAxgE2EomjdgDYBthKpDXXQWx+AABAJAAASAiASAAAkKABIgEAQIIGiAQAAAka%0AIBIAACRogEgAAJCgASIBAECCBogEAAAJGiASAAAkaIBIAACQoAEiAQBAggaIBAAACRogEgAA%0AJGiASAAAkKABIgEAQIIGiAQAAAkaIBIAACRogEgAAJCgASIBAECCBogEAAAJGiASAAAkaIBI%0AAACQoAEiAQBAggaIBAAACRogEgAAJGiASAAAkKABIgEAQIIGiAQAAAkaIBIAACRokEYkofoP%0AAADUASASAAAkAEAkAABIAIBIAACQACBDkShKXyESRSUoRKKoBIVIFJWgEImiEhQiUVSCylCk%0A+lplA1wRRBde+PqPJpCfSPXjj1IBmoGUXPjjDzkE4cU//ngWIuUHcF046y+7/IsBkdqS3rSQ%0ABihbJPHFjyAgUpYAiCReBkSS39cVH0hEkq78Dza0JR1kyctHpDGCnETqHXSUCfIBIDWO4gCP%0AZZcu0gAgJ5G6UjCO4sOISLKLH1k+Ii0kkC1E0rb4/EQS/2K9rqUJpN9IxFdfeotk5BWQoUgU%0Apa8QiaISFCJRVIJCJIpKUIhEUQkKkSgqQSESRSUoRKKoBIVIFJWgECn3qqr+1FH6nINSC5Fy%0Ar1eRKgZUpsg993pVB5GEitzzqKsgH9X+1E791vvL5fRZVZ+nt3+5/g+TZIrY86irNVdH6nMz%0Ata8+L+e6evz+8ZhCJLEi9jzqqs/5sq8OzdT1j8uhun4q3X5//gubdmJF7nnUdYPuujlX7Zqp%0AZjNu1/z5/P02hUhiRe55VCfIU5Wx3xFJsMg9j0Ik5UXuedRtA25/V4VNO2VF7nlUdXXovK++%0A7qq8HGzo/QsiCRW551HNQe+qqi93VRyHvy/df0NtXoiUR1034PaPL2Cbev1C9uP+L0dEkilE%0AyqPYZFNejE8ehUjKi/HJoxBJeTE+eRQiKS/Gh6ISFCJRVIJCJIpKUIhEUQkKkSgqQSESRSUo%0ARKKoBIVIFJWg/gdDBjs7LOvP2AAAAABJRU5ErkJggg==)

In \[66\]:

    ggplot(cotton_new3,aes(probit,log(x-a_cotton3)))+
      geom_point()+
      geom_smooth(method="lm",se=F)+
      ggtitle(paste("Cotton",",","a=",a_cotton3,",",
                    "R2=",format(Rsquared33[which.max(Rsquared33)],digits=3),",",
                    "mu=",mu_cotton3,",","sigma=",sigma_cotton3))

    `geom_smooth()` using formula 'y ~ x'

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA0gAAANICAMAAADKOT/pAAAAOVBMVEUAAAAzMzMzZv9NTU1o%0AaGh8fHyMjIyampqnp6eysrK9vb3Hx8fQ0NDZ2dnh4eHp6enr6+vw8PD///9uyY2wAAAACXBI%0AWXMAABJ0AAASdAHeZh94AAAgAElEQVR4nO2di3qyOhNG+T/UWmvr1vu/2J+DB0QCSRhgJlnv%0As5/dfjZdzoCrIEIoboSQ2Sm2LoCQFIJIhAgEkQgRCCIRIhBEIkQgiESIQBCJEIEgEiECQSRC%0ABIJIhAgkSqSfQ1EUh/PnD65fr/8L5ndfFPu3p/veFeXxWn9XPDP4q48f7trRt9upYu1OY8/2%0ALfrHpUd7FX67/VWL8evy/NGvo4VuPIZM5nNp3o5lVVXz3fVtYb4ev512b8WSXiJWzF95X9j7%0AT1rx+r9cftun+349sm8eKC+3jkjl4O++PCuvr98cKP2Zs2j5PVqn8NtfW8rzxblbR6TPpXkp%0AXwvw3BGp+/ixffh39tOnmvAVU63/Q708qz9shw/aIiLtip9qK9ihnor9pdrudZ//XPwN/u7j%0At6pq6z+tx/o3b79l93X0npNr2xaVHu2t8ENxqh85DA9dLB9Ls7L7dL1dHgvoPPT4uSirdX7Z%0AD/+1IjEi7YrHjtG+6O/dLSLSb/uqP9YvgEcJ197z/BWOvbXnmL/mL2tZdP4xkL994bNh8MwH%0A7a3w94V1lXzikXwuzb/W5WuzTO4V9h/ft9ui39dvkfcEr7vzazvw1/wNu/19VXvSX/UGod0p%0AeO4avH5Qv1yq/e3dwNuqJsfqRbR37TYc243NX9F/69Wxodw5fvn14nx7mbpes0XVkvP1XP3g%0Au23iVLbvs+5DQ2lt4a3UjyYOhfs90t9XtYP19ft6pvO+KE/NPz5rCl6a352/h5XOt6HHH6V9%0A7oSQNsEiHfqbocde9bkvUucH1WP75/efKbsDPw4e7Nvvuuu4TrXP8fzz+O3YsfvYIj0f3g0+%0A1a3WfkSk7/aNwlfz5fQmkj/tUfh3u2vX7mT+Vi9R1xP/dpdw+5t1ju3T9msKXpqH19u0ap0d%0Aay9/+o8/RWLfzpFgkcrXtr9J9Zbp+3q7Hpt3zd29ld4Pqn3s62H4Tf5382L6bn44sOqfK/Gt%0A1q6VF/cfyud7pLI4vh4918849NL/fJ63H5Tn+k962X7ZTYo0SHsV/l2/5O9v1updKtcTN+9q%0Aqrc1jyeslmz1wLlsn7ZfU/DSrL4774ryvsfX5th7fNf+pZI9EJNUghdMf1Ee7y/Qr/prV6Te%0AD+oXz3V4PXy+5Rl8xref/+0PzUuoycG5Qeq8lLoO78b+sI6IdG/i/Bg2sWs39LNX4X/NRvrQ%0A/Nk/dbZvY4jm22P7hvCnraBfU/DSfGzU6gXUOnvZt1u/1+PNUZK7u2Qos0Xa3XcALq+/0O3/%0AB37gfsX9nb/3QSJVud7fAVcvI/fR7IdGbx8ddXda3M838IPuSzRSpGfhf81r829fHwtvd7Rc%0AmMq8r5/Li3ZX5dpW0K/pFro0241aJc9rEdU7mu+P7197k2QowQtm19u1ey7a1wurd+xu+hV3%0AKgf2iT6e4ePnf3eBvh1vvZ6/dC3fRhxHxruL7DcRL1Jb+NfjQNhXc6RhBNN+nvM6utFdIJ81%0ABS/N+9bsr/se9LWxezx+at46IZIrwQvmq/sqvIiIdKp2H44/l/uL5GOv/vBY9R/bnfuYcvJ1%0A/Nv9KPHhUcR7pP6XmPdIvd/t/fLwM5+bQwnfXiIFL82hP1T9Au/546idK8EidQ5/X8q9yK7d%0Arn2Vu1a98/D3HTdyqOH5jN/3Exvq/ary9/EjMyLdmkPg5eCuXb+m4KU5KNJu8PFT94gN6SZ8%0AU10+jjpfmh2m432NDBxs+PiB4zXaPuo8InT/CLGzA3c/cnhun+Hk+jC2+4y7ezWXspw6YSxI%0ApOtY5R+0buH79vvfz03DCKkZcj/YcBoWKXhp3t8cNe+LykdRx8HHvzhHyJVwkX7bU4Su1U5z%0AvSWo9iGO7VHu5kOT9vS3y+cP2mdzbJFOo0eE7ie1vI60fRf7a3NEu/njOnLM7vWMf00Z1bul%0A6Q9CAkTaF4frxLGst591Cz83H5peOqeHuDDtobRjewDg9jj8/VO4tkiBS/P3fvpP+2exaqg+%0AnnjtP14Vfj2NnaOYeSLePD4+aL3vUXU/d90V7aps1uL7B7Lts72e7m2H4R7Hn7v72Z0/r1+7%0An3rafgaz6x+D6+/sN/lq3jN/Te9D9V7PnyV3vrQflY4ey3rfQeoWfv/M5vgxtP+P+wey5fMd%0A6fericH3SIFL89Qp5F7gqff4tXyWQAYTIdLtWh9cLb4ef0k7ZwL97eq/dO3/+6cIdb70vq2P%0ACJVfv2fne53Oif/3X6tP6j883+v0exoSqdo7OXm9GQkR6fbbfGLpL9Jb4efe9QwukW6/zSlC%0Al9ej9W8efodFilia9SlIj88H6gIfRXUev9ZnePEGyZ0YkdQ/syh6qyU0+bxXDqEpymYi/bhO%0AM1WGXrDO2Odt98t+92MHWcjK2Uyk43KvAlH0gnXGPu/jLRLv/BVlu107Ep1zfan/nkuDNAWR%0ACBEIIhEiEEQiRCCIRIhAEIkQgSASIQJBJEIEgkiECASRCBFIqEj/Nbl/mR9AgCyDEAkQIAEQ%0AIgECJABCJECABECIBAiQAAiRAAESACESIEACIEQCBEgAhEiAAAmAEAkQIAEQIgECJABCJECA%0ABECIBAiQAAiRAAESACESIEACIEQCBEgAhEiAAAmAEAkQIAEQIgECJABCJECABECIBAiQAAiR%0AAAESACESIEACIEQCBEgAhEiAAAmAEAkQIAEQIgECJABCJECABECIBAiQAAiRAAESACESIEAC%0AIEQCBGgCVBTF9HBEAgRoFFQUHiYhEiBAo6Ci8DEJkQABGgUV//4hEiBAc0H//jUmTQ1HJECA%0ARkB+HiESIEBjIE+PEAkQoBFQ7ZHfcEQCBMgF8vYIkQABcoL8PUIkQIAcoH8BHiESIEDDoCCP%0AEAkQoMEEaYRIgAANJtAjRAIEaCChHiESIECfaT6GDXteRAIEqBff0xm6z4tIgAC95+4RIgEC%0ANCMPjxAJEKD4RHmESIAAvaU9XhfqESIBAtTN87h3IAiRAAF65fX5ESIBAhSbzuewiAQIUGS6%0A5zMgEiBAcXk7LwiRAAGKyvv5dYgECFBE+pcfIRIgQOH5uIwPkQAB8s7jg9fPyyYQCRAg3zxO%0ABRq4/AiRAAHyzGN+/KHL+BAJECDP3EUavBwWkQAB8syIR4gECJB33B4hEiBA/nF6hEiAAAXE%0AOV0QIgEC5B33tFuIBAiQb0amr0MkQIA8MzYNJCIBAuSX0elUEQkQIK+MT0uMSIAA+WRiem9E%0AAgTII1PT5CMSIECTmb6LGCIBAjSWx+neE7dtQSRAgEYycnpdGKg3HJEA5QTy9QiRAAEaia9H%0AiAQI0EhajzxmyEckQIBG4ntXS0QCBMgd77vDIhIgQM74360ckQABcsXfI0QCBMiVAI/WE4kQ%0AY6k9Wv5Z2CIBShsUsj1i1w4QoOGEeYRIgAANJdAjRAIEaCChHiESIECfCfYIkQAB+ki4R4gE%0ACFA/ER4hEiBAvTTn13mdYDcBGhuOSIASB7UehZqESIAAdfPwCJEAAYoHPT1CJECAokHNcYYI%0AjxAJEKBX7sfrwj1CJECAnok57j0Imh6OSIBSBXlMA+kH8hmOSIASBc3yCJEAAWoySyNEAgSo%0AyUyPEAkQoP/me4RIgAAJeIRIgAAJeIRIgAAJeIRIgLIHSXiESIByB4l4hEiAMgfJeIRIgDIF%0AtSem3oQ8QiRAeYLul0pIeYRIgLIEtdfued7W0ieIBChHkLRHiAQoS5C0R4gEKE/Q3aOtKkIk%0AQImA2suPEAkQoDm5X8aHSIAAzcjj7REiAQIUn+dhBkQCBCg6r8N1iAQIUGw6h70RCRCgyHQ/%0APkIkQIDi8vYxLCIBAhSV99MZEAkQoJj0TgtCJECAItI/vQ6RAAEKz8dpqogECFBwPk/3RiRA%0AgEIzcNkEIgECFJihy48QCRAg77inZ0AkQIB8M3I5LCIBAuSZh0ZDl5UjEiBAnhnxCJEAAfJN%0A69HwfcoRCRAg37g9QiRAgHwz4hEiAQLkmdHZ6xAJECCvjM8CiUiAAPlkYjZVRAIEyCNTsxIj%0AEiBA05mc3RuRAAGazPQs+YgECNBUPO42gUiAAE3E564tiAQI0Hi87n6ESIAAjcbvLmKIBAjQ%0AWDzvxodIgACNxPeulogECJAzzsuPQkH+QSRAyYH8PUIkQIBcCblZOSIBAjScEI8QCRCgwThm%0ACwoHBQaRAKUECvQIkQABGkg7zYnruvIAUHAQCVBCoNYjRFqsbkBZgO4eIdJidQPKAPQv3CNE%0AAgSol/Zj2DCPEAkQoPcEHq5zg+KCSICSAMV5hEiAAHUT6REiAQLUSaxHiAQI0CvRHiESIEDP%0AxHuESIAAPTLDI0QCBOieOR4hEiBAbWZ5hEiAADWZ5xEiAQJUZ6ZHiAQI0H/zPUIkQIAEPEIk%0AQIAEPEIkQIAEPEIkQLmDAmaBHAfJBJEAmQTJeIRIgPIGyWiESIDyBkl5hEiAcgaFzl7nDiIB%0AyhcUMVuQK4gEKFvQwyMJkxAJUK6gl0eItHrdgJIBPWavQ6Qt6gaUCqg9XncT8siESGUVRAIk%0ACrof976FzqjqigGRyuf/EAmQEOjx+ZGeiiJBiARoQ9Dzc1g1FcWCAkR62YRIgERAr/MZtFQU%0ADYoS6X91vIYT4k7t0dY1yMdLJA42ABIDdc+v01HRDBAiAdoI9HaeqoqK5oDCROp4hEiA5oB6%0Alx8pqGgeKEikrkeIBGgGqH8Z3/YVzQSFiPTmESIBigd9XH60eUVzQQEileXbqQ3b1g3IMujz%0AMr6tK5oNCtkivWfbugEZBg1cDmu+NUQCtDZo6LJy860hEqCVQYPTM5hvDZEArQsanubEfGuI%0ABGhVkGO6IPOtIRKgNUGuabfMt4ZIgFYEOaevM98aIgFaD+SeBtJ8a4gEaDXQyHSq1ltDJECr%0AgcamJTbeGiIBWg00Or237db+QyRAa4HGp8k33VozHJEArQGauN2E5dba4YgEaHnQ5F3E7Lb2%0AGI5IgBYHTd+Nz2xrz+GIBGhp0H2W/PkgjyASoFRBPnc/MtpaZzgiAVoW9Lxry1yQVxAJUJog%0AL49stvY2HJEALQlqDzNM3mrCYmvvwxEJ0IIg37uVG2ytNxyRAC0H8vXIYGv94YgEaDGQt0f2%0AWvsYjkiAlgL5e2Sutc/hiARoIVCAR9ZaGxiOSICWAYV4ZKy1oeGIBGgRUJBHtlobHI5IgJYA%0AhXlkqrXh4YgEaAFQoEeWWnMMRyRA8qBQjwy15hqOSICkQdOXH3mCIoJIgFIBRXhkpbWR4YgE%0ASBYUoZGV1saGIxIgUVCURzZaGx2OSIAkQXEemWhtfDgiARIERXpkobWJ4YgESA4U65GB1qaG%0AIxIgMVC0R/pbmxyOSICkQPEeqW9tejgiARICzfBIe2sewxEJkAxojkfKW/MZjkiARECzPNLd%0AmtdwRAIkAZrnkerW/IYjEiAB0EyPNLfmORyRAM0HzfVIcWu+wxEJ0GzQbI/0tuY9HJEAzQXN%0A90hta/7DEQnQTJCAR1pbCxiOSIDmgZrbTUiARIJIgGyCPG7H5weSCSIBMgnyuR2fF0goiATI%0AIsjvNmIeIJFyNgQhEqAZkfJIYWuIBGg1kOft+DyirjVEArQaSOK49z3aWkMkQKuBBD3S1lo4%0ACJEAxaWZBlJVRZuCEAlQVNrpVDVVtC0IkQDF5L5bp6iijUGIBCgij7dHeiraGoRIgMLzPMyg%0ApqLNQYgEKDivw3VaKtoehEiAQtM57K2kIgUgRAIUmO7HRzoq0gBCJEBhefsYVkVFKkCIBCgo%0A76czaKhIBwiRAIWkd1qQgoqUgBAJUED6p9dtX5EWECIB8s/HaaqbV6QGhEiAvPN5uvfWFekB%0AIRIg3wxcNpFKa/NBiATIM0OXHyXSmgAIkQD5ZfAyvjRakwAhEiCf/Bu+HDaF1mRAiATIIw6P%0AUmhNCIRIgKbjnJ3BfmtSIEQCNBn3LCfmWxMDIRKgqYzMFmS9NTkQIgGayNisW8ZbEwQhEqDx%0AjM5eZ7s1SRAiARrN+CyQplsTBSESoLFMzKZquTVZECIBGsnUrMSGWxMGIRIgdyZn97bbmjQI%0AkQA5Mz1LvtnWxEGIBMgVj7tNWG1NHoRIgBzxuWuL0dYWACESoOF43f3IZmtLgBAJ0ECKwu8u%0AYgZbWwiESIA+4+uRwdaWAiESoI80Gv3zucOyudYWAyESoI+0HiFS0HBEAtRLqxEihQ1HJEDv%0ACfDIWmsLghAJ0HuawwyeHhlrbUkQIgF6i+fhumlQWMyDEAlQN2EemWptWRAiAeok0CNLrS0M%0AiheJpJfao61rMB+2SNmDQrdHhlpbHIRIgB4J98hMa8uDEAnQPREeWWltBRAiAWoT45GR1tYA%0AIRKgJlEe2WhtFZBbpOv3oSiKw/cVkTIAxXlkorV1QE6RjsUjR0RKHhTpkYXWVgI5RPoty+P5%0AUn1zOR+L8heR0gbFemSgtbVAwyKdy1PnX6fyjEgJg1x3EQsGzYh50LBIXz1p+v9GpIRAMzzS%0A3tqKIMeunUe2rRuQFGiGRtpbWxM0KtKxKL4uiJQ0aJZHultbFeQQ6VQWx1t9+LsoXSZtWzcg%0AGdA8j1S3ti5oWKRTrdCh+Km/G3p/hEiJgGZ6pLm1lUHDIu2K8+1c7JqHXe+dtq0bkARorkeK%0AW1sbNCxSY8/hiEhpg2Z7pLe11UGuLdL9M9jrN7t2yYLme6S2tfVBwyL9PLZDHGxIFyTgkdbW%0ANgANi3T72bVfy6PrnFVEMg6S8Ehpa1uAHCJ5ZNu6Ac0EiXiks7VNQIiUJ0jGI5WtbQNyi3Qs%0A75dRIFJ6ICGPNLa2Ecgp0ut6JERKDiTlkcLW1IlUFqfbaLatG9CMiHmkrzV9Ijm3RIhkHSTn%0AkbrWFIp0KJwHvhHJNEjQI22tbQhyinQp984rKBDJMKiZllgCVAfQc7hLpIKDDUmCmu2RqorS%0AACFSXqB2v05TRYmAnCJNZtu6AUXl/v5IUUWpgBApJ9DjOIOeipIBuUW6HndFseOk1YRAz+N1%0AaipKB+QU6XI/Q4jLKJIBvY57a6koIZBTpK+iPvx92XNhXyqgzudHSipKCeQU6XVpHyKlAHqb%0ABlJFRWmBECkP0Pt0qhoqSgzkFIldu5RAvdOCFFSUGsgpEgcbEgL1T6/bvqLkQE6ROPydDujj%0ANNXNK0oP5BZpKtvWDcg/n6d7b11RgiBESh40cNlEKq0pAjlFehytK0tEsguq3uQOXX6UQmvK%0AQMMilUXB2d8JgBwepdCaNtCwSKeOR66pG7atG5BHXB4l0Jo60OSunTPb1g3II61HxXyQM4Ce%0Aw10iTWbbugF5pPUIkdYATW6RONhgF+TyKIHW1IGGReJgQwogp0f2W9MHGhaJgw0JgEam3bLe%0AmkLQ5K6dM9vWDWgqY9PXGW9NI8gp0mS2rRvQREangbTdmkqQWyROWjUM+jfqkenWlIKcInEZ%0AhWHQhEeWW9MKcorEhX12QZOze9ttTS3IKRKXmpsFTc+Sb7Y1vSBESg7kcbcJq60pBjlFYtfO%0AKMjnri1GW9MMcorEwQabIK+7H9lsTTXIKRKHv02C/O4iZrI13SC3SFPZtm5Ag/G8G5/F1pSD%0AECklkO9dLQ22ph3kFunQPFDseI9kBuR9d1h7rakHOUU6tse9C47amQH532XZXGv6QU6RyuK3%0A/vLH50hWQAF3K7fWmgGQUyQ+kDUGCvDIWmsWQE6RDsXXtT4GXuwRyQIoxCNjrZkAOUV6fiD7%0Ah0gGQEEe2WrNBsgp0uMDWddBO0TSBArzyFRrRkBukaaybd2AOpm6/MgbFBxAz+E+Ig0ecNi2%0AbkCvBHtkpzU7IEQyDwrWyE5rhkCIZB0U4ZGV1iyBEMk4KMYjI62ZAiGSbVCURzZaswVCJNOg%0AOI9MtGYMhEiWQZEeWWjNGgiRDINiPTLQmjkQItkFRXukvzV7IEQyC4r3SH1rBkFeIg1m27oB%0AzfBIe2sWQW6RjtxoTDNojkfKWzMJcor09AiRNIJmeaS7NZsgp0hl8bcvLtd9e8U5IukCzfNI%0AdWtGQU6Rqi3Rd3G+XblCViFopkeaW7MKGhPpXN8/ll07faC5HiluzSzIKdKh+LkUu9svImkD%0AhV9+5AAJBNBzuEuk2qB9fayBee10gQQ80tqaZZBTpNt5V9/bpTg6PEKkbUACGmltzTTILdJU%0Atq07V5CIRzpbsw1CJFMgGY9UtmYchEiWQEIeaWzNOgiRDIGkPFLYmnkQItkBiXmkrzX7IEQy%0AA5LzSF1rCYAQyQpI0CNtraUAGhXJeVYDIq0Oqj3SVRGgt+GIZALUbI9UVQTofTgiWQC1+3Wa%0AKgLUG45IBkD390eKKgLUH45I+kGP4wx6KgL0MXxMpNFsW3dOoOfxOjUVAfocHiRSiUjrg17H%0AvbVUBGhgeIhIJSKtD+p8fqSkIkBDwwNEKtkirQ/qfg6royJAg8NDtkiItDro7XwGFRUBGh4e%0AI9L/6ngMJ3PTnM9ATKUn0v6v/Xo9dB5ki7QuqHd+nYKKALmGu0S6z9bw/fZZEiKtCuqfp7p9%0ARYCcw10i/e2L8uenLHZ/iLQR6ON8780rAuQe7hLpdqssKnbnt4cQaR1QURRD102k0FqqoBGR%0Azoi0Eai5d8HA9UcJtJYsyL1rt2PXbiPQQ6OP6/jst5YuyClSUXzXX76Zsnh1kMujBFpLF+QU%0A6XBtv74d/kakNUCtR8V8kDOAxEHDIvXn+x6a/3vbupMGOTxKobVkQcMinctT51+n8v2QAyIt%0AC3J5lEBr6YKGRbr9luXxfKm+uZyPRTl4075t604Y5J4uyHxrCYMcInXvIeu4H8W2dacLGpl2%0Ay3prKYOcIt2u34fKosP3ddgjRFoGNDZ9nfHWkga5RZrKtnWnChqdBtJ2a2mDEEkVaHw6VdOt%0AJQ5yivR8i1SUXxdEWgc0MS2x5dZSB3mIVGXIpG3rThI0Nb234daSBzlFOhf7Sp/Lvvi5HQdv%0AyLxt3SmCJqfJt9ta+iCnSLuiPVxX7BwTRW5bd4Kg6dtNmG0tA9DIrt3rKyKtAPK4bYvV1nIA%0AOUXaP3bt9rffequESMuCfG5/ZLS1LEBOkS7l/ZjdpdognW6f2bbu1EBetxGz2VoeIKdIt+v3%0Arih2x+qd0vBJQtvWnRZo+PKjCJBnAImD3CJNZdu6kwJ5emSxtWxAiLQ9yPvusPZaywfkFul6%0AfOzaIdKiIP+7LJtrLSOQU6TOwQZEWhIUcLdya63lBHKK9PU4/D10VgMiiYECPLLWWlYgp0jd%0AD2QRaTFQiEfGWssLhEibgoI8stVaZiCnSOzarQAK88hUa7mBnCJxsGF5UKBHllrLDuQUicPf%0Ai4NCPTLUWn4gt0hT2bbuBEDBHtlpLUMQIm0FCvfITGs5goZFervOHJGWAEV4ZKW1LEGItA0o%0AxiMjreUJGhbJJ9vWbRwU5ZGN1jIFIdIWoDiPTLSWKwiR1gf5Xn40CYoOIHEQIq0OivZIf2sZ%0AgxBpbVC0RvpbyxmESCuDZnikvbWsQYi0LmiOR8pbyxuESKuCZnmku7XMQYi0JmieR6pbyx2E%0ASCuCZnqkubXsQYi0HmiuR4pbA4RIq4Fme6S3NUCItBpovkdqWwOESKuBBDzS2hqg/xBpLZCE%0AR0pbA9QMR6Q1QCIe6WwNUDsckVYAyXiksjVA9+GItDxIyCONrQF6DEekxUFSHilsDdBzOCIt%0ADRLzSF9rgF7DEWlhkJxH6loD1BmOSMuCBD3S1hqg7nBEWhRUe6SrIkCLgBBpUVCzPVJVEaBl%0AQIi0JKjdr9NUEaCFQIi0IOj+/khRRYCWAiHScqDHcQY9FQFaDIRIi4Gex+vUVARoORAiLQTq%0ATAOppCJAS4IQaRlQdzpVHRUBWhSESIuA3j6GVVERoGVBiLQE6P10Bg0VAVoYhEgLgHqnBSmo%0ACNDSIESSB/VPr9u+IkCLgxBJHPRxmurmFQFaHoRI0qDP0723rgjQCiBEEgYNXDaRSmuAxoYj%0Akiho6PKjRFoDNDockSRBg5fxpdEaoPHhiCQIGr4cNonWAE0MRyQ5kOOy8hRaAzQ1HJHEQK7p%0AGRJoDdDkcESSAjmnObHfGqDp4YgkBHJPF2S+NUAewxFJBjQy7Zb11gD5DEckCdC/EY+MtwbI%0AbzgiCYBGPbLdGiDP4Yg0HzQxm6rl1gD5Dkek2aCpWYkNtwbIezgizQVNzu5ttzVA/sMRaSZo%0AepZ8s60BChiOSPNAHnebsNoaoJDhiDQL5HPXFqOtAQoajkhzQF53P7LZGqCw4Yg0A+R3FzGT%0ArQEKHI5I8SDPu/FZbA0QIq0G8r2rpcHWACHSaiDvu8Paaw0QIq0G8r/LsrnWAIWDECkSFHC3%0AcmutAYoAIVIcKMAja60BigEhUhQoxCNjrQGKAiFSBGj88qMAUGAA6QUhUjgo0CNLrQGKBSFS%0AMChQI0utAYoGIVIoKNgjO60BigchUiAo3CMzrQGaAYoXKb8URXGrPdq6DqI6bJEmQJVHRcT2%0AyEJrgGaDEMkXFO2R/tYAzQchki8o2iP9rQGaD0IkX1DrUTEfFB9AekGI5A2K9chAa4BmgxDJ%0AFxTtkf7WAM0HIZInKPL90SdoRgDpBSGSH2iGR9pbAyQBQiQv0ByPlLcGSASESD6gWR7pbg2Q%0ADAiRPEDzPFLdGiAhECJNgkIvP3KCZgeQXhAiTYFme6S3NUByIESaAM3WSG9rgARBiDQOEvBI%0Aa2uAJEGINAqS8Ehpa4BEQYg0BhLxSGdrgGRBiDQCkvFIZWuAhEGI5AYJeaSxNUDSIERyRsoj%0Aha0BEgchkitiHulrDZA8CJEckfNIXWuAFgAh0nAEPdLWGqAlQIg0mGb6OglQHUAZgBBpKM32%0ASFVFgJSDEGkg7X6dpooAaQch0mfu748UVQRIPQiRPvI4zqCnIkD6QYjUz/N4nZqKABkAIVIv%0Ar+PeWioCZAGESO/pfH6kpCJAJkCI9Jbu57A6KgJkA4RI3bydz6CiIkBGQIjUyft5QRoqAmQF%0AhEiv9M6vU5GpIiMAAAp1SURBVFARIDMgRHqmf57q9hUBsgNCpEc+zvfevCJAhkCI1GZgGshU%0AWgO0BgiRmgxNp5pIa4BWASFSncHL+NJoDdA6IET6z3U5bBKtAVoJhEjOy8pTaA3QWiBEck7P%0AkEBrgFYDIZJzmhP7rQFaD5S9SO7pgsy3BmhFUO4ijUy7Zb01QGuCMhdpbPo6460BWhWUt0ij%0A00Dabg3QuqCsRRqfTtV0a4BWBuUs0sS0xJZbA7Q2KGORpqb3NtwaoNVB+Yo0OU2+3dYArQ/K%0AVqTp202YbQ3QBqBcRfK4bYvV1gBtAcpTpKHLj6JAXgGUAShLkbw8stkaoI1AOYrkeTc+i60B%0AQqTVQL53tTTYGiBEWg3kfXdYe60BQqTVQP53WTbXGqANQbmJFHC3cmutAdoSlJlIAR5Zaw3Q%0ApqC8RArxyFhrgLYFZSVSkEe2WgO0MSgnkcI8MtUaoK1BGYkU6JGl1gBtDspHpFCPDLUGaHtQ%0ANiIFe2SnNUAKQLmIFO6RmdYAaQBlIlKER1ZaA6QClIdIMR4ZaQ2QDlAOIvldfuQBigygDEAZ%0AiBTpkYXWAKkBpS9SpEYWWgOkB5S8SNEe6W8NkCJQ6iLFe6S+NUCaQImLNMMj7a0BUgVKW6Q5%0AHilvDZAuUNIizfJId2uAlIFSFmmeR6pbA6QNlLBIMz3S3BogdaB0RZrrkeLWAOkDJSvSbI/0%0AtgZIIShVkeZ7pLY1QBpBiYok4JHW1gCpBKUpkoRHSlsDpBOUpEgiHulsDZBSUIoiyXiksjVA%0AWkHpiRR7+dFH9LUGSC8oOZHEPNLXGiDFoNREEtNIX2uANIMSE0nQI22tAVINSkuk2iNdFQHK%0ABJSUSM32SFVFgHIBpSRSu1+nqSJA2YASEun+/khRRYDyAaUj0uM4g56KAGUESkak5/E6NRUB%0AygmUikiv495aKgKUFSgRkTqfHympCFBeoDRE6n4Oq6MiQJmBQkQqq6gU6e18BhUVAcoNFCBS%0A+fyfMpHezwvSUBGg7EAJiNQ7v05BRYDyA9kXqX+e6vYVAcoQFCXS/+pMDV8pzXmqhKiJzS3S%0A53UTW1cEKEuQcZEGrj8yv0oAWQTZFmnoOj7zqwSQRZBpkQavhzW/SgBZBFkWafi6cvOrBJBF%0AUIBI2s5scMzPYH6VALIIChHpPdvW7Z7nxPwqAWQRZFYk53xB5lcJIIsgoyKNTANpfpUAsgiy%0AKdLYdKrmVwkgiyCTIo1OA2l+lQCyCLIo0vh0quZXCSCLIIMiTUxLbH6VALIIsifS1PTe5lcJ%0AIIsgcyJNTpNvfpUAsgiyJtL07SbMrxJAFkHGRPK4bYv5VQLIIsiWSD63PzK/SgBZBJkSyes2%0AYuZXCSCLIEsi+d2Oz/wqAWQRZEgkz9taml8lgCyC7Ijke3tY86sEkEWQGZG8b7NsfpUAsgiy%0AIpL/7crNrxJAFkFGRPL3yP4qAWQRZEKkscuPgkBBAQQoYLgBkYI8sr9KAFkEGRApSKMEVgkg%0AiyD9IgV6ZH+VALIIUi9SqEf2VwkgiyDtIgV7ZH+VALIIUi5SuEf2VwkgiyDdIkV4ZH+VALII%0AUi1SjEf2VwkgiyDFIoV9fDQCigwgQAHD1YoU6ZH9VQLIIkitSJEaJbBKAFkEaRUp2iP7qwSQ%0ARZBSkeI9sr9KAFkE6RRphkf2VwkgiyCVIs3xyP4qAWQRpFGkWR7ZXyWALIL0iRR72PsDNDeA%0AAAUM1ybSXI/srxJAFkHaRJqrUQKrBJBFkDKR5ntkf5UAsgjSJZKAR/ZXCSCLIFUiSXhkf5UA%0AsgjSJJKIR/ZXCSCLIEUiyXhkf5UAsgjSI5KQR/ZXCSCLIDUiSXlkf5UAsgjSIpKYR/ZXCSCL%0AICUiyXlkf5UAsgjSIZKgR/ZXCSCLIBUi1R6ZX5KAsgZpEKnZHplfkoCyBikQqd2vM78kAWUN%0A2l6k+/sj80sSUNagrUV6Xn5kfkkCyhq0sUivy/jML0lAWYO2Falz2Nv8kgSUNWhTkbofH5lf%0AkoCyBm0p0tvHsOaXJKCsQRuK9H46g/klCShr0HYi9U4LMr8kAWUN2kyk/ul15pckoKxBW4n0%0AcZqq+SUJKGvQRiJ9nu5tfkkCyhq0jUgDl02YX5KAsgZtItLQ5UfmlySgrEFbiDR4GZ/5JQko%0Aa9AGIg1fDmt+SQLKGrS+SI7Lys0vSUBZg1YXyTU9g/klCShr0NoiOac5Mb8kAWUNWlkk93RB%0A5pckoKxB64o0Mu2W+SUJKGvQqiKNTV9nfkkCyhq0pkij00CaX5KAsgatKNL4dKrmlySgrEHr%0AiTQxLbH5JQkoa9BqIk1N721+SQLKGrSWSJPT5JtfkoCyBq0k0vTtJswvSUBZg1YR6d+0R/aX%0AJKCsQWuI5OOR/SUJKGvQCiL53UXM/JIElDVoeZE878ZnfkkCyhq0uEi+d7U0vyQBZQ1aWiTv%0Au8OaX5KAsgYtLJL/XZbNL0lAWYOWFSngbuXmlySgrEFLilQEeGR/SQLKGrSgSEEe2V+SgLIG%0ALSdSUVQmFcVCdQMCpAq0qEh1FqobECBVIEQCBEgAtJxI/wV5ZH9JAsoatKBI/4V4ZH9JAsoa%0AtKRIS9YNCJAqECIBAiQAQiRAgARAiAQIkAAoXiRCyEDYIgECFA1CJECABECIBAiQAAiRAAES%0AACESIEACIEQCBEgAhEiAAAmAEAkQIAEQIgECJABCJECABECIBAiQAAiRAAESACESIEACIEQC%0ABEgAhEiAAAmAEAkQIAEQIgECJABCJECABECIBAiQAAiRAAESACESIEACIEQCBEgAhEiAAAmA%0AEAkQIAEQIgECJACKF6nN/6J+a8lQ0XSoaDrRFSHSUqGi6SRUESItFSqaTkIVIdJSoaLpJFQR%0Ad6MgRCCIRIhAEIkQgSASIQJBJEIEgkiECCROpLKKcCEzo6+iqqatC3gLS2g6M5ZRlEjl839a%0Aoq+ieq1sXUE3LKHpzFlGiLRUSl31sISms7pIr2dVFW0VqapHoUj66qmDSOoqUlUPInlmXZEU%0AvnHVt1ZUFYRIfln1YMOsZ1wsFDQWRPLLSiJ1jg8qWQjPipTUo7GiOojkleiKEjlqp66cOqpK%0A0rjO1NUzo6BURNJVTRtVNSlcZ0nVk8iZDWWpriRtLxOFC0jZEpr1KuJcO0IEgkiECASRCBEI%0AIhEiEEQiRCCIRIhAEIkQgSASIQJBJEIEgkh2UxTd707KThPILIhkN+8iFazKLcPSt5t3dRBp%0A07D0tacS5FDsL813f+X+drt8FcXXpfeT6j9M2jIsfO2prKkcKa/1d/vi63Yti+e/D8/vEGnj%0AsPC1p9LnetsXx/q76n+3Y1Ftle7/fv2EXbuNw9LXnmqHrtqdK3b1d/Vu3K7+/+vf9+8QaeOw%0A9LWnFeSlytC/EWnzsPS1B5FMhKWvPfcduP1DFXbtVIalrz1F5dB1X3w/VHk72ND5CSJtGpa+%0A9tQHvYuivD1UcRz+vrVjyEZBJO2pduD2zw9g67x/IHt4/OSESFsGkbSHXTYTYS1pDyKZCGtJ%0AexDJRFhL2oNIJsJaIkQgiESIQBCJEIEgEiECQSRCBIJIhAgEkQgRCCIRIpD/A8897zWU/F+T%0AAAAAAElFTkSuQmCC)

In \[67\]:

    # 추가 확인: mu=lnb, sigma=1/alpha

    c(log(b_steel),mu_steel1,1/alpha_steel,sigma_steel1) # steel
    c(log(b_cotton),mu_cotton3,1/alpha_cotton,sigma_cotton3) # cotton

1.  1.70055750580158
2.  1.783
3.  0.390625
4.  0.3553

1.  1.36302573418054
2.  1.384
3.  0.666666666666667
4.  0.6569

(c) MLE 모수 추정<a href="#(c)-MLE-모수-추정" class="anchor-link">¶</a>
-----------------------------------------------------------------------

In \[68\]:

    steelfunc<-function(theta) {
      steel<-steel %>% mutate(Fx=1-exp(-((x-theta[1])/theta[2])^theta[3]),pitheta=Fx-lag(Fx))
      steel[1,"pitheta"]<-1-exp(-((steel[1,"x"]-theta[1])/theta[2])^theta[3])
      sum(steel$new*log(steel$pitheta))
    }

    cottonfunc<-function(theta) {
      cotton<-cotton %>% mutate(Fx=1-exp(-((x-theta[1])/theta[2])^theta[3]),pitheta=Fx-lag(Fx))
      cotton[1,"pitheta"]<-1-exp(-((cotton[1,"x"]-theta[1])/theta[2])^theta[3])
      sum(cotton$new*log(cotton$pitheta))
    }

In \[69\]:

    resultsteel<-optim(par=c(a_steel,b_steel,alpha_steel),steelfunc,
                       method="BFGS",control=list(fnscale=-1), hessian=T)
    mle_steel<-resultsteel$par

In \[70\]:

    resultcotton<-optim(par=c(a_cotton,b_cotton,alpha_cotton),cottonfunc,
                        method="BFGS",control=list(fnscale=-1), hessian=T)
    mle_cotton<-resultcotton$par

In \[71\]:

    resultsteel ; resultcotton

\$par  
1.  30.3064156334363
2.  5.9956035352172
3.  2.92251559498542

\$value  
-814.955731690513

\$counts  
function  
27

gradient  
6

\$convergence  
0

\$message  
NULL

\$hessian  
|            |           |           |
|------------|-----------|-----------|
| -100.41284 | -81.50218 | -16.89155 |
| -81.50218  | -89.83017 | 25.34906  |
| -16.89155  | 25.34906  | -76.53918 |

\$par  
1.  0.331802466562371
2.  4.03255123617932
3.  1.55940319223461

\$value  
-6566.49725737941

\$counts  
function  
27

gradient  
7

\$convergence  
0

\$message  
NULL

\$hessian  
|            |           |            |
|------------|-----------|------------|
| -1147.8180 | -392.6649 | -727.8400  |
| -392.6649  | -443.5842 | 315.4172   |
| -727.8400  | 315.4172  | -2107.5619 |

-   (b)는 Q-Q plot을 통해 적합한지 알 수 있다.
-   하지만 최적의 값을 찾을 때까지 반복작업을 해야하고, 방법1과 2처럼
    수식을 알고 있어야한다.

<!-- -->

-   c는 초기값만 설정하면 순식간에 최적값을 알려준다는 장점이 있다.
-   하지만 초기값에 따른 영향이 크고, 초기값을 구하는 과정에서 (b)와
    같은 과정이 필요할 수 있다.

(d) 모수들의 추정오차와 신뢰구간<a href="#(d)-모수들의-추정오차와-신뢰구간" class="anchor-link">¶</a>
-----------------------------------------------------------------------------------------------------

In \[72\]:

    # steel 
    J_steel<--resultsteel$hessian
    V_steel<-solve(J_steel)
    se_steel<-sqrt(diag(V_steel))
    CI_a_steel<-c(mle_steel[1]-1.96*se_steel[1],mle_steel[1]+1.96*se_steel[1])
    CI_b_steel<-c(mle_steel[2]-1.96*se_steel[2],mle_steel[2]+1.96*se_steel[2])
    CI_alpha_steel<-c(mle_steel[3]-1.96*se_steel[3],mle_steel[3]+1.96*se_steel[3])

In \[73\]:

    cat("CI.a: [", CI_a_steel[1], ",", CI_a_steel[2], "] \n")
    cat("CI.b: [", CI_b_steel[1], ",", CI_b_steel[2], "] \n") 
    cat("CI.alpha: [", CI_alpha_steel[1], ",", CI_alpha_steel[2], "] ")

    CI.a: [ 29.26368 , 31.34915 ] 
    CI.b: [ 4.859424 , 7.131783 ] 
    CI.alpha: [ 2.278516 , 3.566515 ] 

In \[74\]:

    # cotton 
    J_cotton<--resultcotton$hessian
    V_cotton<-solve(J_cotton)
    se_cotton<-sqrt(diag(V_cotton))
    CI_a_cotton<-c(mle_cotton[1]-1.96*se_cotton[1],mle_cotton[1]+1.96*se_cotton[1])
    CI_b_cotton<-c(mle_cotton[2]-1.96*se_cotton[2],mle_cotton[2]+1.96*se_cotton[2])
    CI_alpha_cotton<-c(mle_cotton[3]-1.96*se_cotton[3],mle_cotton[3]+1.96*se_cotton[3])

In \[75\]:

    cat("CI.a: [", CI_a_cotton[1], ",", CI_a_cotton[2], "] \n")
    cat("CI.b: [", CI_b_cotton[1], ",", CI_b_cotton[2], "] \n") 
    cat("CI.alpha: [", CI_alpha_cotton[1], ",", CI_alpha_cotton[2], "] ")

    CI.a: [ 0.2106497 , 0.4529552 ] 
    CI.b: [ 3.850353 , 4.214749 ] 
    CI.alpha: [ 1.480429 , 1.638377 ]